In [16]:
if __name__ == '__main__':
    %run ../global.ipynb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [17]:
from modules.utils import expand_cols_to_synonymns

# Map drugs to Pubchem CIDs by name

Although the names we have for the drugs have idiosyncrasies, for purposes of pubchem searching they may be fine because pubchem keeps lists of vendor-supplied identifiers, and our identifiers are from the vendors that supplied these drugs to SBP. 

In [18]:
strings_to_delete = [' (20:4 n-6)',
                    ', molecular biology grade',
                   ]
strings_to_replace = [('pi 3-k?', 'pi 3-kγ'),
              ('nf-?b', 'nf-κb'),
              ('?-catenin', 'ß-catenin'),
              ('acido ar?ndico', 'ácido arúndico'),
              ('gß? modulator ii, gallein', 'gßγ modulator ii, gallein'),
              ('?-secretase inhibitor', 'γ-secretase inhibitor'),
              ('pd\xa00332991\xa0(palbociclib)\xa0hcl', 'pd0332991 (palbociclib)'),
              ('z)-7-oxozeaenol', '(5z)-7-oxozeaenol'),
              ('flumethazone', 'flumethasone')
             ]

In [19]:
sbi_drugname_source_df = pd.read_excel(SBI_DRUGNAME_SOURCE_FILE, index_col=0, skiprows=3)
sbi_drugname_source_df.head()
sbi2drugnames = {}
for i, idx in enumerate(sbi_drugname_source_df.index):
    sbi = sbi_drugname_source_df.iloc[i].loc['Batch ID']
    if not isinstance(idx, float):
        if len(idx.strip()) > 0:
            names = idx.split('\n')
            names = [name.lower() for name in names if name not in ['n/a', 'nan', 'na', '']]
            fixed_names = []
            for name in names:
                name = re.sub(' {2,}', ' ', name)
                for s in strings_to_delete:
                    name = name.replace(s, '')
                for a, b in strings_to_replace:
                    name = name.replace(a, b)
                fixed_names.append(name)
            sbi2drugnames[sbi] = sorted(set(fixed_names))

In [20]:
json.dump({key: SYN_DELIM.join(val) for key, val in sbi2drugnames.items()}, open(SBI_TO_DRUGNAME_FILE, 'w'))

In [21]:
if os.path.exists(SBI_TO_CID_BY_NAME_FILE):
    sbi_to_cids_byname = json.load(open(SBI_TO_CID_BY_NAME_FILE, 'r'))
else:
    seen_names = {}
    sbi_to_cids_byname = defaultdict(set)
    for sbi, names in sbi2drugnames.items():
        for name in names:
            if name in seen_names:
                cids = seen_names[name]
            else:
                cids = pcp.get_cids(name)
            seen_names.add(name)
            sbi_to_cids_byname[sbi].update(cids)
    sbi_to_cids_byname = {key: sorted(val) for key, val in sbi_to_cids_byname.items() if len(val) > 0}
    json.dump(sbi_to_cids_byname, open(SBI_TO_CID_BY_NAME_FILE, 'w'))

In [22]:
sbi_to_cids_union = defaultdict(set)
for d in [{key: [val] for key, val in SBI_TO_CID_BY_STRUCTURE.items()}, sbi_to_cids_byname]:
    for sbi, cids in d.items():
        try:
            sbi_to_cids_union[sbi].update(list(map(int, cids)))
        except Exception as e:
            print(sbi, cids)
            raise e

json.dump({key: sorted(val) for key, val in sbi_to_cids_union.items()}, open(SBI_TO_CID_UNION_FILE, 'w'))

# Standardize and simplify drug names for mergining

Some downstream analyses will require merging screen entries for the same drug. This would ideally require only mapping to pubchem CID. However, two complications arise. One is that not all drugs map to pubchem, and yet may have duplicates in the screen called by different names. The only option there is to assign them the same name. Another complication is that two compounds may be technically, structurally different, but in a minor way that we consider inconsequential for its function: for example, if it is is a hydrochloride or acetate form of a drug. Unfortunately I am not an expert in the chemistry/pharmacology necessary to make some of these calls, but for some of them it is obvious based on different forms of the same drug within the screen itself.

## Merge based on pubchem CID

The `expand_cols_to_synonyms` function is useful for combining synonyms based on possibly transitive overlaps. It is made to operate on a list of dataframes. So I make an empty dataframe as a lazy hack to use this function rather than having to write a new one.

In [23]:
mock_df = pd.DataFrame(columns=sbi_drugname_source_df['Batch ID'])
syn_cols = []
for sbi in mock_df.columns:
    new_col_syns = [sbi]
    if sbi in sbi_to_cids_union:
        cids = sbi_to_cids_union[sbi]
        new_col_syns.extend(cids)
    syn_col = SYN_DELIM.join(map(str, new_col_syns))
    syn_cols.append(syn_col)
mock_df.columns = syn_cols
mock_df = expand_cols_to_synonymns([mock_df], syn_delim=SYN_DELIM)[0]

In [24]:
merged_syns = mock_df.columns.tolist()

In [25]:
sbi2drugname_merge1 = defaultdict(set)
for syn_string in merged_syns:
    sbis = tuple([syn for syn in syn_string.split(SYN_DELIM) if syn.startswith('SBI')])
    for sbi in sbis:
        if sbi in sbi2drugnames:
            sbi2drugname_merge1[sbis].update(sbi2drugnames[sbi])

In [26]:
sbi2drugname_merge1_one2one = {}
for sbis, names in sbi2drugname_merge1.items():
    name_str = SYN_DELIM.join(sorted(names)) # rather than having to choose a name, keep as synonyms
    for sbi in sbis:
        sbi2drugname_merge1_one2one[sbi] = name_str
json.dump(sbi2drugname_merge1_one2one, open(SBI_TO_DRUGNAME_MERGED_BY_CIDS_FILE, 'w'))

## Merged based on name

While not exhaustive, a simple approach that should catch many candidates for merging is to split each name into words, and see which words are found in the names of multiple compounds. Some of these will be the minor chemical modifiers (e.g. "hydrochloride", "acid"), but others will be names from which we will be able to tell that the different compounds are really the same clinical entity.

What I will do is print code to create a dictionary of words, with the names they occur in commented out. I will copy and paste this in a later cell, commment out the lines that don't seem to refer to a single clinical drug entity, and modify the dict values as needed (for example, adding back "acid"). I will then use it to merge compounds.

In [27]:
word2names = defaultdict(list)
for name in sorted(set(list(itertools.chain.from_iterable(list(sbi2drugname_merge1.values()))))):
    words = [re.sub('[\)\]]$', '', re.sub('^[\(\[]', '', w)) for w in re.split(',? ', name)]
    for word in words:
        word2names[word].append(name)

words_in_most_names = sorted(word2names.items(), reverse=True, key=lambda x: len(x[1]))
print('word2stdname = {')
for word, names in words_in_most_names:
    if len(names) > 1:
        print("'{}': '{}', #{}".format(word, word, names))
print('}')

word2stdname = {
'hydrochloride': 'hydrochloride', #['(+) -levobunolol hydrochloride', '(+)-butaclamol hydrochloride', '(+)-isocorydine hydrochloride', '(+)-norfenfluramine hydrochloride', '(+,-)-octopamine hydrochloride', '(-) -levobunolol hydrochloride', '(-)-isoproterenol hydrochloride', '(-)-tetramisole hydrochloride', '(6r)-5,6,7,8-tetrahydro-l-biopterin hydrochloride', '(r)-duloxetine hydrochloride', '(r)-propranolol hydrochloride', '(s)-glycyl-h-1152 (hydrochloride)', '(s)-h-1152 (hydrochloride)', '(s)-map4 hydrochloride', '(s)-propranolol hydrochloride', '(±)-2,3-dichloro-alpha-methylbenzylamine hydrochloride', '(±)-amt hydrochloride', '(±)-cgp-12177a hydrochloride', '(±)-epinephrine hydrochloride', '(±)-isoproterenol hydrochloride', '(±)-methoxyverapamil hydrochloride', '(±)-normetanephrine hydrochloride', '(±)-octopamine hydrochloride', '(±)-pd 128,907 hydrochloride', '(±)-ppht hydrochloride', '(±)-propranolol hydrochloride', '(±)-skf-38393 hydrochloride', '(±)-sotalol hydroc

Here it is pasted and modified. I commented out words to exclude (e.g. not merge based on them) and put "#?" for those exclusions I was less sure of.

In [28]:
word2stdname = {
#'hydrochloride': 'hydrochloride', #['(+) -levobunolol hydrochloride', '(+)-butaclamol hydrochloride', '(+)-isocorydine hydrochloride', '(+)-norfenfluramine hydrochloride', '(+,-)-octopamine hydrochloride', '(-) -levobunolol hydrochloride', '(-)-isoproterenol hydrochloride', '(-)-tetramisole hydrochloride', '(6r)-5,6,7,8-tetrahydro-l-biopterin hydrochloride', '(r)-duloxetine hydrochloride', '(r)-propranolol hydrochloride', '(s)-glycyl-h-1152 (hydrochloride)', '(s)-h-1152 (hydrochloride)', '(s)-map4 hydrochloride', '(s)-propranolol hydrochloride', '(±)-2,3-dichloro-alpha-methylbenzylamine hydrochloride', '(±)-amt hydrochloride', '(±)-cgp-12177a hydrochloride', '(±)-epinephrine hydrochloride', '(±)-isoproterenol hydrochloride', '(±)-methoxyverapamil hydrochloride', '(±)-normetanephrine hydrochloride', '(±)-octopamine hydrochloride', '(±)-pd 128,907 hydrochloride', '(±)-ppht hydrochloride', '(±)-propranolol hydrochloride', '(±)-skf-38393 hydrochloride', '(±)-sotalol hydrochloride', '(±)-threo-1-phenyl-2-decanoylamino-3-morpholino-1-propanol hydrochloride', '(±)-verapamil hydrochloride', '(±)-verapamil, hydrochloride', '(±)-vesamicol hydrochloride', '1,1-dimethylbiguanide, hydrochloride', '1,2-dimethylhydrazine hydrochloride', '1-(1-naphthyl)piperazine hydrochloride', '1-(2-methoxyphenyl)pierazine hydrochloride', '1-(2-methoxyphenyl)piperazine hydrochloride', '1-(m-chlorophenyl)-biguanide hydrochloride', '1-amino-1-cyclohexanecarboxylic acid hydrochloride', '1-aminocyclopropanecarboxylic acid hydrochloride', '1-deoxynojirimycin hydrochloride', '1-phenylbiguanide hydrochloride', '1s,2r-phenylpropanolamine hydrochloride', '2-cyclooctyl-2-hydroxyethylamine hydrochloride', '3-(1h-imidazol-4-yl)propyl di(p-fluorophenyl)methyl ether hydrochloride', '3-morpholinosydnonimine hydrochloride', '3-tropanyl-indole-3-carboxylate hydrochloride', '4-(2-aminoethyl)benzenesulfonyl fluoride hydrochloride', '4-amidinophenylmethanesulfonyl fluoride hydrochloride', '4-chlorophenylguanidine hydrochloride', '4-hydroxyphenethylamine hydrochloride', '4-imidazolemethanol hydrochloride', '4-methylpyrazole hydrochloride', '5-(n,n-dimethyl)amiloride hydrochloride', '5-aminopentanoic acid hydrochloride', '5-aminovaleric acid hydrochloride', '5-chloro-2-methyl-4-isothiazolin-3-one hydrochloride', '6-fluoronorepinephrine hydrochloride', '6-methyl-2-(phenylethynyl)pyridine hydrochloride', '9-amino-1,2,3,4-tetrahydroacridine hydrochloride', 'a-68930 hydrochloride', 'a-77636 hydrochloride', 'a3 hydrochloride', 'abt-418 hydrochloride', 'acebutolol hydrochloride', 'acecainide hydrochloride', 'aceclidine hydrochloride', 'acriflavinium hydrochloride', 'adiphenine hydrochloride', 'aklavine hydrochloride', 'alaproclate hydrochloride', 'alexidine hydrochloride', 'alfuzosin hydrochloride', 'alizapride hydrochloride', 'alosetron hydrochloride', 'alpha-lobeline hydrochloride', 'alpha-methyl-dl-tyrosine methyl ester hydrochloride', 'alprenolol hydrochloride', 'amantadine hydrochloride', 'ambroxol hydrochloride', 'amfepramone hydrochloride', 'amiloride hydrochloride', 'amiloride hydrochloride dihydrate', 'amiloride, hydrochloride', 'aminoguanidine hydrochloride', 'aminolevulinic acid hydrochloride', 'amiodarone hydrochloride', 'amiprilose hydrochloride', 'amitriptyline hydrochloride', 'amitryptiline hydrochloride', 'amperozide hydrochloride', 'amprolium hydrochloride', 'amsacrine hydrochloride', 'amyleine hydrochloride', 'anabasamine hydrochloride', 'anabasine hydrochloride', 'anagrelide hydrochloride', 'ancitabine hydrochloride', 'antazoline hydrochloride', 'antozoline hydrochloride', 'apomorphine hydrochloride', 'apomorphine hydrochloride hemihydrate', 'aprindine hydrochloride', 'arginine hydrochloride', 'articaine hydrochloride', 'atomoxetine hydrochloride', 'azelastine hydrochloride', 'bacampicillin hydrochloride', 'bambuterol hydrochloride', 'bay 61-3606 hydrochloride hydrate', 'benactyzine hydrochloride', 'benazepril hydrochloride', 'bendamustine hydrochloride', 'benfluorex hydrochloride', 'benidipine hydrochloride', 'benoxathian hydrochloride', 'benoxinate hydrochloride', 'benserazide hydrochloride', 'benzamidine hydrochloride', 'benzamil hydrochloride', 'benzydamine hydrochloride', 'bepridil hydrochloride', 'berbamine hydrochloride', 'bestatin hydrochloride', 'betahistine hydrochloride', 'betaine hydrochloride', 'betaxolol hydrochloride', 'betazole hydrochloride', 'bf-170 hydrochloride', 'biperiden hydrochloride', 'bisindolylmaleimide iii, hydrochloride', 'bisindolylmaleimide xi (hydrochloride)', 'brl 52537 hydrochloride', 'bromhexine hydrochloride', 'btcp hydrochloride', 'bu224 hydrochloride', 'buflomedil hydrochloride', 'bupivacaine hydrochloride', 'bupropion hydrochloride', 'buspirone hydrochloride', 'butenafine hydrochloride', 'canertinib (hydrochloride)', 'carnitine (dl) hydrochloride', 'caroverine hydrochloride', 'carteolol hydrochloride', 'cefepime hydrochloride', 'cefotiam hydrochloride', 'celiprolol hydrochloride', 'centrophenoxine hydrochloride', 'cetirizine hydrochloride', 'cgp-74514a hydrochloride', 'cgs-21680 hydrochloride', 'chlophendianol hydrochloride', 'chlorcyclizine hydrochloride', 'chlorguanide hydrochloride', 'chlormethiazole hydrochloride', 'chloroguanide hydrochloride', 'chloropyramine hydrochloride', 'chlorpromazine hydrochloride', 'chlorprothixene hydrochloride', 'chlortetracycline hydrochloride', 'ciprofloxacin hydrochloride hydrate', 'ciprofloxacin hydrochloride monohydrate', 'ciproxifan hydrochloride', 'cirazoline hydrochloride', 'clemizole hydrochloride', 'clenbuterol hydrochloride', 'clindamycin hydrochloride', 'clindamycin palmitate hydrochloride', 'clobutinol hydrochloride', 'clomipramine hydrochloride', 'clonidine hydrochloride', 'cloperastine hydrochloride', 'clorgiline hydrochloride', 'colesevalam hydrochloride (high mol wt copolymer @10mg/ml)', 'colforsin daropate hydrochloride', 'cyclizine hydrochloride', 'cyclobenzaprine hydrochloride', 'cyclopentolate hydrochloride', 'cyproheptadine hydrochloride', 'cysteamine hydrochloride', 'cytarabine hydrochloride', 'cytosine-1-beta-d-arabinofuranoside hydrochloride', 'daunorubicin hydrochloride', 'demeclocycline hydrochloride', 'desipramine hydrochloride', 'dexfenfluramine hydrochloride', 'dexpropranolol hydrochloride', 'diamthazole hydrochloride', 'dibenzepine hydrochloride', 'dibucaine hydrochloride', 'dicyclomine hydrochloride', 'dilazep hydrochloride', 'diltiazem hydrochloride', 'dimethisoquin hydrochloride', 'diperodon hydrochloride', 'diphenhydramine hydrochloride', 'diphenidol hydrochloride', 'diphenylpyraline hydrochloride', 'dipivefrin hydrochloride', 'dl-alpha-difluoromethylornithine hydrochloride', 'dl-p-chlorophenylalanine methyl ester hydrochloride', 'dobutamine hydrochloride', 'donepezil hydrochloride', 'dopamine hydrochloride', 'dorzolamide hydrochloride', 'dosulepin hydrochloride', 'doxapram hydrochloride', 'doxepin hydrochloride', 'doxorubicin hydrochloride', 'doxycycline hydrochloride', 'drofenine hydrochloride', 'duloxetine hydrochloride', 'dyclonine hydrochloride', 'efaroxan hydrochloride', 'ehna, hydrochloride', 'ephedrine (1r,2s) hydrochloride', 'epibestatin hydrochloride', 'epinastine hydrochloride', 'epirubicin hydrochloride', 'erlotinib hydrochloride', 'erythro-9-(2-hydroxy-3-nonyl)adenine hydrochloride', 'esmolol hydrochloride', 'etazolate hydrochloride', 'ethambutol hydrochloride', 'ethaverine hydrochloride', 'ethopropazine hydrochloride', 'ethylnorepinephrine hydrochloride', 'eticlopride hydrochloride', 'etilefrine hydrochloride', 'eucatropine hydrochloride', 'fadrozole hydrochloride', 'fendiline hydrochloride', 'fenspiride hydrochloride', 'fexofenadine hydrochloride', 'fiduxosin hydrochloride', 'fipexide hydrochloride', 'flavopiridol hydrochloride', 'flavoxate hydrochloride', 'flufenazine hydrochloride', 'flunarizine hydrochloride', 'fluoxetine hydrochloride', 'fluphenazine hydrochloride', 'fomepizole hydrochloride', 'furaltadone hydrochloride', 'fursultiamine hydrochloride', 'gabaculine hydrochloride', 'gaboxadol hydrochloride', 'gemcitabine hydrochloride', 'glafenine hydrochloride', 'glucosamine hydrochloride', 'gr 127935 hydrochloride hydrate', 'granisetron hydrochloride', 'guanfacine hydrochloride', 'guanidine hydrochloride', 'guvacine hydrochloride', 'gyki 52466 hydrochloride', 'gö 7874, hydrochloride', 'h-8 (hydrochloride)', 'h-9 (hydrochloride)', 'ha-1077 (hydrochloride)', 'halofantrine hydrochloride', 'harmalol hydrochloride', 'harmol hydrochloride', 'heptaminol hydrochloride', 'hexahydro-sila-difenidol hydrochloride, p-fluoro analog', 'hexylcaine hydrochloride', 'hydralazine hydrochloride', 'hydrastinine hydrochloride', 'hydrochloride, medetomidin', 'hydroxylamine hydrochloride', 'ici 204,448 hydrochloride', 'idazoxan hydrochloride', 'imid-4f hydrochloride', 'imidazol-4-ylacetic acid hydrochloride', 'imidazole-4-acetic acid hydrochloride', 'imiloxan hydrochloride', 'imipramine hydrochloride', 'indatraline hydrochloride', 'indole hydrochloride', 'iofetamine hydrochloride', 'irinotecan hydrochloride', 'isoguvacine hydrochloride', 'isoproterenol hydrochloride', 'isopyrin hydrochloride', 'isoquinoline, 6,7-dimethoxy-1-methyl-1,2,3,4-tetrahydro, hydrochloride', 'isothipendyl hydrochloride', 'isoxsuprine hydrochloride', 'ketamine hydrochloride', 'kn-92 (hydrochloride)', 'l(-)-vesamicol hydrochloride', 'l-3,4-dihydroxyphenylalanine methyl ester hydrochloride', 'l-687,384 hydrochloride', 'l-745,870 hydrochloride', 'l-glutamic acid hydrochloride', 'l-histidine hydrochloride', 'l-n6-(1-iminoethyl)lysine hydrochloride', 'labetalol hydrochloride', 'lazabemide hydrochloride', 'leelamine (hydrochloride)', 'lercanidipine hydrochloride hemihydrate', 'levalbuterol hydrochloride', 'levamisole hydrochloride', 'levobunolol hydrochloride', 'levocabastine hydrochloride', 'lidocaine hydrochloride', 'lidocaïne hydrochloride', 'lincomycin hydrochloride', 'lobelanidine hydrochloride', 'lobeline hydrochloride', 'lomefloxacin hydrochloride', 'loperamide hydrochloride', 'lp 12 hydrochloride hydrate', 'ly-294,002 hydrochloride', 'ly-310,762 hydrochloride', 'ly-317615 hydrochloride', 'mafenide hydrochloride', 'maprotiline hydrochloride', 'mebeverine hydrochloride', 'mecamylamine hydrochloride', 'mechlorethamine hydrochloride', 'meclizine hydrochloride', 'meclofenoxate hydrochloride', 'mecysteine hydrochloride', 'mefexamide hydrochloride', 'mefloquine hydrochloride', 'melphalan hydrochloride', 'memantine hydrochloride', 'mepivacaine hydrochloride', 'meprylcaine hydrochloride', 'meptazinol hydrochloride', 'mercaptamine hydrochloride', 'metatyramine hydrochloride', 'metformin hydrochloride', 'methacycline hydrochloride', 'methapyrilene hydrochloride', 'methoxamine hydrochloride', 'methoxyamine hydrochloride', 'methyl aminolevulinate hydrochloride', 'methyldopate hydrochloride', 'methylphenidate hydrochloride', 'metixene hydrochloride', 'metoclopramide hydrochloride', 'mexiletene hydrochloride', 'mexiletine hydrochloride', 'mianserin hydrochloride', 'mianserine hydrochloride', 'midodrine hydrochloride', 'milnacipran hydrochloride', 'minaprine hydrochloride', 'minocycline hydrochloride', 'minocycline, hydrochloride', 'mitoxanthrone hydrochloride', 'mitoxantrone hydrochloride', 'ml-7, hydrochloride', 'molindone hydrochloride', 'moricizine hydrochloride', 'moroxidine hydrochloride', 'moroxydine hydrochloride', 'moxifloxacin hydrochloride', 'moxisylyte hydrochloride', 'moxonidine hydrochloride', 'n-(3-trifluoromethylphenyl)piperazine hydrochloride', 'n-acetylprocainamide hydrochloride', 'n-carboxyheptylimidazole hydrochloride', 'n-methyldopamine hydrochloride', 'n^g,n^g-dimethylarginine hydrochloride', 'na-p-tosyl-l-lysine chloromethyl ketone hydrochloride', 'naftifine hydrochloride', 'nalbuphine hydrochloride', 'nalmefene hydrochloride', 'naloxone hydrochloride', 'naltrexone hydrochloride', 'naltrexone hydrochloride dihydrate', 'naltrindole hydrochloride', 'naphazoline hydrochloride', 'nefazodone hydrochloride', 'nefopam hydrochloride', 'ng-nitro-l-arginine methyl ester hydrochloride', 'nicardipine hydrochloride', 'nimustine hydrochloride', 'nisoxetine hydrochloride', 'nitrocaramiphen hydrochloride', 'no-711 hydrochloride', 'nordefrin  hydrochloride', 'nortriptyline hydrochloride', 'noscapine hydrochloride', 'ns8593 hydrochloride', 'nvp-aew541 (hydrochloride)', 'nylidrin hydrochloride', 'o-methylserotonin hydrochloride', 'octopamine hydrochloride', 'olprinone hydrochloride', 'ondansetron hydrochloride', 'ornithine hydrochloride', 'orphenadrine hydrochloride', 'oxidopamine hydrochloride', 'oxprenolol hydrochloride', 'oxymetazoline hydrochloride', 'oxyphencyclimine hydrochloride', 'ozagrel hydrochloride', 'p-aminoclonidine hydrochloride', 'p-iodoclonidine hydrochloride', 'palonosetron hydrochloride', 'papaverine hydrochloride', 'pargyline hydrochloride', 'paroxetine hydrochloride', 'paroxetine hydrochloride hemihydrate (mw = 374.83)', 'pelletierine hydrochloride', 'phenazopyridine hydrochloride', 'phenformin hydrochloride', 'phenoxybenzamine hydrochloride', 'phentermine hydrochloride', 'phentolamine hydrochloride', 'phenylephrine hydrochloride', 'phenylpropanolamine hydrochloride', 'pik-75 (hydrochloride)', 'pilocarpine hydrochloride', 'pioglitazone hydrochloride', 'piperidolate hydrochloride', 'pirenzepine hydrochloride', 'piribedil hydrochloride', 'pivmecillinam hydrochloride', 'pramoxine hydrochloride', 'prazosin hydrochloride', 'prilocaine hydrochloride', 'proadifen hydrochloride', 'procainamide hydrochloride', 'procaine hydrochloride', 'procarbazine hydrochloride', 'procyclidine hydrochloride', 'proguanil hydrochloride', 'promazine hydrochloride', 'promethazine hydrochloride', 'pronetalol hydrochloride', 'pronethalol hydrochloride', 'propafenone hydrochloride', 'proparacaine hydrochloride', 'propionylpromazine hydrochloride', 'propoxycaine hydrochloride', 'propranolol hydrochloride', 'propranolol hydrochloride (+/-)', 'protriptyline hydrochloride', 'protryptyline hydrochloride', 'pseudoephedrine hydrochloride', 'puromycin hydrochloride', 'pyridoxine hydrochloride', 'quinacrine hydrochloride', 'quinapril hydrochloride', 'quinidine hydrochloride monohydrate', 'r(+)-3ppp hydrochloride', 'r(+)-sch-23390 hydrochloride', 'r(-) apomorphine hydrochloride hemihydrate', 'r(-)-apocodeine hydrochloride', 'r(-)-propylnorapomorphine hydrochloride', 'r-(-)-desmethyldeprenyl hydrochloride', 'racephedrine hydrochloride', 'racepinephrine hydrochloride', 'raloxifene hydrochloride', 'ranitidine hydrochloride', 'rauwolscine hydrochloride', 'remoxipride hydrochloride', 'riluzole hydrochloride', 'rimantadine hydrochloride', 'ritodrine hydrochloride', 'ro 25-6981 hydrochloride', 'ropinirole hydrochloride', 'rosiglitazone hydrochloride', 'roxatidine acetate hydrochloride', 'rx 821002 hydrochloride', 's(-)-3ppp hydrochloride', 's(-)-uh-301 hydrochloride', 's(-)eticlopride hydrochloride', 's-(+)-fluoxetine hydrochloride', 's-(-)-eticlopride hydrochloride', 'sarafloxacin hydrochloride', 'sb 200646 hydrochloride', 'sb 203580 (hydrochloride)', 'sb 206553 hydrochloride', 'sb 269970 hydrochloride', 'scopolamine hydrochloride', 'sdz-205,557 hydrochloride', 'selegiline hydrochloride', 'semicarbazide hydrochloride', 'serotonin hydrochloride', 'sertraline hydrochloride', 'sibutramine hydrochloride', 'skf 89976a hydrochloride', 'skf-525a hydrochloride', 'skf-96365, hydrochloride', 'sotalol hydrochloride', 'spectinomycin hydrochloride', 'spiperone hydrochloride', 'tacrine hydrochloride', 'talampicillin hydrochloride', 'telenzepine hydrochloride', 'terazosin hydrochloride', 'terbinafine hydrochloride', 'tetracaine hydrochloride', 'tetracaïne hydrochloride', 'tetracycline hydrochloride', 'tetrahydrozoline hydrochloride', 'tetramisole hydrochloride', 'tetramizole hydrochloride', 'thiamine hydrochloride', 'thioridazine hydrochloride', 'thip hydrochloride', 'thonzylamine hydrochloride', 'tiapride hydrochloride', 'ticlopidine hydrochloride', 'tiletamine hydrochloride', 'tizanidine hydrochloride', 'tmb-8 hydrochloride', 'tmph hydrochloride', 'tocainide hydrochloride', 'todralazine hydrochloride', 'tolazoline hydrochloride', 'tolperisone hydrochloride', 'tomatidine hydrochloride', 'tomoxetine hydrochloride', 'topotecan hydrochloride', 'tosufloxacin hydrochloride', 'tracazolate hydrochloride', 'tramadol hydrochloride', 'tranylcypromine hydrochloride', 'trazodone hydrochloride', 'trequinsin hydrochloride', 'trichlormethine hydrochloride', 'trientine hydrochloride', 'trifluoperazine hydrochloride', 'trifluperidol hydrochloride', 'triflupromazine hydrochloride', 'trihexyphenidyl hydrochloride', 'trihexyphenidyl-d,l hydrochloride', 'trimethobenzamide hydrochloride', 'tripelennamine hydrochloride', 'triprolidine hydrochloride', 'tropisetron hydrochloride', 'tryptamine hydrochloride', 'tulobuterol hydrochloride', 'urapidil hydrochloride', 'valaciclovir hydrochloride', 'valacyclovir hydrochloride', 'vancomycin hydrochloride', 'vancomycin hydrochloride from streptomyces orientalis', 'vardenafil hydrochloride', 'venlafaxine hydrochloride', 'verapamil hydrochloride', 'vesamicol hydrochloride', 'vigabatrin hydrochloride', 'viloxazine hydrochloride', 'wb-4101 hydrochloride', 'whi-p180, hydrochloride', 'xylazine hydrochloride', 'xylometazoline hydrochloride', 'y-27632 (hydrochloride)', 'yohimbine hydrochloride', 'ys-035 hydrochloride', 'ziprasidone  hydrochloride', 'ziprasidone hydrochloride monohydrate', 'zm 39923 hydrochloride', 'zuclopenthixol hydrochloride']
#'acid': 'acid', #['(+)-gibberellic acid', '(+)-quisqualic acid', '(-)-perillic acid', '(e)-4-amino-2-butenoic acid', '(±)-2-amino-3-phosphonopropionic acid', '(±)-2-amino-4-phosphonobutyric acid', '(±)-2-amino-5-phosphonopentanoic acid', '(±)-2-amino-7-phosphonoheptanoic acid', '(±)-abscisic acid', '(±)-alpha-lipoic acid', '(±)-cis-piperidine-2,3-dicarboxylic acid', '(±)-ibotenic acid', '(±)-nipecotic acid', '(±)-vanillylmandelic acid', "1,3-naphthalenedisulfonic acid, 6,6'-((3,3'-dimethyl(1,1'-biphenyl)-4,4'-diyl)bis(azo))bis(4-amino-5-hydroxy)-, tetrasodium salt", '1-(4-chlorobenzyl)-5-methoxy-2-methylindole-3-acetic acid', '1-amino-1-cyclohexanecarboxylic acid hydrochloride', '1-aminocyclobutane carboxylic acid', '1-aminocyclopropanecarboxylic acid hydrochloride', '1-benzoyl-5-methoxy-2-methylindole-3-acetic acid', '1-naphthaleneacetic acid', '11-oxoursolic acid acetate', '12a-hydroxy-9-demethylmunduserone-8-carboxylic acid', '13-cis-retinoic acid', '18-glycyrrhetinic acid', '18alpha-glycyrrhetinic acid', '1h-imidazole-5-carboxylic acid, 1-(1-phenylethyl)-, ethyl ester, (r)- [cas]', '1h-indole-2-propanoic acid, 1-[(4-chlorophenyl)methyl]-3-[(1,1-dimethylethyl)thio]-alpha,alpha-dimethyl-5-(1-methylethyl)- [cas]', '2,4,5-trichlorophenoxyacetic acid', '2,4,5-trichlorophenoxyacetic acid, isooctyl ester', '2,4,5-trichlorophenoxyacetic acid, methyl ester', '2,4-dichlorophenoxyacetic acid', '2,4-dichlorophenoxyacetic acid, isooctyl ester', '2,4-dichlorophenoxyacetic acid, methyl ester', '2,4-dichlorophenoxybutyric acid', '2,4-dichlorophenoxybutyric acid, methyl ester', '2,4-pyridinedicarboxylic acid', '2-demethylapiolic acid', '2-hydroxy-4-methoxybenzophenone-5-sulfonic acid', '2-hydroxybenzoic acid, 2- hydroxyethyl ester', '2-isopropyl-3-methoxycinnamic acid', '3,4-dihydroxyphenylacetic acid', '3-(4-methoxyphenyl)-2-propenoic acid 3-methylbutyl ester', '3-amino-1-propanesulfonic acid sodium', '3-aminopropanesulphonic acid', '3-aminopropylphosphonic acid', '3-deoxy-3beta-hydroxyangolensic acid methyl ester', '3-methylorsellinic acid', '3-nitropropionic acid', '3alpha-hydroxy-3-deoxyangolensic acid methyl ester', '3beta-acetoxydeoxyangolensic acid, methyl ester', '4,6-dichloroindole-2-carboxylic acid', '4-aminosalicylic acid', '4-hydroxy-3-methoxyphenylacetic acid', '4-imidazoleacrylic acid', '4-naphthalimidobutyric acid', '4-thiazolidinecarboxylic acid, 2-oxo-, (r)- [cas]', '5,7-dichlorokynurenic acid', '5-amino-2-hydroxy-benzoic acid', '5-aminopentanoic acid hydrochloride', '5-aminovaleric acid hydrochloride', '5-chloroindole-2-carboxylic acid', '5-fluoroindole-2-carboxylic acid', '5-hydroxydecanoic acid sodium', '5-hydroxyindolacetic acid', '5-nitro-2-(3-phenylpropylamino)benzoic acid', '5-nitro-2-phenylpropylaminobenzoic acid [nppb]', '6,7-dichloro-3-hydroxy-2-quinoxalinecarboxylic acid', '6-acetoxy-angolensic acid methyl ester', '6-aminohexanoic acid', '6-hydroxyangolensic acid methyl ester', '7,7-dimethyl-(5z,8z)-eicosadienoic acid', '7,7-dimethyleicosadienoic acid', '7-chlorokynurenic acid', '7-deshydroxypyrogallin-4-carboxylic acid', '7beta-hydroxy-7-desacetoxykhivorinic acid, methyl ester', '8-hydroxycarapinic acid', '9-cis-retinoic acid', 'abietic acid', 'abscisic acid (cis,trans, )', 'abscisic acid (cis,trans; +/-)', 'acetohydroxamic acid', 'acetriazoic acid', 'acetyl isogambogic acid', 'acetyl-11-keto-ß-boswellic acid, boswellia serrata', 'acetylglutamic acid', 'acetylsalicylic acid', 'acetylsalicylsalicylic acid', 'acexamic acid', 'aconitic acid', 'adipic acid', 'all trans retinoic acid', 'alpha-cyano-3-hydroxycinnamic acid', 'alpha-cyano-4-hydroxycinnamic acid', 'alpha-glycerophosphoric acid', 'alpha-guanidinoglutaric acid', 'alpha-keto-glutaric acid, sodium salt', 'aminocaproic acid', 'aminocyclopropanecarboxylic acid', 'aminohippuric acid', 'aminohydroxybutyric acid', 'aminolevulinic acid hydrochloride', 'anacardic acid', 'angolensic acid, methyl ester', 'anhydrobrazilic acid', 'anthracene-9-carboxylic acid', 'aphyllic acid', 'arachidonic acid', 'arachidonic acid (20:4 n-6)', 'arachidonic acid leelamide', 'aristolochic acid', 'aristolochic acid a', 'arsanilic acid', 'ascorbic acid', 'aspergillic acid', 'aurintricarboxylic acid', 'azelaic acid', 'benzeneacetic acid, 2-[(2,6-dichlorophenyl)amino]-, monosodium salt [cas]', 'benzeneacetic acid, alpha-(hydroxymethyl)-, 9-methyl-3-oxa-9-azatricyclo[3.3.1.02,4]non-7-yl ester', 'benzoic acid', 'beta-bromopicrotoxinic acid', 'betulinic acid', 'bovinocidin (3-nitropropionic acid)', 'byssochlamic acid', 'cacodylic acid', 'caffeic acid', 'caffeic acid phenethyl ester', 'canrenoic acid potassium salt', 'canrenoic acid, potassium salt', 'cantharidic acid', 'capobenic acid', 'carminic acid', 'carnosic acid', 'cephalosporanic acid, 7-amino', 'chaulmoogric acid', 'chaulmoogric acid, ethyl ester', 'chelidamic acid', 'chlorogenic acid', 'chol-11-enic acid', 'cholic acid', 'cholic acid, methyl ester', 'chrysanthemic acid, ethyl ester', 'cis-4-aminocrotonic acid', 'clodronic acid', 'clofibric acid', 'coumarinic acid methyl ether', 'cyclopiazonic acid', 'cyclopiazonic acid, penicillium cyclopium', 'd(-)-2-amino-5-phosphonopentanoic acid', 'deacetoxy(7)-7-oxokhivorinic acid', 'decahydrogambogic acid', 'dehydro(11,12)ursolic acid lactone', 'dehydrocholic acid', 'deoxycholic acid', 'desacetyl(7)khivorinic acid, methyl ester', 'diatrizoic acid', 'diatrizoic acid dihydrate', 'dichloroacetic acid', 'diclofenac acid', 'dideacetyl(1,3)-7-desacetoxy-7-oxo-deoxykhivorinic acid', 'diethylenetriaminepentaacetic acid', 'diffratic acid', 'dihydro-beta-tubaic acid', 'dihydrogambogic acid', 'dihydrogedunic acid, methyl ester', 'dihydrogeduninic acid, methyl ester', 'dihydrojasmonic acid', 'dihydrokainic acid', 'dimethylcaffeic acid', 'djenkolic acid', 'dl-a-lipoic acid', 'dl-threo-beta-hydroxyaspartic acid', 'echinocystic acid', 'ellagic acid', 'ellagic acid, dihydrate', 'emodic acid', 'epitheaflavic acid', 'ethacrynic acid', 'etidronic acid, disodium salt', 'eudesmic acid', 'evernic acid', 'everninic acid', 'ferulic acid', 'flufenamic acid', 'folic acid', 'folinic acid calcium salt', 'fumarprotocetraric acid', 'fusaric acid', 'fusidic acid', 'fusidic acid sodium', 'fusidic acid sodium salt', 'gadoteric acid', 'gallic acid', 'gambogic acid', 'gambogic acid amide', 'gambogic acid morpholine adduct', 'gambogic acid, garcinia hanburyi', 'gamboginic acid, methyl ester', 'gamma-aminobutyric acid', 'garcinolic acid', 'gentisic acid', 'gibberellic acid', 'gluconic acid', 'glucosaminic acid', 'glucuronic acid', 'glycyrrhizic acid', 'gossypol-acetic acid complex', 'griseofulvic acid', 'gyrophoric acid', 'haematommic acid, ethyl ester', 'hexadecylcitric acid', 'hopantenic acid', 'hydroxytoluic acid', 'hypoprotocetraric acid', 'imidazol-4-ylacetic acid hydrochloride', 'imidazole-4-acetic acid hydrochloride', 'indole-2-carboxylic acid', 'iocetamic acid', 'iopanic acid', 'iopanoic acid', 'ioxaglic acid', 'isoferulic acid', 'isonipecotic acid', 'kainic acid', 'kainic acid, natural', 'kojic acid', 'kynurenic acid', 'l-2-aminoadipic acid', 'l-aspartic acid', 'l-azetidine-2-carboxylic acid', 'l-cysteinesulfinic acid', 'l-glutamic acid hydrochloride', 'l-glutamic acid, n-[4-[[(2,4-diamino-6-pteridinyl)methyl]methylamino]benzoyl]- [cas]', 'l-glutamic acid, n-phthaloyl-', 'larixinic acid', 'lauric acid diethanolamide', 'lauric acid leelamide', 'lecanoric acid', 'levulinic acid', 'levulinic acid,  3-benzylidenyl-', 'linoleic acid', 'lithochol-11-enic acid', 'lithocholic acid', 'lobaric acid', 'loganic acid', 'lonchocarpic acid', 'lonchocarpic acid acetate', 'mandelic acid, methyl ester', 'mefenamic acid', 'methylorsellinic acid, ethyl ester', 'metiazinic acid', 'mevalonic-d, l acid lactone', 'minodronic acid', 'mucic acid', 'mycophenolic acid', 'n-acetylaspartic acid', 'n-acetylneuramic acid', 'n-methyl-d-aspartic acid', 'n-methylanthanilic acid', 'n-pentanoic acid', 'n-phenylanthranilic acid', 'nalidixic acid', 'nalidixic acid sodium', 'nalidixic acid sodium salt', 'nicotinic acid', 'niflumic acid', 'nipecotic acid', 'nonic acid', 'nordihydroguaiaretic acid', 'nordihydroguaiaretic acid from larrea divaricata (creosote bush)', 'nordihydroguaretic acid', 'norstictic acid', 'norstictic acid pentaacetate', 'okadaic acid, prorocentrum sp.', 'oleananoic acid acetate', 'oleanoic acid', 'oleanolic acid', 'oleanolic acid acetate', 'oleic acid', 'orotic acid', 'orsellinic acid', 'orsellinic acid, ethyl ester', 'orthothymotinic acid', 'oxiranecarboxylic acid, 2-[6-(4-chlorophenoxy)hexyl]-, ethyl ester- [cas]', 'oxolinic acid', 'p-hydroxybenzoic acid propyl ester', 'pamidronic acid', 'pentetic acid', 'perillic acid', 'perillic acid (-)', 'phenoxyacetic acid', 'phenylacetohydroxamic acid', 'phenylbenzene-omega-phosphono-alpha-amino acid', 'phenylbutyric acid', 'phosphonoacetic acid', 'pipemidic acid', 'piromidic acid', 'piscidic acid', 'psoromic acid', 'psoromic acid hydrate', 'pterin-6-carboxylic acid', 'pyridine-3-carboxylic acid benzyl ester', 'pyrrolidone carboxylic acid, pidolic acid', 'pyrrolidone carboxylic acid, pidolic acid', 'quinic acid', 'quinolinic acid', 'quisqualic acid', 'ranelic acid', 'retinoic acid', 'retinoic acid p-hydroxyanilide', 'retinoic acid, all trans', 'risedronic acid monohydrate', 'robustic acid', 'robustic acid methyl ether', 'roccellic acid', 'rosmarinic acid', 'rosolic acid', 'rotenonic acid', 'rotenonic acid, methyl ether', 'shikimic acid', 'sinapic acid', 'sorbic acid', 'spaglumic acid', 'stearic acid', 'stictic acid', 'strophanthidinic acid lactone acetate', 'suberoyl bis-hydroxamic acid', 'syringic acid', 'tannic acid', 'tetradecanoic acid 1-methylethyl ester', 'tetradecylthioacetic acid', 'tetrahydrogambogic acid', 'thioctic acid', 'tiaprofenic acid', 'tiludronic acid', "tiratricol, 3,3',5-triiodothyroacetic acid", 'tolfenamic acid', 'totarol-19-carboxylic acid, methyl ester', 'tranexamic acid', 'trans-retinoic acid', 'tubaic acid', 'undecylenic acid', 'undecylenic acid monoethanolamine sulfosuccinate sodium', 'ursinic acid', 'ursocholanic acid', 'ursolic acid', 'ursolic acid acetate', 'usnic acid', 'valproic acid', 'valproic acid (sodium salt)', 'valproic acid hydroxamate', 'valproic acid sodium', 'valproic acid sodium salt (sodium valproate)', 'valproic acid, sodium salt', 'vanillic acid diethylamide', 'veratric acid', 'vulpinic acid', 'xanthurenic acid', 'yohimbic acid', 'zoledronic acid']
#'inhibitor': 'inhibitor', #['?-catenin/tcf inhibitor, fh535', '?-secretase inhibitor ix', '?-secretase inhibitor vi', '?-secretase inhibitor xxi, compound e', 'adenosine kinase inhibitor', 'adenylyl cyclase type v inhibitor, nky80', 'akt inhibitor iv', 'akt inhibitor v, triciribine', 'akt inhibitor viii, isozyme-selective, akti-1/2', 'akt inhibitor x', 'akt inhibitor xii, isozyme-selective, akti-2', 'akt inhibitor x·hcl', 'ampk inhibitor,                  compound c', 'angiogenesis inhibitor', 'apoptosis inhibitor', 'apoptosis inhibitor ii, ns3694', 'atm kinase inhibitor', 'atm/atr kinase inhibitor', 'aurora a inhibitor i', 'aurora kinase inhibitor ii', 'aurora kinase inhibitor iii', 'aurora kinase/cdk inhibitor', 'autophagy inhibitor, 3-ma', 'bcr-abl inhibitor', "bruton's tyrosine kinase inhibitor iii", 'c-met/ron dual kinase inhibitor', 'c-myc inhibitor', 'camkii inhibitor, ck59', 'casein kinase i inhibitor, d4476', 'casein kinase ii inhibitor i', 'casein kinase ii inhibitor iii, tbca', 'casein kinase ii inhibitor iv', 'cbfß inhibitor', 'cdc2-like kinase inhibitor, tg003', 'cdc7/cdk9 inhibitor', 'cdk inhibitor, p35', 'cdk/crk inhibitor', 'cdk1 inhibitor', 'cdk1 inhibitor iv, ro-3306', 'cdk1 inhibitor,                    cgp74514a', 'cdk1/2 inhibitor iii', 'cdk1/5 inhibitor', 'cdk2 inhibitor ii', 'cdk2 inhibitor iii', 'cdk2 inhibitor iv,                 nu6140', 'cdk2/5 inhibitor', 'cdk2/9 inhibitor', 'cdk4 inhibitor', 'cdk4 inhibitor ii,                 nsc 625987', 'cdk4 inhibitor iii', 'cdk9 inhibitor ii', 'cell sheet migration inhibitor, locostatin', 'cfms receptor tyrosine kinase inhibitor', 'chk2 inhibitor', 'chk2 inhibitor ii', 'cox-1 inhibitor iv, tfap', 'cox-1 inhibitor, fr122047', 'cox-2 inhibitor ii', 'crac channel inhibitor, btp2', 'diacylglycerol kinase inhibitor ii', 'dna base excision repair pathway inhibitor', 'dna methyltransferase inhibitor', 'dna-pk inhibitor ii', 'dna-pk inhibitor iii', 'dna-pk inhibitor v', 'dynamin inhibitor i, dynasore', 'e2f inhibitor, hlm006474', 'eef-2 kinase inhibitor, nh125', 'egfr inhibitor', 'egfr/erbb-2 inhibitor', 'egfr/erbb-2/erbb-4 inhibitor', 'eif-2a inhibitor ii, sal003', 'erk inhibitor ii', 'erk inhibitor ii,                   negative control', 'erk inhibitor ii, fr180204', 'erk inhibitor iii', 'fgf receptor tyrosine kinase inhibitor', 'fgf/vegf receptor tyrosine kinase inhibitor, pd173074', 'flt-3 inhibitor', 'flt-3 inhibitor ii', 'flt-3 inhibitor iii', 'glycogen phosphorylase inhibitor', 'gsk-3 inhibitor ix', 'gsk-3 inhibitor ix, control, mebio', 'gsk-3 inhibitor x', 'gsk-3 inhibitor xiii', 'gsk-3b inhibitor i', 'gsk-3b inhibitor ii', 'gsk-3b inhibitor viii', 'gsk-3b inhibitor xi', 'gsk-3ß inhibitor xii, tws119', 'gsk3b inhibitor xii, tws119', 'hdm2 e3 ligase inhibitor', 'heat shock protein inhibitor i', 'hif-1 inhibitor', 'histone acetyltransferase inhibitor ii', 'histone deacetylase inhibitor iv', 'histone deacetylase inhibitor vi, hnha', 'histone lysine methyltransferase inhibitor', 'hsp90 inhibitor, cct018159', 'igf-1r inhibitor ii', 'ikk inhibitor vii', 'ikk inhibitor x', 'ikk-2 inhibitor iv', 'ikk-2 inhibitor v', 'ikk-2 inhibitor vi', 'ikk-2 inhibitor viii', 'ikk-2 inhibitor xi', 'ikk-3 inhibitor ix', 'interleukin-1 receptor-associated-kinase-1/4 inhibitor', 'ip3k inhibitor', 'irak-1/4 inhibitor', 'irak-1/4 inhibitor i', 'jak inhibitor i', 'jak2 inhibitor ii', 'jak2 inhibitor iii, sd-1029', 'jak3 inhibitor ii', 'jak3 inhibitor iv', 'jak3 inhibitor vi', 'jnk inhibitor ii', 'jnk inhibitor ix', 'jnk inhibitor v', 'jnk inhibitor viii', 'jnk inhibitor xi, bi-87g3', 'jnk inhibitor,                    negative control', 'lck inhibitor', 'luciferase inhibitor i', 'luciferase inhibitor ii', 'm-344, histone deacetylase inhibitor iii', 'mek inhibitor i', 'mek inhibitor ii', 'mek1/2 inhibitor', 'mek1/2 inhibitor ii', 'met kinase inhibitor', 'met/flt-3/vegfr2 tyrosine kinase inhibitor', 'mitochondrial division inhibitor, mdivi-1', 'mk-2 inhibitor iii', 'mk2a inhibitor', 'mnk1 inhibitor', 'na+/h+ exchanger isoform-1 inhibitor', 'nf-?b activation inhibitor ii, jsh-23', 'nf-?b activation inhibitor iii', 'nf-?b activation inhibitor iv', 'nf-?b activation inhibitor v, 5hpp-33', 'nf-kb activation inhibitor', 'nfat activation inhibitor iii', 'p21-activated kinase inhibitor iii, ipa-3', 'p38 map kinase inhibitor', 'p38 map kinase inhibitor iii', 'p38 map kinase inhibitor iv', 'p38 map kinase inhibitor v', 'p38 map kinase inhibitor vi, jx401', 'p38 map kinase inhibitor vii, sd-169', 'p38 map kinase inhibitor viii', 'parp inhibitor i, 3-aba', 'parp inhibitor vi, nu1025', 'parp inhibitor viii, pj34', 'parp inhibitor xi, dr2313', 'parp inhibitor xii', 'pdgf receptor tyrosine kinase inhibitor ii', 'pdgf receptor tyrosine kinase inhibitor iii', 'pdgf receptor tyrosine kinase inhibitor iv', 'pdgf rtk inhibitor', 'pdk1/akt/flt dual pathway inhibitor', 'pi 3-k  inhibitor', 'pi 3-k  inhibitor ii', 'pi 3-k  inhibitor iv', 'pi 3-k  inhibitor viii', 'pi 3-k? inhibitor vii', 'pi 3-k?/ckii inhibitor', 'pi 3-ka inhibitor iv', 'pi 3-ka inhibitor viii', 'pi 3-kb inhibitor ii', 'pi 3-kg inhibitor', 'pi 3-kß inhibitor vi, tgx-221', 'pi3-kinase .alpha. inhibitor 2', 'pikfyve inhibitor', 'pim1 kinase inhibitor ii', 'pim1 kinase inhibitor iv', 'pim1/2 kinase inhibitor v', 'pim1/2 kinase inhibitor vi', 'pkcb inhibitor', 'pkcbii/egfr inhibitor', 'pkr inhibitor', 'pkr inhibitor, negative control', 'polo-like kinase inhibitor i', 'polo-like kinase inhibitor ii, bto-1', 'ppiase-parvulin inhibitor', 'prl-3 inhibitor', 'protein methyltransferase inhibitor, ami-5', 'ptk/pi 3-k/mtor inhibitor, pp121', 'ptp inhibitor v, phps1', 'ptp1b inhibitor', 'raf kinase inhibitor iv', 'raf kinase inhibitor v', 'raf1 kinase inhibitor i', 'ras-net (elk-3) pathway inhibitor, xrp44x', 'rho kinase inhibitor', 'rho kinase inhibitor ii', 'rho kinase inhibitor iii, rockout', 'rho kinase inhibitor iv', 'rho kinase inhibitor v', 'rna polymerase iii inhibitor', 'rock inhibitor, y-27632', 'rsk inhibitor, sl0101', 'sfrp-1 inhibitor', 'shp1/2 ptpase inhibitor, nsc-87877', 'sirt1 inhibitor iii', 'sirt1 inhibitor iv, (s)-35', 'sirt1/2 inhibitor iv, cambinol', 'sirt1/2 inhibitor viii, salermide', 'sirt2 inhibitor, agk2', 'sphingosine kinase inhibitor', 'sphingosine kinase inhibitor 2', 'src kinase inhibitor i', 'stat3 inhibitor iii, wp1066', 'stat3 inhibitor v, stattic', 'stat3 inhibitor vii', 'stat5 inhibitor', 'stat5 inhibitor iii, pimozide', 'stem-cell factor/c-kit inhibitor, isck03', 'syk inhibitor', 'syk inhibitor ii', 'syk inhibitor iii', 'telomerase inhibitor ix', 'tgf-b ri inhibitor iii', 'tgf-b ri kinase inhibitor', 'tie2 kinase inhibitor', 'tnf-a inhibitor', 'tpl2 kinase inhibitor', 'tpl2 kinase inhibitor ii', 'uch-l1 inhibitor', 'uch-l1 inhibitor ii', 'uch-l3 inhibitor', 'vegf receptor 2 kinase inhibitor i', 'vegf receptor 2 kinase inhibitor ii', 'vegf receptor 2 kinase inhibitor iii', 'vegf receptor 2 kinase inhibitor iv', 'vegf receptor tyrosine kinase inhibitor ii', 'vegfr tyrosine kinase inhibitor v', 'vegfr2 kinase inhibitor vi, ki8751', 'vegfr2/flt3/c-kit inhibitor', 'wee1 inhibitor', 'wee1 inhibitor ii', 'wee1/chk1 inhibitor', 'ß-catenin/tcf inhibitor, fh535', 'ß-secretase inhibitor iv', 'ßark1 inhibitor']
#'': '', #['abscisic acid (cis,trans, )', 'alsterpaullone,                   2-cyanoethyl', 'alsterpaullone,                   2-cyanoethyl', 'alsterpaullone,                   2-cyanoethyl', 'alsterpaullone,                   2-cyanoethyl', 'alsterpaullone,                   2-cyanoethyl', 'alsterpaullone,                   2-cyanoethyl', 'alsterpaullone,                   2-cyanoethyl', 'alsterpaullone,                   2-cyanoethyl', 'alsterpaullone,                   2-cyanoethyl', 'alsterpaullone,                   2-cyanoethyl', 'alsterpaullone,                   2-cyanoethyl', 'alsterpaullone,                   2-cyanoethyl', 'alsterpaullone,                   2-cyanoethyl', 'alsterpaullone,                   2-cyanoethyl', 'alsterpaullone,                   2-cyanoethyl', 'alsterpaullone,                   2-cyanoethyl', 'alsterpaullone,                   2-cyanoethyl', 'alsterpaullone,                   2-cyanoethyl', 'ampk inhibitor,                  compound c', 'ampk inhibitor,                  compound c', 'ampk inhibitor,                  compound c', 'ampk inhibitor,                  compound c', 'ampk inhibitor,                  compound c', 'ampk inhibitor,                  compound c', 'ampk inhibitor,                  compound c', 'ampk inhibitor,                  compound c', 'ampk inhibitor,                  compound c', 'ampk inhibitor,                  compound c', 'ampk inhibitor,                  compound c', 'ampk inhibitor,                  compound c', 'ampk inhibitor,                  compound c', 'ampk inhibitor,                  compound c', 'ampk inhibitor,                  compound c', 'ampk inhibitor,                  compound c', 'ampk inhibitor,                  compound c', 'cdk1 inhibitor,                    cgp74514a', 'cdk1 inhibitor,                    cgp74514a', 'cdk1 inhibitor,                    cgp74514a', 'cdk1 inhibitor,                    cgp74514a', 'cdk1 inhibitor,                    cgp74514a', 'cdk1 inhibitor,                    cgp74514a', 'cdk1 inhibitor,                    cgp74514a', 'cdk1 inhibitor,                    cgp74514a', 'cdk1 inhibitor,                    cgp74514a', 'cdk1 inhibitor,                    cgp74514a', 'cdk1 inhibitor,                    cgp74514a', 'cdk1 inhibitor,                    cgp74514a', 'cdk1 inhibitor,                    cgp74514a', 'cdk1 inhibitor,                    cgp74514a', 'cdk1 inhibitor,                    cgp74514a', 'cdk1 inhibitor,                    cgp74514a', 'cdk1 inhibitor,                    cgp74514a', 'cdk1 inhibitor,                    cgp74514a', 'cdk1 inhibitor,                    cgp74514a', 'cdk2 inhibitor iv,                 nu6140', 'cdk2 inhibitor iv,                 nu6140', 'cdk2 inhibitor iv,                 nu6140', 'cdk2 inhibitor iv,                 nu6140', 'cdk2 inhibitor iv,                 nu6140', 'cdk2 inhibitor iv,                 nu6140', 'cdk2 inhibitor iv,                 nu6140', 'cdk2 inhibitor iv,                 nu6140', 'cdk2 inhibitor iv,                 nu6140', 'cdk2 inhibitor iv,                 nu6140', 'cdk2 inhibitor iv,                 nu6140', 'cdk2 inhibitor iv,                 nu6140', 'cdk2 inhibitor iv,                 nu6140', 'cdk2 inhibitor iv,                 nu6140', 'cdk2 inhibitor iv,                 nu6140', 'cdk2 inhibitor iv,                 nu6140', 'cdk4 inhibitor ii,                 nsc 625987', 'cdk4 inhibitor ii,                 nsc 625987', 'cdk4 inhibitor ii,                 nsc 625987', 'cdk4 inhibitor ii,                 nsc 625987', 'cdk4 inhibitor ii,                 nsc 625987', 'cdk4 inhibitor ii,                 nsc 625987', 'cdk4 inhibitor ii,                 nsc 625987', 'cdk4 inhibitor ii,                 nsc 625987', 'cdk4 inhibitor ii,                 nsc 625987', 'cdk4 inhibitor ii,                 nsc 625987', 'cdk4 inhibitor ii,                 nsc 625987', 'cdk4 inhibitor ii,                 nsc 625987', 'cdk4 inhibitor ii,                 nsc 625987', 'cdk4 inhibitor ii,                 nsc 625987', 'cdk4 inhibitor ii,                 nsc 625987', 'cdk4 inhibitor ii,                 nsc 625987', 'erk inhibitor ii,                   negative control', 'erk inhibitor ii,                   negative control', 'erk inhibitor ii,                   negative control', 'erk inhibitor ii,                   negative control', 'erk inhibitor ii,                   negative control', 'erk inhibitor ii,                   negative control', 'erk inhibitor ii,                   negative control', 'erk inhibitor ii,                   negative control', 'erk inhibitor ii,                   negative control', 'erk inhibitor ii,                   negative control', 'erk inhibitor ii,                   negative control', 'erk inhibitor ii,                   negative control', 'erk inhibitor ii,                   negative control', 'erk inhibitor ii,                   negative control', 'erk inhibitor ii,                   negative control', 'erk inhibitor ii,                   negative control', 'erk inhibitor ii,                   negative control', 'erk inhibitor ii,                   negative control', 'ha 1077, dihydrochloride                              fasudil', 'ha 1077, dihydrochloride                              fasudil', 'ha 1077, dihydrochloride                              fasudil', 'ha 1077, dihydrochloride                              fasudil', 'ha 1077, dihydrochloride                              fasudil', 'ha 1077, dihydrochloride                              fasudil', 'ha 1077, dihydrochloride                              fasudil', 'ha 1077, dihydrochloride                              fasudil', 'ha 1077, dihydrochloride                              fasudil', 'ha 1077, dihydrochloride                              fasudil', 'ha 1077, dihydrochloride                              fasudil', 'ha 1077, dihydrochloride                              fasudil', 'ha 1077, dihydrochloride                              fasudil', 'ha 1077, dihydrochloride                              fasudil', 'ha 1077, dihydrochloride                              fasudil', 'ha 1077, dihydrochloride                              fasudil', 'ha 1077, dihydrochloride                              fasudil', 'ha 1077, dihydrochloride                              fasudil', 'ha 1077, dihydrochloride                              fasudil', 'ha 1077, dihydrochloride                              fasudil', 'ha 1077, dihydrochloride                              fasudil', 'ha 1077, dihydrochloride                              fasudil', 'ha 1077, dihydrochloride                              fasudil', 'ha 1077, dihydrochloride                              fasudil', 'ha 1077, dihydrochloride                              fasudil', 'ha 1077, dihydrochloride                              fasudil', 'ha 1077, dihydrochloride                              fasudil', 'ha 1077, dihydrochloride                              fasudil', 'ha 1077, dihydrochloride                              fasudil', 'jnk inhibitor,                    negative control', 'jnk inhibitor,                    negative control', 'jnk inhibitor,                    negative control', 'jnk inhibitor,                    negative control', 'jnk inhibitor,                    negative control', 'jnk inhibitor,                    negative control', 'jnk inhibitor,                    negative control', 'jnk inhibitor,                    negative control', 'jnk inhibitor,                    negative control', 'jnk inhibitor,                    negative control', 'jnk inhibitor,                    negative control', 'jnk inhibitor,                    negative control', 'jnk inhibitor,                    negative control', 'jnk inhibitor,                    negative control', 'jnk inhibitor,                    negative control', 'jnk inhibitor,                    negative control', 'jnk inhibitor,                    negative control', 'jnk inhibitor,                    negative control', 'jnk inhibitor,                    negative control', 'levulinic acid,  3-benzylidenyl-', 'nordefrin  hydrochloride', 'pi 3-k  inhibitor', 'pi 3-k  inhibitor ii', 'pi 3-k  inhibitor iv', 'pi 3-k  inhibitor viii', 'ziprasidone  hydrochloride']
#'sodium': 'sodium', #['(-)-naproxen sodium', '(r) -naproxen sodium salt', '1,3,5(10)-estratrien-3-ol-17-one sulphate, sodium salt', '3-amino-1-propanesulfonic acid sodium', '5-hydroxydecanoic acid sodium', '8-(4-chlorophenylthio)-camp sodium', '8-bromo-camp sodium', '8-bromo-cgmp sodium', 'alendronate sodium', 'alpha-keto-glutaric acid, sodium salt', 'aminosalicylate sodium', 'amobarbital sodium', 'ampicillin sodium', 'androsterone sodium sulfate', 'azlocillin sodium', 'azlocillin sodium salt', 'balsalazide sodium', 'benzylpenicillin sodium', 'betamethasone sodium phosphate', 'bithionate sodium', 'brl 37344 sodium', 'bucladesine sodium salt', 'carbenoxolone sodium', 'cardiolipin sodium', 'cefamandole sodium', 'cefamandole sodium salt', 'cefazolin sodium', 'cefazolin sodium salt', 'cefmetazole sodium', 'cefmetazole sodium salt', 'cefonicid sodium', 'cefoperazole sodium', 'cefotaxime sodium', 'cefotaxime sodium salt', 'cefoxitin sodium', 'cefoxitin sodium salt', 'cefsulodin sodium', 'cefsulodin sodium salt', 'cefsulodin sodium salt hydrate', 'ceftriaxone sodium', 'ceftriaxone sodium trihydrate', 'cefuroxime sodium', 'cefuroxime sodium salt', 'cephalosporin c sodium', 'cephalothin sodium', 'cephalothin sodium salt', 'cephapirin sodium', 'cephradine sodium', 'chloramphenicol sodium succinate', 'cinnamate sodium', 'cloxacillin sodium', 'cloxacillin sodium salt', 'colistimethate sodium', 'cromolyn sodium', 'dantrolene sodium', 'dantrolene sodium salt', 'dehydrocholate sodium', 'delta1-hydrocortisone 21-hemisuccinate sodium salt', 'dexamethasone sodium phosphate', 'diclofenac sodium', 'dicloxacillin sodium', 'dicloxacillin sodium salt hydrate', 'erythrosine sodium', 'estradiol-3-sulfate, sodium salt', 'estramustine phosphate sodium', 'flucloxacillin sodium', 'fluvastatin sodium salt', 'foscarnet sodium', 'fosinopril sodium', 'furegrelate sodium', 'fusidic acid sodium', 'fusidic acid sodium salt', 'hydrocortisone sodium phosphate', 'ibandronate sodium', 'iodohippurate sodium i 125', 'lasalocid sodium', 'liothyronine (l- isomer) sodium', 'lorglumide sodium', 'loxoprofen sodium', 'meclofenamate sodium', 'meloxicam sodium', 'metampicillin sodium', 'methicillin sodium', 'methylprednisolone sodium succinate', 'mevastatin, sodium salt', 'monensin sodium (monensin a is shown)', 'monensin sodium salt', 'montelukast sodium', 'nafcillin sodium', 'nafcillin sodium salt monohydrate', 'nalidixic acid sodium', 'nalidixic acid sodium salt', 'naproxen sodium', 'nigericin sodium', 'nigericin, sodium salt, streptomyces hygroscopicus', 'novobiocin sodium', 'novobiocin sodium salt', 'oxacillin sodium', 'pantoprazole sodium', 'phenylbutyrate sodium', 'phenytoin sodium', 'picosulfate de sodium', 'piperacillin sodium', 'piperacillin sodium salt', 'pravastatin sodium', 'prednisolone sodium phosphate', 'pregnenolone sulfate sodium', 'pregnenolone sulfate, sodium salt', 'rabeprazole sodium', 'rabeprazole sodium salt', 'riboflavin 5-phosphate sodium', 'risedronate sodium', 'salinomycin, sodium', 'simvastatin, sodium salt', 'sivelestat sodium salt hydrate', 'sodium 4-phenylbutyrate', 'sodium 5-difluoromethoxy-2-((3,4-dimethoxy-2-pyridyl)methylsulfinyl)-1h-benzimidazole', 'sodium beta-nicotinamide adenine dinucleotide phosphate', 'sodium dehydrocholate', 'sodium fluoroacetate', 'sodium lauryl sulfoacetate', 'sodium metrizoate', 'sodium nitroprusside', 'sodium oxamate', 'sodium oxybate', 'sodium p-aminosalicylate', 'sodium salicylate', 'sodium taurocholate hydrate', 'sodium tetradecyl sulfate', 'sodium thiosalicylate', 'sodium trichloroacetate', 'sodium valproate', 'sulfamethazine sodium salt', 'sulfaquinoxaline sodium', 'sulfaquinoxaline sodium salt', 'suramin sodium salt', 'symclosene sodium', 'thiamylal sodium', 'thio-nadp sodium', 'thiopental sodium', 'ticarcillin sodium', 'tolmetin sodium', 'tolmetin sodium salt dihydrate', 'triethylene glycol, sodium salt', 'undecylenic acid monoethanolamine sulfosuccinate sodium', "uridine 5'-diphosphate sodium", 'valproate sodium', 'valproic acid (sodium salt)', 'valproic acid sodium', 'valproic acid sodium salt (sodium valproate)', 'valproic acid sodium salt (sodium valproate)', 'valproic acid, sodium salt', 'zomepirac sodium', 'zomepirac sodium salt', 'zonisamide sodium']
#'salt': 'salt', #['(+)-isoproterenol (+)-bitartrate salt', '(-)-eseroline fumarate salt', '(r) -naproxen sodium salt', '1,3,5(10)-estratrien-3-ol-17-one sulphate, sodium salt', "1,3-naphthalenedisulfonic acid, 6,6'-((3,3'-dimethyl(1,1'-biphenyl)-4,4'-diyl)bis(azo))bis(4-amino-5-hydroxy)-, tetrasodium salt", "2-(methylthio)adenosine 5'-diphosphate trisodium salt hydrate", 'acetopromazine maleate salt', 'alpha-keto-glutaric acid, sodium salt', 'alverine citrate salt', 'arl 67156 trisodium salt', 'as-605240 (potassium salt)', 'atractyloside potassium salt', 'azlocillin sodium salt', 'benzeneacetic acid, 2-[(2,6-dichlorophenyl)amino]-, monosodium salt [cas]', 'bisoprolol hemifumarate salt', 'bntx maleate salt hydrate', 'bucladesine sodium salt', 'butirosin disulfate salt', 'canrenoic acid potassium salt', 'canrenoic acid, potassium salt', 'carbenoxolone disodium salt', 'carbinoxamine maleate salt', 'cefamandole sodium salt', 'cefazolin sodium salt', 'cefmetazole sodium salt', 'cefotaxime sodium salt', 'cefoxitin sodium salt', 'cefsulodin sodium salt', 'cefsulodin sodium salt hydrate', 'cefuroxime sodium salt', 'cephalosporin c zinc salt', 'cephalothin sodium salt', 'chlorophyllide cu complex na salt', 'cinnamyl alcohol, titanium (4+) salt', 'clavulanate potassium salt', 'cloxacillin sodium salt', 'cromolyn disodium salt', 'dantrolene sodium salt', 'delta1-hydrocortisone 21-hemisuccinate sodium salt', 'dicloxacillin sodium salt hydrate', 'eserine hemisulfate salt', 'estradiol-3-sulfate, sodium salt', 'etidronic acid, disodium salt', 'fenoprofen calcium salt dihydrate', 'fluvastatin sodium salt', 'folinic acid calcium salt', 'fusidic acid sodium salt', 'isoetharine mesylate salt', 'ketorolac tris salt', 'l-703,606 oxalate salt hydrate', 'lidocaine n-ethyl bromide quaternary salt', 'methotrimeprazine maleat salt', 'metoprolol-(+,-) (+)-tartrate salt', 'mevastatin, sodium salt', 'monensin sodium salt', 'moxalactam disodium salt', 'n-methylhistaprodifen dioxalate salt', 'n-omega-methyl-5-hydroxytryptamine oxalate salt', 'nafcillin sodium salt monohydrate', 'nalidixic acid sodium salt', 'nigericin, sodium salt, streptomyces hygroscopicus', 'novobiocin sodium salt', 'org 24598 lithium salt', 'oxalamine citrate salt', 'oxiglutatione disodium salt', 'oxotremorine sesquifumarate salt', 'pantothenic acid(d) na salt', 'phenelzine sulfate salt', 'phenethicillin potassium salt', 'piperacillin sodium salt', 'pregnenolone sulfate, sodium salt', 'pridinol methanesulfonate salt', 'quipazine dimaleate salt', 'rabeprazole sodium salt', 'ribostamycin sulfate salt', 's-(+)-pd 123177 trifluoroacetate salt hydrate', 'simvastatin, sodium salt', 'sivelestat sodium salt hydrate', 'sulfamethazine sodium salt', 'sulfaquinoxaline sodium salt', 'suramin sodium salt', 'timolol maleate salt', 'tolmetin sodium salt dihydrate', 'triethylene glycol, sodium salt', 'trimipramine maleate salt', 'valproic acid (sodium salt)', 'valproic acid sodium salt (sodium valproate)', 'valproic acid, sodium salt', 'ver-3323 hemifumarate salt', 'vinblastine sulfate salt', 'zomepirac sodium salt']
#'acetate': 'acetate', #['11-ketorockogenin acetate', '11-oxoursolic acid acetate', '6alpha-methylprednisolone acetate', 'alfadolone acetate', 'alpha-tochopheryl acetate', 'alpha-tocopherol acetate', 'avocadene acetate', 'beta-amyrin acetate', 'betamethasone acetate', 'bisabolol acetate', 'bisindolylmaleimide viii (acetate)', 'cadmium acetate', 'cafestol acetate', 'caryophyllenyl acetate', 'cedryl acetate', 'chlormadinone acetate', 'cortisol acetate', 'cortisone acetate', 'crassin acetate', 'cyproterone acetate', 'd[-arg-2]kyotorphan acetate', 'deoxygedunol acetate', "deoxysappanone b 7,3'-dimethyl ether acetate", 'desoxycorticosterone acetate', 'dexamethasone acetate', 'dichlorisone acetate', 'digitoxigenin acetate', 'ergosterol acetate', 'estradiol acetate', 'estrone acetate', 'euphol acetate', 'flecainide acetate', 'fludrocortisone acetate', 'gangleoidin acetate', 'goserelin acetate', 'guanabenz acetate', 'hecogenin acetate', 'hydrocortisone acetate', 'hydroxypropylmethylcellulose acetate succinate', 'isoflupredone acetate', 'kyotorphin acetate', 'lanosterol acetate', 'larixol acetate', 'lonchocarpic acid acetate', 'lysyl-tyrosyl-lysine acetate', 'lysyltryptophanyl-lysine acetate', 'marmesin acetate', 'medroxyprogesterone acetate', 'megestrol acetate', 'melengestrol acetate', 'meta-cresyl acetate', 'methionyl-leucylphenylalanine acetate', 'neotigogenin acetate', 'ng-monomethyl-l-arginine acetate', 'nomegestrol acetate', 'norethindrone acetate', 'oleananoic acid acetate', 'oleanolic acid acetate', 'phenylmercuric acetate', 'prasterone acetate', 'prednisolone acetate', 'retinyl acetate', 'rhodinyl acetate', 'roxatidine acetate hcl', 'roxatidine acetate hydrochloride', 's-methyl-l-thiocitrulline acetate', 'sitosteryl acetate', 'smilagenin acetate', 'strophanthidin acetate', 'strophanthidinic acid lactone acetate', 'sumatrol acetate', 'totarol acetate', 'ursolic acid acetate']
#'sulfate': 'sulfate', #['(4-(p-(diethylamino)-alpha-phenylbenzylidene)-2,5-cyclohexadien-1-ylidene)diethylammonium sulfate', 'abacavir sulfate', 'agmatine sulfate', 'amikacin sulfate', 'aminopentamide sulfate', 'aminoresveratrol sulfate', 'androsterone sodium sulfate', 'apramycin sulfate', 'arcaine sulfate', 'atropine sulfate', 'becanamycin sulfate', 'bekanamycin sulfate', 'bleomycin sulfate', 'cadaverine sulfate', 'capreomycin sulfate', 'clopidogrel sulfate', 'colistin sulfate', 'debrisoquin sulfate', 'dihydrostreptomycin sulfate', 'diisopropylamine sulfate (2:1)', 'dipyrithione magnesium sulfate', 'erythromycin propionate lauryl sulfate', 'eserine sulfate, physostigmine sulfate', 'eserine sulfate, physostigmine sulfate', 'gentamicin sulfate', 'gentamicine sulfate', 'guanadrel sulfate', 'guanethidine sulfate', 'hydroxychloroquine sulfate', 'indinavir sulfate', 'iobenguane sulfate', 'iproniazid sulfate', 'kanamycin a sulfate', 'kanamycin sulfate', 'l-canavanine sulfate', 'lobeline sulfate', 'mephentermine sulfate', 'metaproterenol sulfate, orciprenaline sulfate', 'metaproterenol sulfate, orciprenaline sulfate', 'mhpg sulfate potassium', 'neomycin sulfate', 'netilmicin sulfate', 'oxyquinoline sulfate', 'paromomycin sulfate', 'penbutolol sulfate', 'phenelzine sulfate', 'phenelzine sulfate salt', 'physostigmine sulfate', 'polymyxin b sulfate', 'pregnenolone sulfate sodium', 'pregnenolone sulfate, sodium salt', 'quinine sulfate', 'ribostamycin sulfate', 'ribostamycin sulfate salt', 'salbutamol sulfate', 'sanguinarine sulfate', 'sisomicin sulfate', 'sodium tetradecyl sulfate', 'sparteine sulfate', 'streptomycin sulfate', 'tranylcypromine sulfate', 'tuaminoheptane sulfate', 'veratrine sulfate', 'vinblastine sulfate', 'vinblastine sulfate salt', 'vincristine sulfate', 'vindesine sulfate', 'viomycin sulfate']
#'dihydrochloride': 'dihydrochloride', #['(±)-quinpirole dihydrochloride', '1-(5-isoquinolinylsulfonyl)-2-methylpiperazine dihydrochloride', '1-(5-isoquinolinylsulfonyl)-3-methylpiperazine dihydrochloride', '1-methylhistamine dihydrochloride', '3-hydroxybenzylhydrazine dihydrochloride', '4-aminobenzamidine dihydrochloride', '4h-thiazolo(4,5-d)azepin-2-amine, 5,6,7,8-tetrahydro-6-(2-propenyl)-, dihydrochloride', 'alexidine dihydrochloride', 'amodiaquin dihydrochloride dihydrate', 'amodiaquine dihydrochloride', 'b-ht 933 dihydrochloride', 'bmy 7378 dihydrochloride', 'camylofine dihydrochloride', 'cetirizine dihydrochloride', 'chlorhexidine dihydrochloride', 'cis-(z)-flupenthixol dihydrochloride', 'cystamine dihydrochloride', 'dihydrochloride, perazin', 'dilazep dihydrochloride', 'dimaprit dihydrochloride', 'emetine dihydrochloride', 'emetine dihydrochloride hydrate', 'ethambutol dihydrochloride', 'flunarizine dihydrochloride', 'fluphenazine dihydrochloride', 'gbr 12909 dihydrochloride', 'gbr-12909 dihydrochloride', 'gbr-12935 dihydrochloride', 'h-8 dihydrochloride', 'h-8, dihydrochloride', 'h-89, dihydrochloride', 'ha 1004, dihydrochloride', 'ha 1077, dihydrochloride                              fasudil', 'histamine dihydrochloride', 'histamine, r(-)-alpha-methyl-, dihydrochloride', 'homochlorcyclizine dihydrochloride', 'hydroxyzine dihydrochloride', 'meclozine dihydrochloride', 'mibefradil dihydrochloride', 'minaprine dihydrochloride', 'mitoxantrone dihydrochloride', 'mk-2206 dihydrochloride', 'naftopidil dihydrochloride', 'nor-binaltorphimine dihydrochloride', 'opipramol dihydrochloride', 'p-mppf dihydrochloride', 'pirenzepine dihydrochloride', 'pramipexole dihydrochloride', 'putrescine dihydrochloride', 'quinacrine dihydrochloride', 'quinacrine dihydrochloride dihydrate', 'quinacrine dihydrochloride hydrate', 'quinacrine, dihydrochloride', 'quinelorane dihydrochloride', 'ranolazine dihydrochloride', 'ro 04-6790 dihydrochloride', 'sb 242084 dihydrochloride hydrate', 'spectinomycin dihydrochloride', 'sr-95639a dihydrochloride', 'telenzepine dihydrochloride', 'tremorine dihydrochloride', 'trifluoperazine dihydrochloride', 'trimetazidine dihydrochloride', 'vatalanib dihydrochloride (ptk787)', 'y-27632 dihydrochloride', 'zimelidine dihydrochloride', 'zimelidine dihydrochloride monohydrate', 'zuclopenthixol dihydrochloride']
#'maleate': 'maleate', #['(+)-brompheniramine maleate', '(+)-chlorpheniramine maleate', '(+)-mk 801 maleate', '(+)-mk-801 hydrogen maleate', '(-)-mk 801 hydrogen maleate', '(-)-mk-801 hydrogen maleate', '(±)-brompheniramine maleate', '(±)-chlorpheniramine maleate', '(±)-octoclothepin maleate', '2-methyl-5-hydroxytryptamine maleate', '5-carboxamidotryptamine maleate', 'acepromazine maleate', 'acetopromazine maleate salt', 'alpha-methyl-5-hydroxytryptamine maleate', 'asenapine maleate', 'azatadine maleate', 'bntx maleate salt hydrate', 'brl 54443 maleate', 'brompheniramine maleate', 'carbinoxamine maleate', 'carbinoxamine maleate salt', 'cgs-12066a maleate', 'chlorpheniramine maleate', 'clebopride maleate', 'dexbrompheniramine maleate', 'dexchlorpheniramine maleate', 'dizocilpine maleate', 'enalapril maleate', 'ergonovine maleate', 'flupirtine maleate', 'fluvoxamine maleate', 'hydroxytacrine maleate', 'hydroxytacrine maleate (r,s)', 'irsogladine maleate', 'methiothepin maleate', 'methylergometrine maleate', 'methylergonovine maleate', 'methysergide maleate', 'metitepine maleate', 'nomifensin maleate', 'nomifensine maleate', 'pd 168,077 maleate', 'perhexiline maleate', 'perhexilline maleate', 'pheniramine maleate', 'phenylpropanolamine maleate', 'pimethixene maleate', 'piribedil maleate', 'propiomazine maleate', 'pyrilamine maleate', 'quipazine maleate', 'quipazine, 6-nitro-, maleate', 'rosiglitazone maleate', 's(-)-timolol maleate', 's-chlorpheniramine maleate', 'tegaserod maleate', 'thioperamide maleate', 'timolol maleate', 'timolol maleate salt', 'trimebutine maleate', 'trimipramine maleate', 'trimipramine maleate salt', 'u-101958 maleate', 'u-74389g maleate', 'u-99194a maleate', 'u74389g maleate', 'way-100635 maleate']
#'kinase': 'kinase', #['adenosine kinase inhibitor', 'atm kinase inhibitor', 'atm/atr kinase inhibitor', 'aurora kinase inhibitor ii', 'aurora kinase inhibitor iii', "bruton's tyrosine kinase inhibitor iii", 'c-met/ron dual kinase inhibitor', 'casein kinase i inhibitor, d4476', 'casein kinase ii inhibitor i', 'casein kinase ii inhibitor iii, tbca', 'casein kinase ii inhibitor iv', 'cdc2-like kinase inhibitor, tg003', 'cfms receptor tyrosine kinase inhibitor', 'diacylglycerol kinase inhibitor ii', 'eef-2 kinase inhibitor, nh125', 'fgf receptor tyrosine kinase inhibitor', 'fgf/vegf receptor tyrosine kinase inhibitor, pd173074', 'met kinase inhibitor', 'met/flt-3/vegfr2 tyrosine kinase inhibitor', 'p21-activated kinase inhibitor iii, ipa-3', 'p38 map kinase inhibitor', 'p38 map kinase inhibitor iii', 'p38 map kinase inhibitor iv', 'p38 map kinase inhibitor v', 'p38 map kinase inhibitor vi, jx401', 'p38 map kinase inhibitor vii, sd-169', 'p38 map kinase inhibitor viii', 'pdgf receptor tyrosine kinase inhibitor ii', 'pdgf receptor tyrosine kinase inhibitor iii', 'pdgf receptor tyrosine kinase inhibitor iv', 'pim1 kinase inhibitor ii', 'pim1 kinase inhibitor iv', 'pim1/2 kinase inhibitor v', 'pim1/2 kinase inhibitor vi', 'polo-like kinase inhibitor i', 'polo-like kinase inhibitor ii, bto-1', 'raf kinase inhibitor iv', 'raf kinase inhibitor v', 'raf1 kinase inhibitor i', 'rho kinase inhibitor', 'rho kinase inhibitor ii', 'rho kinase inhibitor iii, rockout', 'rho kinase inhibitor iv', 'rho kinase inhibitor v', 'sphingosine kinase inhibitor', 'sphingosine kinase inhibitor 2', 'src kinase inhibitor i', 'tgf-b ri kinase inhibitor', 'tie2 kinase inhibitor', 'tpl2 kinase inhibitor', 'tpl2 kinase inhibitor ii', 'vegf receptor 2 kinase inhibitor i', 'vegf receptor 2 kinase inhibitor ii', 'vegf receptor 2 kinase inhibitor iii', 'vegf receptor 2 kinase inhibitor iv', 'vegf receptor tyrosine kinase inhibitor ii', 'vegfr tyrosine kinase inhibitor v', 'vegfr2 kinase inhibitor vi, ki8751']
#'ether': 'ether', #['16-deoxomexicanolide 16-methyl ether', '2-deshydroxydesmoxyphyllin b methyl ether', "3'-iodoapigenin dimethyl ether", '3-(1h-imidazol-4-yl)propyl di(p-fluorophenyl)methyl ether hydrochloride', '3-deoxo-3beta-hydroxymexicanolide 16-enol ether', '3-deshydroxysappanol trimethyl ether', '3-hydroxy-5,6-dehydrodinortotarol methyl ether', '8-iodocatechin tetramethyl ether', 'alfalone methyl ether', 'alpinetin methyl ether', 'biochanin a, monomethyl ether', 'chrysin dimethyl ether', 'chukrasin methyl ether', "cotogenin 3'-methyl ether", 'coumarinic acid methyl ether', 'cuneatin methyl ether', "deoxysappanone b 7,3'-dimethyl ether", "deoxysappanone b 7,3'-dimethyl ether acetate", 'deoxysappanone b trimethyl ether', 'difucol hexamethyl ether', 'dihydro-alpha-toxicarol methyl ether', 'dihydroformononetin methyl ether', 'duartin, dimethyl ether', 'epiafzelechin trimethyl ether', 'epicatechin, tetramethyl ether', 'epigallocatechin octamethyl ether', 'estriol benzyl ether', 'estriol methyl ether', 'ethylene glycol phenyl ether', 'ethynylestradiol 3-methyl ether', 'fraxidin methyl ether', 'ginkgetin tetraethyl ether', 'heteropeucenin, methyl ether', 'hymecromone methyl ether', 'irigenin trimethyl ether', 'irigenin, 7-benzyl ether', 'isotectorigenin trimethyl ether', 'leoidin dimethyl ether', 'liquiritigenin dimethyl ether', 'mexicanolide enol ether', 'mucronulatone diethyl ether', 'pinosylvin methyl ether', 'pseudo-baptigenin, methyl ether', 'purpurogallin tetramethyl ether', "quercetin tetramethyl(5,7,3',4') ether", 'resistomycin, tetramethyl ether', "resveratrol 4'-methyl ether", 'robinetin trimethyl ether', 'robustic acid methyl ether', 'rotenonic acid, methyl ether', 'sappanone a trimethyl ether', 'tetrahydrosappanone a trimethyl ether', 'theaflavin heptamethyl ether', 'totarol methyl ether', 'totarolone methyl ether']
#'methyl': 'methyl', #['(-)-scopolamine methyl bromide', '2,4,5-trichlorophenoxyacetic acid, methyl ester', '2,4-dichlorophenoxyacetic acid, methyl ester', '2,4-dichlorophenoxybutyric acid, methyl ester', '2-deshydroxydesmoxyphyllin b methyl ether', '3-deoxy-3beta-hydroxyangolensic acid methyl ester', '3-hydroxy-5,6-dehydrodinortotarol methyl ether', '3alpha-hydroxy-3-deoxyangolensic acid methyl ester', '3beta-acetoxydeoxyangolensic acid, methyl ester', '6-acetoxy-angolensic acid methyl ester', '6-hydroxyangolensic acid methyl ester', '7beta-hydroxy-7-desacetoxykhivorinic acid, methyl ester', 'alfalone methyl ether', 'alpha-methyl-dl-tyrosine methyl ester hydrochloride', 'alpinetin methyl ether', 'angolensic acid, methyl ester', 'atropine methyl nitrate', 'azinphos methyl', 'cholic acid, methyl ester', 'chukrasin methyl ether', 'coumarinic acid methyl ether', 'cuneatin methyl ether', 'desacetyl(7)khivorinic acid, methyl ester', 'dihydro-alpha-toxicarol methyl ether', 'dihydroformononetin methyl ether', 'dihydrogedunic acid, methyl ester', 'dihydrogeduninic acid, methyl ester', 'dl-p-chlorophenylalanine methyl ester hydrochloride', 'estriol methyl ether', 'fraxidin methyl ether', 'gamboginic acid, methyl ester', 'heteropeucenin, methyl ether', 'hymecromone methyl ether', 'l-3,4-dihydroxyphenylalanine methyl ester hydrochloride', 'mandelic acid, methyl ester', 'methyl 7-deshydroxypyrogallin-4-carboxylate', 'methyl aminolevulinate hydrochloride', 'methyl benzethonium chloride', 'methyl coclaurine', 'methyl gamboginate', 'methyl nicotinate', 'methyl parathione', 'methyl robustone', 'ng-nitro-l-arginine methyl ester hydrochloride', 'pinosylvin methyl ether', 'pseudo-baptigenin, methyl ether', 'robustic acid methyl ether', 'rotenonic acid, methyl ether', 'suprofen methyl ester', 'totarol methyl ether', 'totarol-19-carboxylic acid, methyl ester', 'totarolone methyl ether']
#'hcl': 'hcl', #['alizapride hcl', 'alosetron hcl', 'amfebutamone hcl', 'anagrelide hcl', 'atomoxetine hcl', 'azelastine hcl', 'benazepril hcl', 'benidipine hcl', 'bs-181 hcl', 'buflomedil hcl', 'celiprolol hcl', 'cetraxate hcl', 'duloxetine hcl', 'erlotinib hcl', 'granisetron hcl', 'idarubicin hcl', 'itopride hcl', 'maprotilline hcl', 'midazolam hcl', 'moxifloxacin hcl', 'moxonidine hcl', 'nefazodone hcl', 'nifekalant hcl', 'olopatadine hcl', 'palonosetron hcl', 'pazopanib hcl', 'pd153035 hcl', 'perospirone hcl', 'pioglitazone hcl', 'pj34 hcl', 'pramipexole hcl', 'procainamide hcl', 'quinapril hcl', 'ropinirole hcl', 'ropivacaine hcl', 'rosiglitazone hcl', 'roxatidine acetate hcl', 'rufloxacin hcl', 'sibutramine hcl', 'terbinafine hcl', 'tiagabine hcl', 'ticlopidine hcl', 'tizanidine hcl', 'topotecan hcl', 'tropisetron hcl', 'venlafaxine hcl', 'zm 39923 hcl']
#'ii': 'ii', #['apoptosis inhibitor ii, ns3694', 'aurora kinase inhibitor ii', 'bisindolylmaleimide ii', 'casein kinase ii inhibitor i', 'casein kinase ii inhibitor iii, tbca', 'casein kinase ii inhibitor iv', 'cdk2 inhibitor ii', 'cdk4 inhibitor ii,                 nsc 625987', 'cdk9 inhibitor ii', 'chk2 inhibitor ii', 'cox-2 inhibitor ii', 'diacylglycerol kinase inhibitor ii', 'dna-pk inhibitor ii', 'eif-2a inhibitor ii, sal003', 'erk inhibitor ii', 'erk inhibitor ii,                   negative control', 'erk inhibitor ii, fr180204', 'flt-3 inhibitor ii', 'glucagon receptor antagonist ii', 'gsk-3b inhibitor ii', 'gß? modulator ii, gallein', 'histone acetyltransferase inhibitor ii', 'igf-1r inhibitor ii', 'jak2 inhibitor ii', 'jak3 inhibitor ii', 'jnk inhibitor ii', 'luciferase inhibitor ii', 'mek inhibitor ii', 'mek1/2 inhibitor ii', 'nf-?b activation inhibitor ii, jsh-23', 'olomoucine ii', 'pdgf receptor tyrosine kinase inhibitor ii', 'pi 3-k  inhibitor ii', 'pi 3-kb inhibitor ii', 'pim1 kinase inhibitor ii', 'polo-like kinase inhibitor ii, bto-1', 'pp1 analog ii, 1nm-pp1', 'rho kinase inhibitor ii', 'syk inhibitor ii', 'toll-like receptor 7 ligand ii', 'tpl2 kinase inhibitor ii', 'uch-l1 inhibitor ii', 'vegf receptor 2 kinase inhibitor ii', 'vegf receptor tyrosine kinase inhibitor ii', 'wee1 inhibitor ii']
#'a': 'a', #['aloisine a, rp107', 'aminopurvalanol a', 'antimycin a', 'antimycin a (a1 shown)', 'arcyriaflavin a, synthetic', 'aristolochic acid a', 'aurora a inhibitor i', 'avocatin a', 'biochanin a', 'biochanin a diacetate', 'biochanin a, monomethyl ether', 'brefeldin a from penicillium brefeldianum', 'chrysanthellin a', 'cyclosporin a', 'cyclosporin a, tolypocladium inflatum', 'diprotin a', 'ginkgolide a', 'glucokinase activator, cpd a', 'gramicidin (gramicidin a shown)', 'griseofulvin analog a', 'herbimycin a, streptomyces sp.', 'humilin a', 'kanamycin a sulfate', 'latrunculin a, latrunculia magnifica', 'licochalcone a', 'lividomycin a', 'manumycin a, streptomyces parvulus', 'mithramycin a', 'monensin sodium (monensin a is shown)', 'muristerone a, ipomoea spp.', 'oligomycin (a shown)', 'oligomycin a', 'pepstatin a', 'proscillaridin a', 'protoveratrine a', 'sapindoside a', 'sappanone a trimethyl ether', 'sennoside a', 'tetrahydrosappanone a trimethyl ether', 'trichostatin a', 'trichostatin a (tsa)', 'trichostatin a, streptomyces sp.', 'withaferin a, withania somnifera', 'xylocarpus a']
#'chloride': 'chloride', #['(+)-muscarine chloride', '(+)-tubocurarine chloride', '(±)-muscarine chloride', 'acetyl-beta-methylcholine chloride', 'acetylcholine chloride', 'acetylthiocholine chloride', 'alcuronium chloride', 'ambenonium chloride', 'benzalkonium chloride', 'benzethonium chloride', 'berberine chloride', 'betaine aldehyde chloride', 'bethanechol chloride', 'calmidazolium chloride', 'carnitine chloride', 'cetalkonium chloride', 'cetylpyridinium chloride', 'chelerythrine chloride', 'choline chloride', 'coralyne chloride', 'cotarnine chloride', 'dequalinium chloride', 'dequalinium chloride hydrate', 'diphenyleneiodonium chloride', 'dl-stearoylcarnitine chloride', 'edrophonium chloride', 'lithium chloride', 'lithium chloride, molecular biology grade', 'methacholine chloride', 'methyl benzethonium chloride', 'methylbenzethonium chloride', 'methylbenzthonium chloride', 'oxybutynin chloride', 'palmatine chloride', 'palmitoyl-dl-carnitine chloride', 'pralidoxime chloride', 'sanguinarine chloride', 'stearoylcarnitine chloride', 'succinylcholine chloride', 'tetraethylammonium chloride', 'tridihexethyl chloride', 'tubocurarine chloride']
#'ester': 'ester', #['1h-imidazole-5-carboxylic acid, 1-(1-phenylethyl)-, ethyl ester, (r)- [cas]', '2,4,5-trichlorophenoxyacetic acid, isooctyl ester', '2,4,5-trichlorophenoxyacetic acid, methyl ester', '2,4-dichlorophenoxyacetic acid, isooctyl ester', '2,4-dichlorophenoxyacetic acid, methyl ester', '2,4-dichlorophenoxybutyric acid, methyl ester', '2-hydroxybenzoic acid, 2- hydroxyethyl ester', '3-(4-methoxyphenyl)-2-propenoic acid 3-methylbutyl ester', '3-deoxy-3beta-hydroxyangolensic acid methyl ester', '3alpha-hydroxy-3-deoxyangolensic acid methyl ester', '3beta-acetoxydeoxyangolensic acid, methyl ester', '6-acetoxy-angolensic acid methyl ester', '6-hydroxyangolensic acid methyl ester', '7beta-hydroxy-7-desacetoxykhivorinic acid, methyl ester', 'acetyl tyrosine ethyl ester', 'alpha-methyl-dl-tyrosine methyl ester hydrochloride', 'angolensic acid, methyl ester', 'arecaidine propargyl ester hydrobromide', 'benzeneacetic acid, alpha-(hydroxymethyl)-, 9-methyl-3-oxa-9-azatricyclo[3.3.1.02,4]non-7-yl ester', 'caffeic acid phenethyl ester', 'chaulmoogric acid, ethyl ester', 'cholic acid, methyl ester', 'chrysanthemic acid, ethyl ester', 'desacetyl(7)khivorinic acid, methyl ester', 'dihydrogedunic acid, methyl ester', 'dihydrogeduninic acid, methyl ester', 'dl-p-chlorophenylalanine methyl ester hydrochloride', 'felbinac ethyl ester', 'gamboginic acid, methyl ester', 'glycine, n-[2-[(acetylthio)methyl]-1-oxo-3-phenylpropyl]-,phenylmethyl ester [cas]', 'haematommic acid, ethyl ester', 'l-3,4-dihydroxyphenylalanine methyl ester hydrochloride', 'mandelic acid, methyl ester', 'methylorsellinic acid, ethyl ester', 'ng-nitro-l-arginine methyl ester hydrochloride', 'orsellinic acid, ethyl ester', 'p-hydroxybenzoic acid propyl ester', 'pyridine-3-carboxylic acid benzyl ester', 'suprofen methyl ester', 'tetradecanoic acid 1-methylethyl ester', 'totarol-19-carboxylic acid, methyl ester']
#'hydrobromide': 'hydrobromide', #['(-)-scopolamine hydrobromide', '(±)-6-chloro-pb hydrobromide', '(±)-7-hydroxy-dpat hydrobromide', '(±)-8-hydroxy-dpat hydrobromide', '(±)-ampa hydrobromide', '(±)-chloro-apb hydrobromide', '(±)-skf 38393, n-allyl-, hydrobromide', '6,7-adtn hydrobromide', 'anisodamine hydrobromide', 'arecaidine propargyl ester hydrobromide', 'arecoline hydrobromide', 'cis(+/-)-8-oh-pbzi hydrobromide', 'citalopram hydrobromide', 'd-3-methoxy-n-methylmorphinan hydrobromide', 'darifenacin hydrobromide', 'dextromethorphan hydrobromide', 'dextromethorphan hydrobromide monohydrate', 'dihydro-beta-erythroidine hydrobromide', 'dipropyldopamine hydrobromide', 'dl-homatropine hydrobromide', 'eletriptan hydrobromide', 'fenoterol hydrobromide', 'galanthamine hydrobromide', 'homatropine hydrobromide', 'homatropine hydrobromide (r,s)', 'muscimol hydrobromide', 'nan-190 hydrobromide', 'r(+)-6-bromo-apb hydrobromide', 'r(-)-2,10,11-trihydroxy-n-propylnoraporphine hydrobromide', 'r-(+)-7-hydroxy-dpat hydrobromide', 'r-(+)-8-hydroxy-dpat hydrobromide', 's-ethylisothiourea hydrobromide', 'sch-202676 hydrobromide', 'scopolamin-n-oxide hydrobromide', 'scopolamine hydrobromide', 'skf 83959 hydrobromide', 'skf-89145 hydrobromide']
#'bromide': 'bromide', #['(-)-scopolamine methyl bromide', '(-)-scopolamine,n-butyl-, bromide', 'bromoacetylcholine bromide', 'butylscopolammonium (n-) bromide', 'cetrimonium bromide', 'choline bromide', 'clidinium bromide', 'decamethonium bromide', 'demecarium bromide', 'domiphen bromide', 'fenoldopam bromide', 'fenpiverinium bromide', 'gabazine bromide', 'hemicholinium bromide', 'hexamethonium bromide', 'homatropine bromide', 'homidium bromide', 'ipratropium bromide', 'lidocaine n-ethyl bromide quaternary salt', 'mepenzolate bromide', 'methantheline bromide', 'methscopolamine bromide', 'neostigmine bromide', 'oxyphenonium bromide', 'pancuronium bromide', 'pinaverium bromide', 'pipenzolate bromide', 'propantheline bromide', 'pyridostigmine bromide', 'thonzonium bromide', 'vecuronium bromide']
#'iii': 'iii', #['aurora kinase inhibitor iii', 'bisindolylmaleimide iii, hydrochloride', "bruton's tyrosine kinase inhibitor iii", 'casein kinase ii inhibitor iii, tbca', 'cdk1/2 inhibitor iii', 'cdk2 inhibitor iii', 'cdk4 inhibitor iii', 'dna-pk inhibitor iii', 'erk inhibitor iii', 'flt-3 inhibitor iii', 'jak2 inhibitor iii, sd-1029', 'm-344, histone deacetylase inhibitor iii', 'mk-2 inhibitor iii', 'neuronal differentiation inducer iii', 'nf-?b activation inhibitor iii', 'nfat activation inhibitor iii', 'p21-activated kinase inhibitor iii, ipa-3', 'p38 map kinase inhibitor iii', 'p53 activator iii, rita', 'pdgf receptor tyrosine kinase inhibitor iii', 'rho kinase inhibitor iii, rockout', 'rna polymerase iii inhibitor', 'sirt1 inhibitor iii', 'stat3 inhibitor iii, wp1066', 'stat5 inhibitor iii, pimozide', 'syk inhibitor iii', 'tgf-b ri inhibitor iii', 'tnf-a antagonist iii, r-7050', 'vegf receptor 2 kinase inhibitor iii']
#'hydrate': 'hydrate', #['(+)-catechin hydrate', "2-(methylthio)adenosine 5'-diphosphate trisodium salt hydrate", 'amikacin hydrate', 'ara-g hydrate', 'bay 61-3606 hydrochloride hydrate', 'bix 01294 trihydrochloride hydrate', 'bntx maleate salt hydrate', 'cefaclor hydrate', 'cefsulodin sodium salt hydrate', 'cephalexin hydrate', 'chm-1 hydrate', 'ciprofloxacin hydrochloride hydrate', 'cyclophosphamide hydrate', 'dequalinium chloride hydrate', 'dicloxacillin sodium salt hydrate', 'emetine dihydrochloride hydrate', 'gr 127935 hydrochloride hydrate', 'ketanserin tartrate hydrate', 'l-703,606 oxalate salt hydrate', 'lp 12 hydrochloride hydrate', 'methotrexate hydrate', 'psoromic acid hydrate', 'quinacrine dihydrochloride hydrate', 's-(+)-pd 123177 trifluoroacetate salt hydrate', 'sb 242084 dihydrochloride hydrate', 'sivelestat sodium salt hydrate', 'sodium taurocholate hydrate', 'sulfacetamide sodic hydrate']
#'mesylate': 'mesylate', #['benztropine mesylate', 'betahistine mesylate', 'bromocriptine mesylate', 'bromocryptine mesylate', 'deferoxamine mesylate', 'dihydroergotamine mesylate', 'doxazosin mesylate', 'fenoldopam mesylate', 'gabexate mesylate', 'gemifloxacin mesylate', 'isoetharine mesylate', 'isoetharine mesylate salt', 'isotharine mesylate', 'methiothepin mesylate', 'nafamostat mesylate', 'nelfinavir mesylate', 'pefloxacin mesylate', 'pefloxacine mesylate', 'pergolide mesylate', 'phentolamine mesylate', 'pirlindole mesylate', 'pralidoxime mesylate', 'reboxetine mesylate', 'saquinavir mesylate', 'sns-314 mesylate']
#'sb': 'sb', #['sb 200646 hydrochloride', 'sb 202190', 'sb 202474, negative control for p38 mapk inhibition studies', 'sb 203580', 'sb 203580 (hydrochloride)', 'sb 203580, sulfone', 'sb 204741', 'sb 205384', 'sb 205607', 'sb 206553 hydrochloride', 'sb 216763', 'sb 218078', 'sb 218795', 'sb 222200', 'sb 225002', 'sb 239063', 'sb 242084 dihydrochloride hydrate', 'sb 269970 hydrochloride', 'sb 328437', 'sb 415286', 'sb 431542', 'sb 505124', 'sb 525334']
#'cas': 'cas', #['10h-phenothiazine, 2-chloro-10-[3-(4-methyl-1-piperazinyl)propyl- [cas]', '1h-cyclopenta[b]quinolin-9-amine, 2,3,5,6,7,8-hexahydro-, monohydrochloride- [cas]', '1h-imidazol-2-amine, n-(2,6-dichlorophenyl)-4,5-dihydro- [cas]', '1h-imidazole-5-carboxylic acid, 1-(1-phenylethyl)-, ethyl ester, (r)- [cas]', '1h-indole-2-propanoic acid, 1-[(4-chlorophenyl)methyl]-3-[(1,1-dimethylethyl)thio]-alpha,alpha-dimethyl-5-(1-methylethyl)- [cas]', '2(1h)-pyrimidinone, 4-amino-1-d-arabinofuranosyl- [cas]', '2h-indol-2-one, 1,3-dihydro-1-phenyl-3,3-bis(4-pyridinylmethyl)- [cas]', '3(2h)-pyridazinone, 6-[4-(difluoromethoxy)-3-methoxyphenyl]- [cas]', '4-thiazolidinecarboxylic acid, 2-oxo-, (r)- [cas]', '6h-pyrido[2,3-b][1,4]benzodiazepin-6-one, 11-[[2-[(diethylamino)methyl]-1-piperidinyl]acetyl]-5,11-dihydro- [cas]', '8-azaspiro[4.5]decane-7,9-dione, 8-[2-[[(2,3-dihydro-1,4-benzodioxin-2-yl)methyl]amino]ethyl]-, monomethanesulfonate [cas]', 'acetamide, 2-amino-n-(1-methyl-1,2-diphenylethyl)-, (+/-)- [cas]', 'adenosine, n-(2-hydroxycyclopentyl)-, (1s-trans)- [cas]', 'benzeneacetic acid, 2-[(2,6-dichlorophenyl)amino]-, monosodium salt [cas]', 'benzeneacetonitrile, alpha-[3-[[2-(3,4-dimethoxyphenyl)ethyl]methylamino]propyl]-3,4-dimethoxy-alpha-(1-methylethyl)-, (r)- [cas]', 'benzo[a]phenanthridine-10,11-diol, 5,6,6a,7,8,12b-hexahydro-, trans- [cas]', 'glycine, n-[2-[(acetylthio)methyl]-1-oxo-3-phenylpropyl]-,phenylmethyl ester [cas]', "guanidine, n-cyano-n'-(1,1-dimethylpropyl)-n''-3-pyridinyl- [cas]", 'l-glutamic acid, n-[4-[[(2,4-diamino-6-pteridinyl)methyl]methylamino]benzoyl]- [cas]', 'oxiranecarboxylic acid, 2-[6-(4-chlorophenoxy)hexyl]-, ethyl ester- [cas]', 'pyrazinecarboxamide, 3,5-diamino-n-(aminoiminomethyl)-6-chloro- [cas]', 'thiophene, 5-bromo-2-(4-fluorophenyl)-3-[4-(methylsulfonyl)phenyl]- [cas]']
#'iv': 'iv', #['akt inhibitor iv', 'bisindolylmaleimide iv', 'casein kinase ii inhibitor iv', 'cdk1 inhibitor iv, ro-3306', 'cdk2 inhibitor iv,                 nu6140', 'cox-1 inhibitor iv, tfap', 'histone deacetylase inhibitor iv', 'ikk-2 inhibitor iv', 'jak3 inhibitor iv', 'nf-?b activation inhibitor iv', 'p38 map kinase inhibitor iv', 'pdgf receptor tyrosine kinase inhibitor iv', 'pi 3-k  inhibitor iv', 'pi 3-ka inhibitor iv', 'pim1 kinase inhibitor iv', 'raf kinase inhibitor iv', 'rho kinase inhibitor iv', 'sirt1 inhibitor iv, (s)-35', 'sirt1/2 inhibitor iv, cambinol', 'vegf receptor 2 kinase inhibitor iv', 'ß-secretase inhibitor iv']
#'monohydrate': 'monohydrate', #['1,10-phenanthroline monohydrate', "adenosine 5'-monophosphate monohydrate", 'cephalexin monohydrate', 'ciprofloxacin hydrochloride monohydrate', 'cyclophosphamide monohydrate', 'd-lactitol monohydrate', 'dextromethorphan hydrobromide monohydrate', 'halometasone monohydrate', 'imipenem monohydrate', 'nafcillin sodium salt monohydrate', 'picotamide monohydrate', 'pinacidil monohydrate', 'quinidine hydrochloride monohydrate', 'risedronic acid monohydrate', 'sematilide monohydrochloride monohydrate', 'sitagliptin monophosphate monohydrate', 'tetrahydroxy-1,4-quinone monohydrate', 'theophylline monohydrate', 'zimelidine dihydrochloride monohydrate', 'ziprasidone hydrochloride monohydrate']
#'ag': 'ag', #['ag 1024', 'ag 112', 'ag 1295', 'ag 1296', 'ag 1478', 'ag 18', 'ag 490', 'ag 879', 'ag 9', 'i-ome-tyrphostin ag 538', 'tyrphostin ag 112', 'tyrphostin ag 1478', 'tyrphostin ag 494', 'tyrphostin ag 537', 'tyrphostin ag 538', 'tyrphostin ag 698', 'tyrphostin ag 835', 'tyrphostin ag 879', 'tyrphostin ag 879 (ag 879)', 'tyrphostin ag 879 (ag 879)']
#'citrate': 'citrate', #['alverine citrate', 'alverine citrate salt', 'carbetapentane citrate', 'clomiphene citrate', 'clomiphene citrate (z,e)', 'deptropine citrate', 'diethylcarbamazine citrate', 'lithium citrate', 'morantel citrate', 'mosapride citrate', 'orphenadrine citrate', 'oxalamine citrate salt', 'oxelaidin citrate', 'oxolamine citrate', 'sildenafil citrate', 'tamoxifen citrate', 'toremifene citrate', 'toremiphene citrate', 'tripelennamine citrate', 'vardenafil citrate']
#'i': 'i', #['aurora a inhibitor i', 'bisindolylmaleimide i', 'casein kinase i inhibitor, d4476', 'casein kinase ii inhibitor i', 'dynamin inhibitor i, dynasore', 'grayanotoxin i', 'gsk-3b inhibitor i', 'heat shock protein inhibitor i', 'iodohippurate sodium i 125', 'irak-1/4 inhibitor i', 'jak inhibitor i', 'luciferase inhibitor i', 'mek inhibitor i', 'parp inhibitor i, 3-aba', 'polo-like kinase inhibitor i', 'raf1 kinase inhibitor i', 'src kinase inhibitor i', 'vegf receptor 2 kinase inhibitor i', 'wnt antagonist i, iwr-1-endo']
#'receptor': 'receptor', #['c5a receptor antagonist, w-54011', 'cfms receptor tyrosine kinase inhibitor', 'fgf receptor tyrosine kinase inhibitor', 'fgf/vegf receptor tyrosine kinase inhibitor, pd173074', 'glucagon receptor antagonist ii', 'glucocorticoid receptor modulator, cpda', 'naadp receptor modulator', 'pdgf receptor tyrosine kinase inhibitor ii', 'pdgf receptor tyrosine kinase inhibitor iii', 'pdgf receptor tyrosine kinase inhibitor iv', 's1p1 receptor agonist, sew2871', 'thyroid hormone receptor antagonist, 1-850', 'toll-like receptor 7 ligand ii', 'vegf receptor 2 kinase inhibitor i', 'vegf receptor 2 kinase inhibitor ii', 'vegf receptor 2 kinase inhibitor iii', 'vegf receptor 2 kinase inhibitor iv', 'vegf receptor tyrosine kinase inhibitor ii']
#'tartrate': 'tartrate', #['(-)-cis-(1s,2r)-u-50488 tartrate', 'dihydroergotamine tartrate', 'ergotamine tartrate', 'ifenprodil tartrate', 'ketanserin tartrate', 'ketanserin tartrate hydrate', 'levallorphan tartrate', 'metoprolol tartrate', 'morantel tartrate', 'nicotinyl alcohol tartrate', 'nicotinyl tartrate', 'pempidine tartrate', 'pyrantel tartrate', 'tolterodine tartrate', 'trimeprazine tartrate', 'tylosin tartrate', 'zolpidem tartrate']
#'phosphate': 'phosphate', #['adenosine phosphate', 'antazoline phosphate', 'benproperine phosphate', 'betamethasone sodium phosphate', 'dexamethasone sodium phosphate', 'disopyramide phosphate', 'estramustine phosphate sodium', 'fludarabine phosphate', 'hydrocortisone phosphate triethylamine', 'hydrocortisone sodium phosphate', 'iproniazid phosphate', 'iproniazide phosphate', 'oleandomycin phosphate', 'prednisolone sodium phosphate', 'primaquine phosphate', 'sodium beta-nicotinamide adenine dinucleotide phosphate', 'triciribine (triciribine phosphate)']
#'tyrphostin': 'tyrphostin', #['ag-1478 (tyrphostin ag-1478)', 'ag-490 (tyrphostin b42)', 'tyrphostin 1', 'tyrphostin 23', 'tyrphostin 47', 'tyrphostin 51', 'tyrphostin a9', 'tyrphostin ag 112', 'tyrphostin ag 1478', 'tyrphostin ag 494', 'tyrphostin ag 537', 'tyrphostin ag 538', 'tyrphostin ag 698', 'tyrphostin ag 835', 'tyrphostin ag 879', 'tyrphostin ag 879 (ag 879)', 'tyrphostin b44(-)']
#'disodium': 'disodium', #['carbenicillin disodium', 'carbenoxolone disodium salt', 'cnqx disodium', 'cromolyn disodium salt', 'disodium 2,3,4,5-tetrachloro-6-(2,4,5,7-tetraiodo-6-oxido-3-oxo-3h-xanthen-9-yl)benzoate', 'etidronate disodium', 'etidronic acid, disodium salt', 'moxalactam disodium', 'moxalactam disodium salt', 'nbqx disodium', 'oxiglutatione disodium salt', 'phosphomycin disodium', 'phosphoramidon disodium', 'protoporphyrin ix disodium', 'r935788 (fostamatinib disodium, r788 disodium)', 'r935788 (fostamatinib disodium, r788 disodium)', 'ticarcillin disodium']
#'dihydrate': 'dihydrate', #['amiloride hydrochloride dihydrate', 'amodiaquin dihydrochloride dihydrate', 'cefoperazone dihydrate', 'diatrizoic acid dihydrate', 'ellagic acid, dihydrate', 'enalaprilat dihydrate', 'esomeprazole magnesium dihydrate', 'fenoprofen calcium salt dihydrate', 'formoterol fumarate dihydrate', 'hexamethonium dibromide dihydrate', 'naltrexone hydrochloride dihydrate', 'oxytetracycline dihydrate', 'quercetin dihydrate', 'quercetin, dihydrate', 'quinacrine dihydrochloride dihydrate', 'tolmetin sodium salt dihydrate']
#'b': 'b', #['2-deshydroxydesmoxyphyllin b methyl ether', 'amphotericin b', 'avocatin b', "deoxysappanone b 7,3'-dimethyl ether", "deoxysappanone b 7,3'-dimethyl ether acetate", 'deoxysappanone b trimethyl ether', 'erythrosin b', 'griseofulvin analog b', 'leucodinine b', 'polymyxin b sulfate', 'protoveratrine b', 'punctaporin b', 'rhodomyrtoxin b', 'sarmentoside b', 'sennoside b']
#'v': 'v', #['adenylyl cyclase type v inhibitor, nky80', 'akt inhibitor v, triciribine', 'bisindolylmaleimide v', 'dna-pk inhibitor v', 'ikk-2 inhibitor v', 'jnk inhibitor v', 'nf-?b activation inhibitor v, 5hpp-33', 'p38 map kinase inhibitor v', 'penicillin v potassium', 'pim1/2 kinase inhibitor v', 'ptp inhibitor v, phps1', 'raf kinase inhibitor v', 'rho kinase inhibitor v', 'stat3 inhibitor v, stattic', 'vegfr tyrosine kinase inhibitor v']
#'antagonist': 'antagonist', #['ahr antagonist', 'c5a receptor antagonist, w-54011', 'ccr2 antagonist', 'ccr4 antagonist', 'glucagon receptor antagonist ii', 'hedgehog antagonist viii', 'hh signaling antagonist vii, jk184', 'hh/gli antagonist, gant58', 'hh/gli antagonist, gant61', 'il-2ra antagonist', 'mdm2 antagonist, nutlin-3, racemic', 'mif antagonist, iso-1', 'thyroid hormone receptor antagonist, 1-850', 'tnf-a antagonist iii, r-7050', 'wnt antagonist i, iwr-1-endo']
#'ethyl': 'ethyl', #['1h-imidazole-5-carboxylic acid, 1-(1-phenylethyl)-, ethyl ester, (r)- [cas]', 'acetyl tyrosine ethyl ester', 'chaulmoogric acid, ethyl ester', 'chrysanthemic acid, ethyl ester', 'ethyl 1-benzyl-3-hydroxy- 2-oxo[5h]pyrrole-4-carboxylate', 'ethyl 4-(bis (hydroxypropyl)) aminobenzoate', 'ethyl linoleate-9,10,12,13-d4', 'ethyl paraben', 'felbinac ethyl ester', 'haematommic acid, ethyl ester', 'methylorsellinic acid, ethyl ester', 'orsellinic acid, ethyl ester', 'oxiranecarboxylic acid, 2-[6-(4-chlorophenoxy)hexyl]-, ethyl ester- [cas]', 'quinine ethyl carbonate']
#'diacetate': 'diacetate', #['acacetin diacetate', 'ajmaline diacetate', 'biochanin a diacetate', 'clovanediol diacetate', 'diflorasone diacetate', 'dihydrocelastryl diacetate', 'estradiol diacetate', 'ethynodiol diacetate', 'gitoxigenin diacetate', 'leoidin diacetate', 'scandenin diacetate', 'sericetin diacetate', 'swietenolide diacetate', 'triamcinolone diacetate']
#'c': 'c', #['ampk inhibitor,                  compound c', 'cardiogenol c', 'cephalosporin c sodium', 'cephalosporin c zinc salt', 'fissinolide c', 'griseofulvin analog c', 'hederacoside c', 'heudelottin c', 'khayasin c', 'lanatoside c', 'lavendustin c', 'mitomycin c', 'oligomycin c', 'vitamin c']
#'potassium': 'potassium', #['as-605240 (potassium salt)', 'atractyloside potassium', 'atractyloside potassium salt', 'canrenoic acid potassium salt', 'canrenoic acid, potassium salt', 'clavulanate potassium salt', 'd-609 potassium', 'hetacillin potassium', 'mhpg sulfate potassium', 'penicillin g potassium', 'penicillin v potassium', 'phenethicillin potassium', 'phenethicillin potassium salt', 'potassium p-aminobenzoate']
#'fumarate': 'fumarate', #['(-)-eseroline fumarate', '(-)-eseroline fumarate salt', '3-aminopropionitrile fumarate', 'adamantamine fumarate', 'bisoprolol fumarate', 'clemastine fumarate', 'cotinine fumarate', 'formoterol fumarate', 'formoterol fumarate dihydrate', 'gr-89696 fumarate', 'ibutilide fumarate', 'ketotifen fumarate']
#'alcohol': 'alcohol', #['2,4-dichlorobenzyl alcohol', 'batyl alcohol', 'beta-caryophyllene alcohol', 'cetyl alcohol, 14c-labeled', 'cetyl stearyl alcohol', 'chrysanthemyl alcohol', 'cinnamyl alcohol, titanium (4+) salt', 'coniferyl alcohol', 'nicotinyl alcohol tartrate', 'phenylethyl alcohol', 'salicyl alcohol']
#'tyrosine': 'tyrosine', #['acetyl tyrosine ethyl ester', "bruton's tyrosine kinase inhibitor iii", 'cfms receptor tyrosine kinase inhibitor', 'fgf receptor tyrosine kinase inhibitor', 'fgf/vegf receptor tyrosine kinase inhibitor, pd173074', 'met/flt-3/vegfr2 tyrosine kinase inhibitor', 'pdgf receptor tyrosine kinase inhibitor ii', 'pdgf receptor tyrosine kinase inhibitor iii', 'pdgf receptor tyrosine kinase inhibitor iv', 'vegf receptor tyrosine kinase inhibitor ii', 'vegfr tyrosine kinase inhibitor v']
#'viii': 'viii', #['akt inhibitor viii, isozyme-selective, akti-1/2', 'bisindolylmaleimide viii (acetate)', 'gsk-3b inhibitor viii', 'hedgehog antagonist viii', 'ikk-2 inhibitor viii', 'jnk inhibitor viii', 'p38 map kinase inhibitor viii', 'parp inhibitor viii, pj34', 'pi 3-k  inhibitor viii', 'pi 3-ka inhibitor viii', 'sirt1/2 inhibitor viii, salermide']
#'pd': 'pd', #['pd 0325901', 'pd 158780', 'pd 168,077 maleate', 'pd 168393', 'pd 169316', 'pd 173074', 'pd 174265', 'pd 184161', 'pd 81723', 'pd 98,059', 'pd 98059']
#'pi': 'pi', #['pi 3-k  inhibitor', 'pi 3-k  inhibitor ii', 'pi 3-k  inhibitor iv', 'pi 3-k  inhibitor viii', 'pi 3-k? inhibitor vii', 'pi 3-k?/ckii inhibitor', 'pi 3-ka inhibitor iv', 'pi 3-ka inhibitor viii', 'pi 3-kb inhibitor ii', 'pi 3-kg inhibitor', 'pi 3-kß inhibitor vi, tgx-221']
#'vi': 'vi', #['?-secretase inhibitor vi', 'apoptosis activator vi, cd437/ahpn', 'histone deacetylase inhibitor vi, hnha', 'ikk-2 inhibitor vi', 'jak3 inhibitor vi', 'p38 map kinase inhibitor vi, jx401', 'parp inhibitor vi, nu1025', 'pi 3-kß inhibitor vi, tgx-221', 'pim1/2 kinase inhibitor vi', 'vegfr2 kinase inhibitor vi, ki8751']
#'hemisulfate': 'hemisulfate', #['albuterol hemisulfate', 'aminoguanidine hemisulfate', 'eserine hemisulfate salt', 'm-iodobenzylguanidine hemisulfate', 'mephentermine hemisulfate', 'metaproterenol hemisulfate', 'proflavine hemisulfate', 's-methylisothiourea hemisulfate', 'terbutaline hemisulfate', 'tranylcypromine hemisulfate']
#'nitrate': 'nitrate', #['atropine methyl nitrate', 'butoconazole nitrate', 'econazole nitrate', 'isopilocarpine nitrate', 'methylatropine nitrate', 'miconazole nitrate', 'oxiconazole nitrate', 'pilocarpine nitrate', 'sertaconazole nitrate', 'sulconazole nitrate']
'estradiol': 'estradiol', #['estradiol', 'estradiol acetate', 'estradiol benzoate', 'estradiol cypionate', 'estradiol diacetate', 'estradiol dipropionate', 'estradiol enanthate', 'estradiol propionate', 'estradiol valerate', 'ethinyl estradiol']
#'2': '2', #['meriolin 2', 'nymania 2', 'pi3-kinase .alpha. inhibitor 2', 'reactive blue 2', 'sphingosine kinase inhibitor 2', 'torin 2', 'vegf receptor 2 kinase inhibitor i', 'vegf receptor 2 kinase inhibitor ii', 'vegf receptor 2 kinase inhibitor iii', 'vegf receptor 2 kinase inhibitor iv']
#'methanesulfonate': 'methanesulfonate', #['(+)-bromocriptine methanesulfonate', '(±) trans-u-50488 methanesulfonate', 'cgp 20712a methanesulfonate', 'dihydroergotamine methanesulfonate', 'naltriben methanesulfonate', 'pergolide methanesulfonate', 'phenamil methanesulfonate', 'pridinol methanesulfonate', 'pridinol methanesulfonate salt']
#'adenosine': 'adenosine', #['adenosine', "adenosine 3',5'-cyclic monophosphate", "adenosine 5'-monophosphate monohydrate", 'adenosine kinase inhibitor', 'adenosine monophosphate', 'adenosine phosphate', 'adenosine triphosphate', 'adenosine, n-(2-hydroxycyclopentyl)-, (1s-trans)- [cas]', "alpha,beta-methylene adenosine 5'-triphosphate dilithium"]
#'sp.': 'sp.', #['apicidin, fusarium sp.', 'chetomin, chaetomium sp.', 'herbimycin a, streptomyces sp.', 'k-252a, nocardiopsis sp.', 'okadaic acid, prorocentrum sp.', 'paclitaxel, taxus sp.', 'scytonemin, lyngbya sp.', 'staurosporine, streptomyces sp.', 'trichostatin a, streptomyces sp.']
#'succinate': 'succinate', #['chloramphenicol sodium succinate', 'cibenzoline succinate', 'doxylamine succinate', 'hydroxypropylmethylcellulose acetate succinate', 'loxapine succinate', 'methylprednisolone sodium succinate', 'pregnenolone succinate', 'sumatriptan succinate', 'tocopheryl succinate']
#'trimethyl': 'trimethyl', #['3-deshydroxysappanol trimethyl ether', 'deoxysappanone b trimethyl ether', 'epiafzelechin trimethyl ether', 'irigenin trimethyl ether', 'isotectorigenin trimethyl ether', 'robinetin trimethyl ether', 'sappanone a trimethyl ether', 'tetrahydrosappanone a trimethyl ether']
#'ix': 'ix', #['?-secretase inhibitor ix', 'gsk-3 inhibitor ix', 'gsk-3 inhibitor ix, control, mebio', 'ikk-3 inhibitor ix', 'jnk inhibitor ix', 'protoporphyrin ix', 'protoporphyrin ix disodium', 'telomerase inhibitor ix']
#'shown': 'shown', #['abamectin (avermectin b1a shown)', 'antimycin a (a1 shown)', 'bleomycin (bleomycin b2 shown)', 'gramicidin (gramicidin a shown)', 'monensin sodium (monensin a is shown)', 'oligomycin (a shown)', 'omega-3-acid esters (epa shown)', 'teicoplanin [a(2-1) shown]']
#'calcium': 'calcium', #['acamprosate calcium', 'atorvastatin calcium', 'calcium gluceptate', 'fenoprofen calcium salt dihydrate', 'folinic acid calcium salt', 'fosfomycin calcium', 'leucovorin calcium', 'rosuvastatin calcium']
#'streptomyces': 'streptomyces', #['anisomycin, streptomyces griseolus', 'herbimycin a, streptomyces sp.', 'manumycin a, streptomyces parvulus', 'nigericin, sodium salt, streptomyces hygroscopicus', 'staurosporine, streptomyces sp.', 'trichostatin a, streptomyces sp.', 'valinomycin, streptomyces fulvissimus', 'vancomycin hydrochloride from streptomyces orientalis']
#'oxalate': 'oxalate', #['benazoline oxalate', 'escitalopram oxalate', 'l-703,606 oxalate salt hydrate', 'metrazoline oxalate', 'n-omega-methyl-5-hydroxytryptamine oxalate salt', 'nafronyl oxalate', 's(-)-p-bromotetramisole oxalate', 'sr 59230a oxalate']
'hydrocortisone': 'hydrocortisone', #['hydrocortisone', 'hydrocortisone acetate', 'hydrocortisone base', 'hydrocortisone butyrate', 'hydrocortisone hemisuccinate', 'hydrocortisone phosphate triethylamine', 'hydrocortisone sodium phosphate', 'hydrocortisone valerate']
#'p38': 'p38', #['p38 map kinase inhibitor', 'p38 map kinase inhibitor iii', 'p38 map kinase inhibitor iv', 'p38 map kinase inhibitor v', 'p38 map kinase inhibitor vi, jx401', 'p38 map kinase inhibitor vii, sd-169', 'p38 map kinase inhibitor viii', 'sb 202474, negative control for p38 mapk inhibition studies']
#'bitartrate': 'bitartrate', #['(-)-epinephrine bitartrate', 'adrenaline bitartrate', 'epinephrine bitartrate', 'l(-)-norepinephrine bitartrate', 'metaraminol bitartrate', 'nicotine bitartrate', 'pentolinium bitartrate']
#'iodide': 'iodide', #['1,1-dimethyl-4-phenyl-piperazinium iodide', '1,1-dimethyl-4-phenylpiperazinium iodide', 'ac-93253 iodide', 'isopropamide iodide', 'propidium iodide', 'pyridostigmine iodide', 'thymol iodide']
#'dimethyl': 'dimethyl', #["3'-iodoapigenin dimethyl ether", 'chrysin dimethyl ether', 'dimethyl 4,4-o-phenylene-bis(3-thiophanate)', 'dimethyl gambogate', 'duartin, dimethyl ether', 'leoidin dimethyl ether', 'liquiritigenin dimethyl ether']
#'palmitate': 'palmitate', #['ascorbyl palmitate', 'chloramphenicol palmitate', 'clindamycin palmitate hydrochloride', 'colfosceril palmitate', 'isopropyl palmitate', 'pipothiazine palmitate', 'retinyl palmitate']
#'bay': 'bay', #['bay 11-7082', 'bay 11-7085', 'bay 41-2272', 'bay 61-3606 hydrochloride hydrate', 'bay t 7391', 'regorafenib (bay 73-4506)', 'telatinib (bay 57-9352)']
#'bisindolylmaleimide': 'bisindolylmaleimide', #['bisindolylmaleimide i', 'bisindolylmaleimide ii', 'bisindolylmaleimide iii, hydrochloride', 'bisindolylmaleimide iv', 'bisindolylmaleimide v', 'bisindolylmaleimide viii (acetate)', 'bisindolylmaleimide xi (hydrochloride)']
#'analog': 'analog', #['erbstatin analog', 'griseofulvin analog a', 'griseofulvin analog b', 'griseofulvin analog c', 'griseofulvin analog d', 'hexahydro-sila-difenidol hydrochloride, p-fluoro analog', 'pp1 analog ii, 1nm-pp1']
#'agonist': 'agonist', #['gpr109b agonist', 'gpr30 agonist, g-1', 'gpr40 agonist', 'lxra/ß agonist', 's1p1 receptor agonist, sew2871', 'smoothened agonist, sag', 'wnt agonist']
#'map': 'map', #['p38 map kinase inhibitor', 'p38 map kinase inhibitor iii', 'p38 map kinase inhibitor iv', 'p38 map kinase inhibitor v', 'p38 map kinase inhibitor vi, jx401', 'p38 map kinase inhibitor vii, sd-169', 'p38 map kinase inhibitor viii']
#'skf': 'skf', #['skf 83566', 'skf 83959 hydrobromide', 'skf 86466', 'skf 89626', 'skf 89976a hydrochloride', 'skf 95282 dimaleate', 'skf 96365']
#'-': '-', #['(-) -levobunolol hydrochloride', 'asarinin (-)', 'duartin (-)', 'eburnamonine (-)', 'perillic acid (-)', 'picrotin - picrotoxinin']
#'akt': 'akt', #['akt inhibitor iv', 'akt inhibitor v, triciribine', 'akt inhibitor viii, isozyme-selective, akti-1/2', 'akt inhibitor x', 'akt inhibitor xii, isozyme-selective, akti-2', 'akt inhibitor x·hcl']
#'hemifumarate': 'hemifumarate', #['aliskiren hemifumarate', 'bisoprolol hemifumarate salt', 'quetiapine hemifumarate', 'rilmenidine hemifumarate', 'ver-3323 hemifumarate salt', 'xamoterol hemifumarate']
#'benzoate': 'benzoate', #['benzyl benzoate', 'denatonium benzoate', 'estradiol benzoate', 'eugenyl benzoate', 'propyl benzoate', 'rizatriptan benzoate']
#'na': 'na', #['cerivastatin na', 'chlorophyllide cu complex na salt', 'cilastatin na', 'montelukast na', 'pantothenic acid(d) na salt', 'parecoxib na']
#'histone': 'histone', #['histone acetyltransferase activator, ctb', 'histone acetyltransferase inhibitor ii', 'histone deacetylase inhibitor iv', 'histone deacetylase inhibitor vi, hnha', 'histone lysine methyltransferase inhibitor', 'm-344, histone deacetylase inhibitor iii']
#'pamoate': 'pamoate', #['hydroxyzine pamoate', 'hyydroxyzine pamoate', 'oxantel pamoate', 'pararosaniline pamoate', 'pyrantel pamoate', 'pyrvinium pamoate']
#'jnk': 'jnk', #['jnk inhibitor ii', 'jnk inhibitor ix', 'jnk inhibitor v', 'jnk inhibitor viii', 'jnk inhibitor xi, bi-87g3', 'jnk inhibitor,                    negative control']
#'activation': 'activation', #['nf-?b activation inhibitor ii, jsh-23', 'nf-?b activation inhibitor iii', 'nf-?b activation inhibitor iv', 'nf-?b activation inhibitor v, 5hpp-33', 'nf-kb activation inhibitor', 'nfat activation inhibitor iii']
#'ro': 'ro', #['ro 04-6790 dihydrochloride', 'ro 20-1724', 'ro 25-6981 hydrochloride', 'ro 41-0960', 'ro 8-4304', 'ro 90-7501']
'valproic': 'valproic acid', #['valproic acid', 'valproic acid (sodium salt)', 'valproic acid hydroxamate', 'valproic acid sodium', 'valproic acid sodium salt (sodium valproate)', 'valproic acid, sodium salt']
#'vegf': 'vegf', #['vegf inducer, gs4012', 'vegf receptor 2 kinase inhibitor i', 'vegf receptor 2 kinase inhibitor ii', 'vegf receptor 2 kinase inhibitor iii', 'vegf receptor 2 kinase inhibitor iv', 'vegf receptor tyrosine kinase inhibitor ii']
#'+': '+', #['(+) -levobunolol hydrochloride', 'bicuculline (+)', 'chelidonine (+)', 'isocorydine (+)', 'mammea a/ba + a/bb']
#'ketone': 'ketone', #['(z-ll)2 ketone', 'ekebergin ketone', 'n-p-tosyl-l-phenylalanine chloromethyl ketone', 'na-p-tosyl-l-lysine chloromethyl ketone hydrochloride', 'z-l-phe chloromethyl ketone']
#'tetrasodium': 'tetrasodium', #["1,3-naphthalenedisulfonic acid, 6,6'-((3,3'-dimethyl(1,1'-biphenyl)-4,4'-diyl)bis(azo))bis(4-amino-5-hydroxy)-, tetrasodium salt", '2-chloroadenosine triphosphate tetrasodium', '2-methylthioadenosine triphosphate tetrasodium', 'nadph tetrasodium', 'ppnds tetrasodium']
#'compound': 'compound', #['?-secretase inhibitor xxi, compound e', 'ampk inhibitor,                  compound c', 'compound 401', 'compound 52', 'compound 56']
#'d': 'd', #['actinomycin d', 'cycloserine (d)', 'd 4476', 'griseofulvin analog d', 'panthenol (d)']
#'r,s': 'r,s', #['amethopterin (r,s)', 'baclofen (r,s)', 'dropropizine (r,s)', 'homatropine hydrobromide (r,s)', 'hydroxytacrine maleate (r,s)']
#'activator': 'activator', #['ampk activator', 'apoptosis activator vi, cd437/ahpn', 'glucokinase activator, cpd a', 'histone acetyltransferase activator, ctb', 'p53 activator iii, rita']
#'benzyl': 'benzyl', #['benzyl benzoate', 'benzyl cinnamate', 'benzyl isothiocyanate', 'estriol benzyl ether', 'pyridine-3-carboxylic acid benzyl ester']
'betamethasone': 'betamethasone', #['betamethasone', 'betamethasone 17,21-dipropionate', 'betamethasone acetate', 'betamethasone sodium phosphate', 'betamethasone valerate']
#'xi': 'xi', #['bisindolylmaleimide xi (hydrochloride)', 'gsk-3b inhibitor xi', 'ikk-2 inhibitor xi', 'jnk inhibitor xi, bi-87g3', 'parp inhibitor xi, dr2313']
#'from': 'from', #['brefeldin a from penicillium brefeldianum', 'l-mimosine from koa hoale seeds', 'nordihydroguaiaretic acid from larrea divaricata (creosote bush)', 'vancomycin hydrochloride from streptomyces orientalis', 'wortmannin from penicillium funiculosum']
#'lactone': 'lactone', #['bromoenol lactone', 'dehydro(11,12)ursolic acid lactone', 'deoxyandirobin lactone', 'mevalonic-d, l acid lactone', 'strophanthidinic acid lactone acetate']
#'nsc': 'nsc', #['cdk4 inhibitor ii,                 nsc 625987', 'nsc 210902', 'nsc 663284', 'nsc 721648', 'nsc 95397']
#'hemisuccinate': 'hemisuccinate', #['chloramphenicol hemisuccinate', 'estrone hemisuccinate', 'hydrocortisone hemisuccinate', 'methylprednisolone hemisuccinate', 'prednisolone hemisuccinate']
#'propionate': 'propionate', #['clobetasol propionate', 'erythromycin propionate lauryl sulfate', 'estradiol propionate', 'fluticasone propionate', 'testosterone propionate']
#'control': 'control', #['erk inhibitor ii,                   negative control', 'gsk-3 inhibitor ix, control, mebio', 'jnk inhibitor,                    negative control', 'pkr inhibitor, negative control', 'sb 202474, negative control for p38 mapk inhibition studies']
'erythromycin': 'erythromycin', #['erythromycin', 'erythromycin estolate', 'erythromycin ethylsuccinate', 'erythromycin propionate lauryl sulfate', 'erythromycin stearate']
#'gr': 'gr', #['gr 113808', 'gr 127935 hydrochloride hydrate', 'gr 46611', 'gr 55562 dihydrobromide', 'gr 89696']
'griseofulvin': 'griseofulvin', #['griseofulvin', 'griseofulvin analog a', 'griseofulvin analog b', 'griseofulvin analog c', 'griseofulvin analog d']
#'vii': 'vii', #['hh signaling antagonist vii, jk184', 'ikk inhibitor vii', 'p38 map kinase inhibitor vii, sd-169', 'pi 3-k? inhibitor vii', 'stat3 inhibitor vii']
#'ikk-2': 'ikk-2', #['ikk-2 inhibitor iv', 'ikk-2 inhibitor v', 'ikk-2 inhibitor vi', 'ikk-2 inhibitor viii', 'ikk-2 inhibitor xi']
#'ly': 'ly', #['ly 171883', 'ly 231617', 'ly 294002', "ly 294002, 4'-nh2", 'ly 303511']
#'parp': 'parp', #['parp inhibitor i, 3-aba', 'parp inhibitor vi, nu1025', 'parp inhibitor viii, pj34', 'parp inhibitor xi, dr2313', 'parp inhibitor xii']
'quercetin': 'quercetin', #['quercetin', 'quercetin (sophoretin)', 'quercetin dihydrate', "quercetin tetramethyl(5,7,3',4') ether", 'quercetin, dihydrate']
'quinacrine': 'quinacrine', #['quinacrine dihydrochloride', 'quinacrine dihydrochloride dihydrate', 'quinacrine dihydrochloride hydrate', 'quinacrine hydrochloride', 'quinacrine, dihydrochloride']
#'rho': 'rho', #['rho kinase inhibitor', 'rho kinase inhibitor ii', 'rho kinase inhibitor iii, rockout', 'rho kinase inhibitor iv', 'rho kinase inhibitor v']
'y-27632': 'y-27632', #['rock inhibitor, y-27632', 'y-27632', 'y-27632 (hydrochloride)', 'y-27632 2hcl', 'y-27632 dihydrochloride']
#'trisodium': 'trisodium', #["2-(methylthio)adenosine 5'-diphosphate trisodium salt hydrate", 'arl 67156 trisodium salt', 'trisodium ethylenediamine tetracetate', 'uridine triphosphate trisodium']
#'triphosphate': 'triphosphate', #['2-chloroadenosine triphosphate tetrasodium', '2-methylthioadenosine triphosphate tetrasodium', 'adenosine triphosphate', 'uridine triphosphate trisodium']
#'trihydrochloride': 'trihydrochloride', #['4-(1,1-difluoro-1,1a,6,10b-tetrahydrodibenzo(a,e)cyclopropa(c)cyclohepten-6-yl)-a-((5-quinolinyloxy)methyl)-1-piperazineethanol trihydrochloride, (6r)-(1aa,6a,10ba)', 'bix 01294 trihydrochloride hydrate', 'mdl 26,630 trihydrochloride', 'spermidine trihydrochloride']
#'tetramethyl': 'tetramethyl', #['8-iodocatechin tetramethyl ether', 'epicatechin, tetramethyl ether', 'purpurogallin tetramethyl ether', 'resistomycin, tetramethyl ether']
'879': 'ag-879', #['ag 879', 'tyrphostin ag 879', 'tyrphostin ag 879 (ag 879)', 'tyrphostin ag 879 (ag 879)']
'triciribine': 'triciribine', #['akt inhibitor v, triciribine', 'triciribine', 'triciribine (triciribine phosphate)', 'triciribine (triciribine phosphate)']
#'xii': 'xii', #['akt inhibitor xii, isozyme-selective, akti-2', 'gsk-3ß inhibitor xii, tws119', 'gsk3b inhibitor xii, tws119', 'parp inhibitor xii']
'retinoic': 'retinoic acid', #['all trans retinoic acid', 'retinoic acid', 'retinoic acid p-hydroxyanilide', 'retinoic acid, all trans']
#'hemihydrate': 'hemihydrate', #['apomorphine hydrochloride hemihydrate', 'lercanidipine hydrochloride hemihydrate', 'paroxetine hydrochloride hemihydrate (mw = 374.83)', 'r(-) apomorphine hydrochloride hemihydrate']
#'aurora': 'aurora', #['aurora a inhibitor i', 'aurora kinase inhibitor ii', 'aurora kinase inhibitor iii', 'aurora kinase/cdk inhibitor']
#'bix': 'bix', #['bix 01294', 'bix 01294 trihydrochloride hydrate', 'bix 02188', 'bix 02189']
#'brl': 'brl', #['brl 37344 sodium', 'brl 50481', 'brl 52537 hydrochloride', 'brl 54443 maleate']
#'casein': 'casein', #['casein kinase i inhibitor, d4476', 'casein kinase ii inhibitor i', 'casein kinase ii inhibitor iii, tbca', 'casein kinase ii inhibitor iv']
#'zinc': 'zinc', #['cephalosporin c zinc salt', 'pyrithione zinc', 'sulfanilate zinc', 'zinc undecylenate']
'chloramphenicol': 'chloramphenicol', #['chloramphenicol', 'chloramphenicol hemisuccinate', 'chloramphenicol palmitate', 'chloramphenicol sodium succinate']
#'pivalate': 'pivalate', #['clocortolone pivalate', 'diflucortolone pivalate', 'flumethasone pivalate', 'flumethazone pivalate']
'erlotinib': 'erlotinib', #['desmethyl erlotinib (cp-473420)', 'erlotinib', 'erlotinib hcl', 'erlotinib hydrochloride']
#'base': 'base', #['dna base excision repair pathway inhibitor', 'hydrocortisone base', 'tobramycin, free base', 'xl-184 free base (cabozantinib)']
'epicatechin': 'epicatechin', #['epicatechin', 'epicatechin 3,5-digallate', 'epicatechin monogallate', 'epicatechin, tetramethyl ether']
'epigallocatechin': 'epigallocatechin', #['epigallocatechin', 'epigallocatechin 3,5-digallate', 'epigallocatechin gallate', 'epigallocatechin octamethyl ether']
#'erk': 'erk', #['erk inhibitor ii', 'erk inhibitor ii,                   negative control', 'erk inhibitor ii, fr180204', 'erk inhibitor iii']
#'negative': 'negative', #['erk inhibitor ii,                   negative control', 'jnk inhibitor,                    negative control', 'pkr inhibitor, negative control', 'sb 202474, negative control for p38 mapk inhibition studies']
'physostigmine': 'eserine', #['eserine sulfate, physostigmine sulfate', 'physostigmine', 'physostigmine salicylate', 'physostigmine sulfate']
'gambogic': 'gambogic acid', #['gambogic acid', 'gambogic acid amide', 'gambogic acid morpholine adduct', 'gambogic acid, garcinia hanburyi']
#'l': 'l', #['glutamine (l)', 'hyoscyamine (l)', 'mevalonic-d, l acid lactone', 'thyroxine (l)']
#'gsk-3': 'gsk-3', #['gsk-3 inhibitor ix', 'gsk-3 inhibitor ix, control, mebio', 'gsk-3 inhibitor x', 'gsk-3 inhibitor xiii']
#'gsk-3b': 'gsk-3b', #['gsk-3b inhibitor i', 'gsk-3b inhibitor ii', 'gsk-3b inhibitor viii', 'gsk-3b inhibitor xi']
'homatropine': 'homatropine', #['homatropine bromide', 'homatropine hydrobromide', 'homatropine hydrobromide (r,s)', 'homatropine methylbromide']
'leoidin': 'leoidin', #['leoidin', 'leoidin diacetate', 'leoidin dimethyl ether', 'leoidin monoacetate']
#'lithium': 'lithium', #['lithium chloride', 'lithium chloride, molecular biology grade', 'lithium citrate', 'org 24598 lithium salt']
#'salicylate': 'salicylate', #['menthyl salicylate', 'phenyl salicylate', 'physostigmine salicylate', 'sodium salicylate']
'methylprednisolone': 'methylprednisolone', #['methylprednisolone', 'methylprednisolone hemisuccinate', 'methylprednisolone sodium succinate', 'methylprednisolone, 6-alpha']
#'nf-?b': 'nf-?b', #['nf-?b activation inhibitor ii, jsh-23', 'nf-?b activation inhibitor iii', 'nf-?b activation inhibitor iv', 'nf-?b activation inhibitor v, 5hpp-33']
#'pdgf': 'pdgf', #['pdgf receptor tyrosine kinase inhibitor ii', 'pdgf receptor tyrosine kinase inhibitor iii', 'pdgf receptor tyrosine kinase inhibitor iv', 'pdgf rtk inhibitor']
#'3-k': '3-k', #['pi 3-k  inhibitor', 'pi 3-k  inhibitor ii', 'pi 3-k  inhibitor iv', 'pi 3-k  inhibitor viii']
'prednisolone': 'prednisolone', #['prednisolone', 'prednisolone acetate', 'prednisolone hemisuccinate', 'prednisolone sodium phosphate']
'pregnenolone': 'pregnenolone', #['pregnenolone', 'pregnenolone succinate', 'pregnenolone sulfate sodium', 'pregnenolone sulfate, sodium salt']
'staurosporine': 'staurosporine', #['staurosporine', 'staurosporine aglycone', 'staurosporine, n-benzoyl-', 'staurosporine, streptomyces sp.']
#'su': 'su', #['su 4312', 'su 5416', 'su 6656', 'su 6668']
'theaflavin': 'theaflavin', #['theaflavin', 'theaflavin digallate', 'theaflavin heptamethyl ether', 'theaflavin monogallates']
#'zm': 'zm', #['zm 336372', 'zm 39923 hcl', 'zm 39923 hydrochloride', 'zm 447439']
#'+)-bitartrate': '+)-bitartrate', #['(+)-isoproterenol (+)-bitartrate salt', 'r(-)-isoproterenol (+)-bitartrate', 's(+)-isoproterenol (+)-bitartrate']
#'hydrogen': 'hydrogen', #['(+)-mk-801 hydrogen maleate', '(-)-mk 801 hydrogen maleate', '(-)-mk-801 hydrogen maleate']
'-)-scopolamine': 'scopolamine', #['(-)-scopolamine hydrobromide', '(-)-scopolamine methyl bromide', '(-)-scopolamine n-oxide']
#?'gaba': 'gaba', #['(±)-gamma-vinyl gaba', 'gaba', 'gamma-acetylinic gaba']
#'gallate': 'gallate', #['(–)-epigallocatechin gallate', 'epigallocatechin gallate', 'theaflavin-3 gallate']
'wortmannin': 'wortmannin', #['17.beta.-hydroxy wortmannin', 'wortmannin', 'wortmannin from penicillium funiculosum']
#'r)-': 'r)-', #['1h-imidazole-5-carboxylic acid, 1-(1-phenylethyl)-, ethyl ester, (r)- [cas]', '4-thiazolidinecarboxylic acid, 2-oxo-, (r)- [cas]', 'benzeneacetonitrile, alpha-[3-[[2-(3,4-dimethoxyphenyl)ethyl]methylamino]propyl]-3,4-dimethoxy-alpha-(1-methylethyl)-, (r)- [cas]']
#?'2,3-butanedione': '2,3-butanedione', #['2,3-butanedione', '2,3-butanedione 2-monoxime', '2,3-butanedione monoxime']
'2,4,5-trichlorophenoxyacetic acid': '2,4,5-trichlorophenoxyacetic', #['2,4,5-trichlorophenoxyacetic acid', '2,4,5-trichlorophenoxyacetic acid, isooctyl ester', '2,4,5-trichlorophenoxyacetic acid, methyl ester']
'2,4-dichlorophenoxyacetic acid': '2,4-dichlorophenoxyacetic', #['2,4-dichlorophenoxyacetic acid', '2,4-dichlorophenoxyacetic acid, isooctyl ester', '2,4-dichlorophenoxyacetic acid, methyl ester']
#'dihydrobromide': 'dihydrobromide', #['2-(2-aminoethyl)isothiourea dihydrobromide', 'gr 55562 dihydrobromide', 'imetit dihydrobromide']
#'methiodide': 'methiodide', #['3-tropanylindole-3-carboxylate methiodide', '4-damp methiodide', 'oxotremorine methiodide']
#'?-secretase': '?-secretase', #['?-secretase inhibitor ix', '?-secretase inhibitor vi', '?-secretase inhibitor xxi, compound e']
'avermectin': 'avermectin', #['abamectin (avermectin b1a shown)', 'avermectin b1', 'avermectin b1a']
#'acetyl': 'acetyl', #['acetyl isogambogic acid', 'acetyl tyrosine ethyl ester', 'sulfisoxazole acetyl']
#'a1': 'a1', #['aclacinomycin a1', 'antimycin a (a1 shown)', 'antimycin a1']
#'monophosphate': 'monophosphate', #["adenosine 3',5'-cyclic monophosphate", 'adenosine monophosphate', 'sitagliptin monophosphate monohydrate']
'ag-1478': 'ag-1478', #['ag-1478', 'ag-1478 (tyrphostin ag-1478)', 'ag-1478 (tyrphostin ag-1478)']
#'x': 'x', #['akt inhibitor x', 'gsk-3 inhibitor x', 'ikk inhibitor x']
#'dipropionate': 'dipropionate', #['alclometasone dipropionate', 'beclomethasone dipropionate', 'estradiol dipropionate']
'alprenolol': 'alprenolol', #['alprenolol', 'alprenolol hydrochloride', 'bromoacetyl alprenolol menthane']
'amiloride': 'amiloride', #['amiloride hydrochloride', 'amiloride hydrochloride dihydrate', 'amiloride, hydrochloride']
#'trihydrate': 'trihydrate', #['ampicillin trihydrate', 'cefixime trihydrate', 'ceftriaxone sodium trihydrate']
'anagrelide': 'anagrelide', #['anagrelide', 'anagrelide hcl', 'anagrelide hydrochloride']
'antimycin': 'antimycin', #['antimycin a', 'antimycin a (a1 shown)', 'antimycin a1']
'apomorphine': 'apomorphine', #['apomorphine hydrochloride', 'apomorphine hydrochloride hemihydrate', 'r(-) apomorphine hydrochloride hemihydrate']
#'apoptosis': 'apoptosis', #['apoptosis activator vi, cd437/ahpn', 'apoptosis inhibitor', 'apoptosis inhibitor ii, ns3694']
#'arachidonic': 'arachidonic', #['arachidonic acid', 'arachidonic acid (20:4 n-6)', 'arachidonic acid leelamide']
'atropine': 'atropine', #['atropine', 'atropine methyl nitrate', 'atropine sulfate']
#'valerate': 'valerate', #['betamethasone valerate', 'estradiol valerate', 'hydrocortisone valerate']
'biochanin': 'biochanin', #['biochanin a', 'biochanin a diacetate', 'biochanin a, monomethyl ether']
'bleomycin': 'bleomycin', #['bleomycin (bleomycin b2 shown)', 'bleomycin (bleomycin b2 shown)', 'bleomycin sulfate']
#'penicillium': 'penicillium', #['brefeldin a from penicillium brefeldianum', 'cyclopiazonic acid, penicillium cyclopium', 'wortmannin from penicillium funiculosum']
#'tosylate': 'tosylate', #['bretylium tosylate', 'clofilium tosylate', 'tosufloxacin tosylate']
#'3': '3', #['butyrolactone 3', 'meriolin 3', 'mn-cpx 3']
'camphor': 'camphor', #['camphor (&gt', 'camphor (1r)', 'camphor, monobromated']
'catechin': 'catechin', #['catechin pentaacetate', 'catechin pentabenzoate', 'catechin tetramethylether']
#'pentaacetate': 'pentaacetate', #['catechin pentaacetate', 'haematoxylin pentaacetate', 'norstictic acid pentaacetate']
#'cdk1': 'cdk1', #['cdk1 inhibitor', 'cdk1 inhibitor iv, ro-3306', 'cdk1 inhibitor,                    cgp74514a']
#'cdk2': 'cdk2', #['cdk2 inhibitor ii', 'cdk2 inhibitor iii', 'cdk2 inhibitor iv,                 nu6140']
#'cdk4': 'cdk4', #['cdk4 inhibitor', 'cdk4 inhibitor ii,                 nsc 625987', 'cdk4 inhibitor iii']
'cefamandole': 'cefamandole', #['cefamandole nafate', 'cefamandole sodium', 'cefamandole sodium salt']
#'glycol': 'glycol', #['cefatrizine propylene glycol', 'ethylene glycol phenyl ether', 'triethylene glycol, sodium salt']
'cefsulodin': 'cefsulodin', #['cefsulodin sodium', 'cefsulodin sodium salt', 'cefsulodin sodium salt hydrate']
'cefuroxime': 'cefuroxime', #['cefuroxime axetil', 'cefuroxime sodium', 'cefuroxime sodium salt']
'cephalexin': 'cephalexin', #['cephalexin', 'cephalexin hydrate', 'cephalexin monohydrate']
#'blue': 'blue', #['chicago sky blue 6b', 'methylene blue', 'reactive blue 2']
'diphosphate': 'diphosphate', #['chloroquine diphosphate', 'motesanib diphosphate', 'primaquine diphosphate']
'ciprofloxacin': 'ciprofloxacin', #['ciprofloxacin', 'ciprofloxacin hydrochloride hydrate', 'ciprofloxacin hydrochloride monohydrate']
'clindamycin': 'clindamycin', #['clindamycin', 'clindamycin hydrochloride', 'clindamycin palmitate hydrochloride']
'cortisone': 'cortisone', #['cortisone', 'cortisone 21-acetate', 'cortisone acetate']
'cyclophosphamide': 'cyclophosphamide', #['cyclophosphamide', 'cyclophosphamide hydrate', 'cyclophosphamide monohydrate']
'deoxysappanone': 'deoxysappanone', #["deoxysappanone b 7,3'-dimethyl ether", "deoxysappanone b 7,3'-dimethyl ether acetate", 'deoxysappanone b trimethyl ether']
'dequalinium': 'dequalinium', #['dequalinium chloride', 'dequalinium chloride hydrate', 'dequalinium dichloride']
'dexamethasone': 'dexamethasone', #['dexamethasone', 'dexamethasone acetate', 'dexamethasone sodium phosphate']
'dihydroergotamine': 'dihydroergotamine', #['dihydroergotamine mesylate', 'dihydroergotamine methanesulfonate', 'dihydroergotamine tartrate']
#'pathway': 'pathway', #['dna base excision repair pathway inhibitor', 'pdk1/akt/flt dual pathway inhibitor', 'ras-net (elk-3) pathway inhibitor, xrp44x']
#'methyltransferase': 'methyltransferase', #['dna methyltransferase inhibitor', 'histone lysine methyltransferase inhibitor', 'protein methyltransferase inhibitor, ami-5']
#'dna-pk': 'dna-pk', #['dna-pk inhibitor ii', 'dna-pk inhibitor iii', 'dna-pk inhibitor v']
'duloxetine': 'duloxetine', #['duloxetine', 'duloxetine hcl', 'duloxetine hydrochloride']
'emetine': 'emetine', #['emetine', 'emetine dihydrochloride', 'emetine dihydrochloride hydrate']
#'ligand': 'ligand', #['epigenetic multiple ligand', 'integrin amß2 ligand', 'toll-like receptor 7 ligand ii']
'eserine': 'eserine', #['eserine', 'eserine hemisulfate salt', 'eserine sulfate, physostigmine sulfate']
'estriol': 'estriol', #['estriol', 'estriol benzyl ether', 'estriol methyl ether']
'estrone': 'estrone', #['estrone', 'estrone acetate', 'estrone hemisuccinate']
#'phenyl': 'phenyl', #['ethylene glycol phenyl ether', 'phenyl aminosalicylate', 'phenyl salicylate']
'fenoldopam': 'fenoldopam', #['fenoldopam', 'fenoldopam bromide', 'fenoldopam mesylate']
#'flt-3': 'flt-3', #['flt-3 inhibitor', 'flt-3 inhibitor ii', 'flt-3 inhibitor iii']
'fluphenazine': 'fluphenazine', #['fluphenazine decanoate', 'fluphenazine dihydrochloride', 'fluphenazine hydrochloride']
'formoterol': 'formoterol', #['formoterol', 'formoterol fumarate', 'formoterol fumarate dihydrate']
'fusidic': 'fusidic acid', #['fusidic acid', 'fusidic acid sodium', 'fusidic acid sodium salt']
#'modulator': 'modulator', #['glucocorticoid receptor modulator, cpda', 'gß? modulator ii, gallein', 'naadp receptor modulator']
'gramicidin': 'gramicidin', #['gramicidin', 'gramicidin (gramicidin a shown)', 'gramicidin (gramicidin a shown)']
'granisetron': 'granisetron', #['granisetron', 'granisetron hcl', 'granisetron hydrochloride']
'tws119': 'tws119', #['gsk-3ß inhibitor xii, tws119', 'gsk3b inhibitor xii, tws119', 'tws119']
#'carbonate': 'carbonate', #['guaiacol carbonate', 'guanidine carbonate', 'quinine ethyl carbonate']
#'guanidine': 'guanidine', #['guanidine carbonate', 'guanidine hydrochloride', "guanidine, n-cyano-n'-(1,1-dimethylpropyl)-n''-3-pyridinyl- [cas]"]
#'gö': 'gö', #['gö 6976', 'gö 6983', 'gö 7874, hydrochloride']
'h-8': 'h-8', #['h-8 (hydrochloride)', 'h-8 dihydrochloride', 'h-8, dihydrochloride']
#'deacetylase': 'deacetylase', #['histone deacetylase inhibitor iv', 'histone deacetylase inhibitor vi, hnha', 'm-344, histone deacetylase inhibitor iii']
'isosorbide': 'isosorbide', #['isosorbide', 'isosorbide dinitrate', 'isosorbide mononitrate']
#'jak3': 'jak3', #['jak3 inhibitor ii', 'jak3 inhibitor iv', 'jak3 inhibitor vi']
#'1': '1', #['janex 1', 'torin 1', 'tyrphostin 1']
#'inducer': 'inducer', #['keratinocyte differentiation inducer', 'neuronal differentiation inducer iii', 'vegf inducer, gs4012']
'l-glutamic': 'l-glutamic acid', #['l-glutamic acid hydrochloride', 'l-glutamic acid, n-[4-[[(2,4-diamino-6-pteridinyl)methyl]methylamino]benzoyl]- [cas]', 'l-glutamic acid, n-phthaloyl-']
'lidocaine': 'lidocaine', #['lidocaine', 'lidocaine hydrochloride', 'lidocaine n-ethyl bromide quaternary salt']
#'mdl': 'mdl', #['mdl 105,519', 'mdl 26,630 trihydrochloride', 'mdl 28170']
'metaproterenol': 'metaproterenol', #['metaproterenol', 'metaproterenol hemisulfate', 'metaproterenol sulfate, orciprenaline sulfate']
'minocycline': 'minocycline', #['minocycline', 'minocycline hydrochloride', 'minocycline, hydrochloride']
'mitoxantrone': 'mitoxantrone', #['mitoxantrone', 'mitoxantrone dihydrochloride', 'mitoxantrone hydrochloride']
'monensin': 'monensin sodium', #['monensin sodium (monensin a is shown)', 'monensin sodium (monensin a is shown)', 'monensin sodium salt']
'montelukast': 'montelukast', #['montelukast', 'montelukast na', 'montelukast sodium']
'moxifloxacin': 'moxifloxacin', #['moxifloxacin', 'moxifloxacin hcl', 'moxifloxacin hydrochloride']
'moxonidine': 'moxonidine', #['moxonidine', 'moxonidine hcl', 'moxonidine hydrochloride']
#'mrs': 'mrs', #['mrs 1523', 'mrs 2159', 'mrs 2179']
'chloromethyl': 'chloromethyl', #['n-p-tosyl-l-phenylalanine chloromethyl ketone', 'na-p-tosyl-l-lysine chloromethyl ketone hydrochloride', 'z-l-phe chloromethyl ketone']
'nalidixic': 'nalidixic acid', #['nalidixic acid', 'nalidixic acid sodium', 'nalidixic acid sodium salt']
'nefazodone': 'nefazodone', #['nefazodone', 'nefazodone hcl', 'nefazodone hydrochloride']
'oligomycin': 'oligomycin', #['oligomycin (a shown)', 'oligomycin a', 'oligomycin c']
'oxotremorine': 'oxotremorine', #['oxotremorine methiodide', 'oxotremorine sesquifumarate', 'oxotremorine sesquifumarate salt']
#'propyl': 'propyl', #['p-hydroxybenzoic acid propyl ester', 'propyl benzoate', 'propyl p-benzoate']
'paroxetine': 'paroxetine', #['paroxetine', 'paroxetine hydrochloride', 'paroxetine hydrochloride hemihydrate (mw = 374.83)']
'pj34': 'pj34', #['parp inhibitor viii, pj34', 'pj34', 'pj34 hcl']
'pentolinium': 'pentolinium', #['pentolinium bitartrate', 'pentolinium di[l(+)-tartrate]', 'pentolinium ditartrate']
#'phenol': 'phenol', #['phenol', 'phenol sulfonate', 'phenol, 5-methyl-2-(1-methylethyl)-']
'pilocarpine': 'pilocarpine', #['pilocarpine', 'pilocarpine hydrochloride', 'pilocarpine nitrate']
'pioglitazone': 'pioglitazone', #['pioglitazone', 'pioglitazone hcl', 'pioglitazone hydrochloride']
'piribedil': 'piribedil', #['piribedil', 'piribedil hydrochloride', 'piribedil maleate']
#'malate': 'malate', #['pizotifen malate', 'pizotyline malate', 'sunitinib malate (sutent)']
'pramipexole': 'pramipexole', #['pramipexole', 'pramipexole dihydrochloride', 'pramipexole hcl']
#'dimaleate': 'dimaleate', #['prochlorperazine dimaleate', 'quipazine dimaleate salt', 'skf 95282 dimaleate']
'prostaglandin': 'prostaglandin', #['prostaglandin e1', 'prostaglandin f2', 'prostaglandin f2a']
'pyridostigmine': 'pyridostigmine', #['pyridostigmine bromide', 'pyridostigmine iodid', 'pyridostigmine iodide']
'quinidine': 'quinidine', #['quinidine', 'quinidine gluconate', 'quinidine hydrochloride monohydrate']
'quipazine': 'quipazine', #['quipazine dimaleate salt', 'quipazine maleate', 'quipazine, 6-nitro-, maleate']
'rabeprazole': 'rabeprazole', #['rabeprazole', 'rabeprazole sodium', 'rabeprazole sodium salt']
'riboflavin': 'riboflavin', #['riboflavin', 'riboflavin 5-phosphate sodium', 'riboflavin tetrabutyrate, riboflavin-2-14c-labeled']
'roscovitine': 'roscovitine', #['roscovitine', 'roscovitine (seliciclib, cyc202)', 'roscovitine, (s)-isomer']
'rosiglitazone': 'rosiglitazone', #['rosiglitazone hcl', 'rosiglitazone hydrochloride', 'rosiglitazone maleate']
'sanguinarine': 'sanguinarine', #['sanguinarine', 'sanguinarine chloride', 'sanguinarine sulfate']
'203580': 'sb 203580', #['sb 203580', 'sb 203580 (hydrochloride)', 'sb 203580, sulfone']
'scopolamine': 'scopolamine', #['scopolamine hydrobromide', 'scopolamine hydrochloride', 'scopolamine n-butylbromide']
'sibutramine': 'sibutramine', #['sibutramine', 'sibutramine hcl', 'sibutramine hydrochloride']
'valproate': 'valproic acid', #['sodium valproate', 'valproate sodium', 'valproic acid sodium salt (sodium valproate)']
'spectinomycin': 'spectinomycin', #['spectinomycin', 'spectinomycin dihydrochloride', 'spectinomycin hydrochloride']
#'sphingosine': 'sphingosine', #['sphingosine', 'sphingosine kinase inhibitor', 'sphingosine kinase inhibitor 2']
#'sr': 'sr', #['sr 2640', 'sr 57,227a', 'sr 59230a oxalate']
#'stat3': 'stat3', #['stat3 inhibitor iii, wp1066', 'stat3 inhibitor v, stattic', 'stat3 inhibitor vii']
'strophanthidin': 'strophanthidin', #['strophanthidin', 'strophanthidin acetate', 'strophanthidin semicarbazide']
#'sucrose': 'sucrose', #['sucrose', 'sucrose octaacetate, ((alpha-d)-fructofuranosyl)-isomer', 'sucrose octasulfate']
'sulfaquinoxaline': 'sulfaquinoxaline', #['sulfaquinoxaline', 'sulfaquinoxaline sodium', 'sulfaquinoxaline sodium salt']
'sulindac': 'sulindac', #['sulindac', 'sulindac sulfide', 'sulindac sulfone']
'suramin': 'suramin', #['suramin', 'suramin hexasodium', 'suramin sodium salt']
#'syk': 'syk', #['syk inhibitor', 'syk inhibitor ii', 'syk inhibitor iii']
'terbinafine': 'terbinafine', #['terbinafine', 'terbinafine hcl', 'terbinafine hydrochloride']
'topotecan': 'topotecan', #['topotecan', 'topotecan hcl', 'topotecan hydrochloride']
'totarol': 'totarol', #['totarol', 'totarol acetate', 'totarol methyl ether']
'tranylcypromine': 'tranylcypromine', #['tranylcypromine hemisulfate', 'tranylcypromine hydrochloride', 'tranylcypromine sulfate']
'triamcinolone': 'triamcinolone', #['triamcinolone', 'triamcinolone acetonide', 'triamcinolone diacetate']
'trichostatin': 'trichostatin', #['trichostatin a', 'trichostatin a (tsa)', 'trichostatin a, streptomyces sp.']
'urapidil': 'urapidil', #['urapidil', 'urapidil hydrochloride', 'urapidil, 5-methyl-']
'vardenafil': 'vardenafil', #['vardenafil', 'vardenafil citrate', 'vardenafil hydrochloride']
'venlafaxine': 'venlafaxine', #['venlafaxine', 'venlafaxine hcl', 'venlafaxine hydrochloride']
'-levobunolol': 'levobunolol', #['(+) -levobunolol hydrochloride', '(-) -levobunolol hydrochloride']
'801': 'mk 801', #['(+)-mk 801 maleate', '(-)-mk 801 hydrogen maleate']
'-)-eseroline': 'eseroline fumarate', #['(-)-eseroline fumarate', '(-)-eseroline fumarate salt']
#'r': 'r', #['(r) -naproxen sodium salt', 'angolensin (r)']
#'k': 'k', #['(±)-bay k 8644', 'k 185']
'+)-tartrate': 'metoprolol', #['(±)-metoprolol (+)-tartrate', 'metoprolol-(+,-) (+)-tartrate salt']
'±)-verapamil': 'verapamil', #['(±)-verapamil hydrochloride', '(±)-verapamil, hydrochloride']
#'sulphate': 'sulphate', #['1,3,5(10)-estratrien-3-ol-17-one sulphate, sodium salt', 'indinavir sulphate']
#'carboxylic': 'carboxylic', #['1-aminocyclobutane carboxylic acid', 'pyrrolidone carboxylic acid, pidolic acid']
'1-phenylbiguanide': '1-phenylbiguanide', #['1-phenylbiguanide', '1-phenylbiguanide hydrochloride']
#'digallate': 'digallate', #['2\',2\'"-bisepigallocatechin digallate', 'theaflavin digallate']
#'monogallate': 'monogallate', #["2',2'-bisepigallocatechin monogallate", 'epicatechin monogallate']
#?'isooctyl': 'isooctyl', #['2,4,5-trichlorophenoxyacetic acid, isooctyl ester', '2,4-dichlorophenoxyacetic acid, isooctyl ester']
'2,4-dichlorophenoxybutyric': '2,4-dichlorophenoxybutyric acid', #['2,4-dichlorophenoxybutyric acid', '2,4-dichlorophenoxybutyric acid, methyl ester']
'bht': 'bht', #['2,6-di-t-butyl-4-methylphenol (bht)', 'bht']
#'5'-diphosphate': '5'-diphosphate', #["2-(methylthio)adenosine 5'-diphosphate trisodium salt hydrate", "uridine 5'-diphosphate sodium"]
#'2-chloroadenosine': '2-chloroadenosine', #['2-chloroadenosine', '2-chloroadenosine triphosphate tetrasodium']
'2-methoxyestradiol': '2-methoxyestradiol', #['2-methoxyestradiol', '2-methoxyestradiol (2-meoe2)']
'3,8-hemiacetal': '8beta-hydroxycarapin, 3,8-hemiacetal', #['3-chloro-8beta-hydroxycarapin, 3,8-hemiacetal', '8beta-hydroxycarapin, 3,8-hemiacetal']
'3-nitropropionic': '3-nitropropionic acid', #['3-nitropropionic acid', 'bovinocidin (3-nitropropionic acid)']
#'fluoride': 'fluoride', #['4-(2-aminoethyl)benzenesulfonyl fluoride hydrochloride', '4-amidinophenylmethanesulfonyl fluoride hydrochloride']
'12-myristate': 'phorbol 12-myristate 13-acetate', #['4±-phorbol 12-myristate 13-acetate', 'phorbol 12-myristate 13-acetate']
#'13-acetate': '13-acetate', #['4±-phorbol 12-myristate 13-acetate', 'phorbol 12-myristate 13-acetate']
'5hpp-33': '5hpp-33', #['5hpp-33', 'nf-?b activation inhibitor v, 5hpp-33']
#'indole': 'indole', #['6-methoxy-1,2,3,4-tetrahydro-9h-pyrido[3,4b] indole', 'indole hydrochloride']
'8beta-hydroxycarapin': '8beta-hydroxycarapin, 3,8-hemiacetal', #['8beta-hydroxycarapin', '8beta-hydroxycarapin, 3,8-hemiacetal']
'fh535': 'fh535', #['?-catenin/tcf inhibitor, fh535', 'ß-catenin/tcf inhibitor, fh535']
#'a3': 'a3', #['a3 hydrochloride', 'chromomycin a3']
#'b1a': 'b1a', #['abamectin (avermectin b1a shown)', 'avermectin b1a']
'abscisic': 'abscisic acid', #['abscisic acid (cis,trans, )', 'abscisic acid (cis,trans; +/-)']
#'+/-': '+/-', #['abscisic acid (cis,trans; +/-)', 'propranolol hydrochloride (+/-)']
'abt-869': 'abt-869', #['abt-869', 'linifanib (abt-869)']
'acacetin': 'acacetin', #['acacetin', 'acacetin diacetate']
#'acetamide': 'acetamide', #['acetamide', 'acetamide, 2-amino-n-(1-methyl-1,2-diphenylethyl)-, (+/-)- [cas]']
'acetylcholine': 'acetylcholine', #['acetylcholine', 'acetylcholine chloride']
#'adenine': 'adenine', #['adenine', 'sodium beta-nicotinamide adenine dinucleotide phosphate']
'112': 'ag-112', #['ag 112', 'tyrphostin ag 112']
'1478': 'ag-1478', #['ag 1478', 'tyrphostin ag 1478']
'ag-490': 'ag-490', #['ag-490', 'ag-490 (tyrphostin b42)']
'agk2': 'agk2', #['agk2', 'sirt2 inhibitor, agk2']
'ajmaline': 'ajmaline', #['ajmaline', 'ajmaline diacetate']
#'isozyme-selective': 'isozyme-selective', #['akt inhibitor viii, isozyme-selective, akti-1/2', 'akt inhibitor xii, isozyme-selective, akti-2']
'alaproclate': 'alaproclate', #['alaproclate', 'alaproclate hydrochloride']
'albuterol': 'albuterol', #['albuterol', 'albuterol hemisulfate']
'alexidine': 'alexidine', #['alexidine dihydrochloride', 'alexidine hydrochloride']
'alfuzosin': 'alfuzosin', #['alfuzosin', 'alfuzosin hydrochloride']
'alisertib': 'alisertib', #['alisertib (mln8237)', 'mln8237 (alisertib)']
'mln8237': 'mln8237', #['alisertib (mln8237)', 'mln8237 (alisertib)']
'alizapride': 'alizapride', #['alizapride hcl', 'alizapride hydrochloride']
#'all': 'all', #['all trans retinoic acid', 'retinoic acid, all trans']
#'trans': 'trans', #['all trans retinoic acid', 'retinoic acid, all trans']
'aloisine': 'aloisine', #['aloisine a, rp107', 'aloisine, rp106']
'alosetron': 'alosetron', #['alosetron hcl', 'alosetron hydrochloride']
'alsterpaullone': 'alsterpaullone', #['alsterpaullone', 'alsterpaullone,                   2-cyanoethyl']
'alverine': 'alverine', #['alverine citrate', 'alverine citrate salt']
'ambroxol': 'ambroxol', #['ambroxol', 'ambroxol hydrochloride']
#'amg': 'amg', #['amg 900', 'amg 9810']
'amikacin': 'amikacin', #['amikacin hydrate', 'amikacin sulfate']
'aminoguanidine': 'aminoguanidine', #['aminoguanidine hemisulfate', 'aminoguanidine hydrochloride']
'aminophylline': 'aminophylline', #['aminophylline', 'aminophylline ethylenediamine']
#'ethylenediamine': 'ethylenediamine', #['aminophylline ethylenediamine', 'trisodium ethylenediamine tetracetate']
#'aminosalicylate': 'aminosalicylate', #['aminosalicylate sodium', 'phenyl aminosalicylate']
'amiprilose': 'amiprilose', #['amiprilose', 'amiprilose hydrochloride']
#'ammonium': 'ammonium', #['ammonium mandelate', 'ammonium pyrrolidinedithiocarbamate']
'ampicillin': 'ampicillin', #['ampicillin sodium', 'ampicillin trihydrate']
#'ampk': 'ampk', #['ampk activator', 'ampk inhibitor,                  compound c']
'amprolium': 'amprolium', #['amprolium', 'amprolium hydrochloride']
'amsacrine': 'amsacrine', #['amsacrine', 'amsacrine hydrochloride']
'anabasamine': 'anabasamine', #['anabasamine', 'anabasamine hydrochloride']
'androsterone': 'androsterone', #['androsterone', 'androsterone sodium sulfate']
#?'anethole': 'anethole', #['anethole', 'anethole trithione']
'anisodamine': 'anisodamine', #['anisodamine', 'anisodamine hydrobromide']
'anisomycin': 'anisomycin', #['anisomycin', 'anisomycin, streptomyces griseolus']
'antazoline': 'antazoline', #['antazoline hydrochloride', 'antazoline phosphate']
'antipyrine': 'antipyrine', #['antipyrine', 'antipyrine, 4-hydroxy']
'apicidin': 'apicidin', #['apicidin', 'apicidin, fusarium sp.']
#'fusarium': 'fusarium', #['apicidin, fusarium sp.', 'fumonisin b1, fusarium moniliforme']
'apramycin': 'apramycin', #['apramycin', 'apramycin sulfate']
'leelamide': 'leelamide', #['arachidonic acid leelamide', 'lauric acid leelamide']
#'synthetic': 'synthetic', #['arcyriaflavin a, synthetic', 'fascaplysin, synthetic']
'aristolochic': 'aristolochic acid', #['aristolochic acid', 'aristolochic acid a']
#'83': '83', #['arlacel 83', 'fp 83']
'as-605240': 'as-605240', #['as-605240', 'as-605240 (potassium salt)']
'atomoxetine': 'atomoxetine', #['atomoxetine hcl', 'atomoxetine hydrochloride']
'atorvastatin': 'atorvastatin', #['atorvastatin', 'atorvastatin calcium']
'atractyloside': 'atractyloside potassium', #['atractyloside potassium', 'atractyloside potassium salt']
#'b1': 'b1', #['avermectin b1', 'fumonisin b1, fusarium moniliforme']
'avocadene': 'avocadene', #['avocadene', 'avocadene acetate']
'avocatin': 'avocatin', #['avocatin a', 'avocatin b']
'azatadine': 'azatadine', #['azatadine', 'azatadine maleate']
'azelastine': 'azelastine', #['azelastine hcl', 'azelastine hydrochloride']
'azlocillin': 'azlocillin sodium', #['azlocillin sodium', 'azlocillin sodium salt']
#'b2': 'b2', #['b2', 'bleomycin (bleomycin b2 shown)']
'baclofen': 'baclofen', #['baclofen', 'baclofen (r,s)']
#'channel': 'channel', #['bax channel blocker', 'crac channel inhibitor, btp2']
#'blocker': 'blocker', #['bax channel blocker', 'ras/rac transformation blocker, sch 51344']
#'brand': 'brand', #['baxter brand of sorbitol', 'ophthalmic brand of sulthiame']
#'of': 'of', #['baxter brand of sorbitol', 'ophthalmic brand of sulthiame']
'sorbitol': 'sorbitol', #['baxter brand of sorbitol', 'sorbitol']
'beclomethasone': 'beclomethasone', #['beclomethasone', 'beclomethasone dipropionate']
'benazepril': 'benazepril', #['benazepril hcl', 'benazepril hydrochloride']
'benfluorex': 'benfluorex', #['benfluorex', 'benfluorex hydrochloride']
'benidipine': 'benidipine', #['benidipine hcl', 'benidipine hydrochloride']
#?'benzylpenicillin': 'benzylpenicillin', #['benzathine benzylpenicillin', 'benzylpenicillin sodium']
'benzeneacetic': 'benzeneacetic acid', #['benzeneacetic acid, 2-[(2,6-dichlorophenyl)amino]-, monosodium salt [cas]', 'benzeneacetic acid, alpha-(hydroxymethyl)-, 9-methyl-3-oxa-9-azatricyclo[3.3.1.02,4]non-7-yl ester']
'benzethonium': 'benzethonium', #['benzethonium chloride', 'methyl benzethonium chloride']
'benztropine': 'benztropine', #['benztropine', 'benztropine mesylate']
'cinnamate': 'cinnamate', #['benzyl cinnamate', 'cinnamate sodium']
'bephenium': 'bephenium', #['bephenium hydroxynaphthoate', 'bephenium hydroxynapthoate']
'berberine': 'berberine', #['berberine', 'berberine chloride']
'bestatin': 'bestatin', #['bestatin', 'bestatin hydrochloride']
'beta-amyrin': 'beta-amyrin', #['beta-amyrin', 'beta-amyrin acetate']
'betahistine': 'betahistine', #['betahistine hydrochloride', 'betahistine mesylate']
#?'betaine': 'betaine', #['betaine aldehyde chloride', 'betaine hydrochloride']
#'aldehyde': 'aldehyde', #['betaine aldehyde chloride', 'cinnamic aldehyde']
'betaxolol': 'betaxolol', #['betaxolol', 'betaxolol hydrochloride']
'nvp-bez235': 'nvp-bez235', #['bez235 (nvp-bez235)', 'nvp-bez235']
'bicalutamide': 'bicalutamide', #['bicalutamide', 'bicalutamide (cdx)']
'biperiden': 'biperiden', #['biperiden', 'biperiden hydrochloride']
'bisabolol': 'bisabolol', #['bisabolol', 'bisabolol acetate']
'bisoprolol': 'bisoprolol', #['bisoprolol fumarate', 'bisoprolol hemifumarate salt']
'01294': 'bix-01294', #['bix 01294', 'bix 01294 trihydrochloride hydrate']
#'bms': 'bms', #['bms 777607', 'bms 794833']
#'l-tartrate': 'l-tartrate', #['brimonidine l-tartrate', 's(+)-raclopride l-tartrate']
'brivanib': 'brivanib', #['brivanib (bms-540215)', 'brivanib alaninate (bms-582664)']
'bromocriptine': 'bromocriptine', #['bromocriptine', 'bromocriptine mesylate']
'brompheniramine': 'brompheniramine', #['brompheniramine', 'brompheniramine maleate']
'bto-1': 'bto-1', #['bto-1', 'polo-like kinase inhibitor ii, bto-1']
'bucladesine': 'bucladesine', #['bucladesine', 'bucladesine sodium salt']
'buflomedil': 'buflomedil', #['buflomedil hcl', 'buflomedil hydrochloride']
'bupivacaine': 'bupivacaine', #['bupivacaine', 'bupivacaine hydrochloride']
'bupropion': 'bupropion', #['bupropion', 'bupropion hydrochloride']
#?'sulfoximine': 'sulfoximine', #['buthionine sulfoximine', 'l-methionine sulfoximine']
'butirosin': 'butirosin disulfate', #['butirosin disulfate', 'butirosin disulfate salt']
#'disulfate': 'disulfate', #['butirosin disulfate', 'butirosin disulfate salt']
#'paraben': 'paraben', #['butyl paraben', 'ethyl paraben']
#'bw': 'bw', #['bw 284c51', 'bw 723c86']
#'dual': 'dual', #['c-met/ron dual kinase inhibitor', 'pdk1/akt/flt dual pathway inhibitor']
#'ca': 'ca', #['ca 1001', 'itavastatin ca']
'caffeic': 'caffeic acid', #['caffeic acid', 'caffeic acid phenethyl ester']
'cambinol': 'cambinol', #['cambinol', 'sirt1/2 inhibitor iv, cambinol']
#'1r': '1r', #['camphor (1r)', 'hydrastine (1r, 9s)']
'camylofine': 'camylofine', #['camylofine chlorhydrate', 'camylofine dihydrochloride']
'candesartan': 'candesartan', #['candesartan', 'candesartan cilextil']
'canertinib': 'canertinib', #['canertinib (hydrochloride)', 'ci-1033 (canertinib)']
'canrenoic': 'canrenoic acid potassium salt', #['canrenoic acid potassium salt', 'canrenoic acid, potassium salt']
'canthaxanthin': 'canthaxanthin', #['canthaxanthin', 'canthaxanthin (euglenanone)']
'carbenoxolone': 'carbenoxolone', #['carbenoxolone disodium salt', 'carbenoxolone sodium']
'carbinoxamine': 'carbinoxamine', #['carbinoxamine maleate', 'carbinoxamine maleate salt']
'carnitine': 'carnitine', #['carnitine (dl) hydrochloride', 'carnitine chloride']
'castanospermine': 'castanospermine', #['castanospermine', 'castanospermine, castanospermum australe']
'tetramethylether': 'tetramethylether', #['catechin tetramethylether', 'catechin(+) tetramethylether, toluenesulfonate']
'cct018159': 'cct018159', #['cct018159', 'hsp90 inhibitor, cct018159']
'tg003': 'tg003', #['cdc2-like kinase inhibitor, tg003', 'tg003']
'cefaclor': 'cefaclor', #['cefaclor', 'cefaclor hydrate']
#'propylene': 'propylene', #['cefatrizine propylene glycol', 'propylene phenoxetol']
'cefazolin': 'cefazolin sodium', #['cefazolin sodium', 'cefazolin sodium salt']
'cefixime': 'cefixime', #['cefixime', 'cefixime trihydrate']
'cefmetazole': 'cefmetazole sodium', #['cefmetazole sodium', 'cefmetazole sodium salt']
'cefoperazone': 'cefoperazone', #['cefoperazone', 'cefoperazone dihydrate']
'cefotaxime': 'cefotaxime sodium', #['cefotaxime sodium', 'cefotaxime sodium salt']
'cefotiam': 'cefotiam', #['cefotiam', 'cefotiam hydrochloride']
'cefoxitin': 'cefoxitin sodium', #['cefoxitin sodium', 'cefoxitin sodium salt']
'ceftazidime': 'ceftazidime', #['ceftazidime', 'ceftazidime pentahydrate']
'ceftriaxone': 'ceftriaxone sodium', #['ceftriaxone sodium', 'ceftriaxone sodium trihydrate']
'celastrol': 'celastrol', #['celastrol', 'celastrol, celastrus scandens']
'celiprolol': 'celiprolol', #['celiprolol hcl', 'celiprolol hydrochloride']
'cephalosporin': 'cephalosporin c', #['cephalosporin c sodium', 'cephalosporin c zinc salt']
'cephalothin': 'cephalothin', #['cephalothin sodium', 'cephalothin sodium salt']
'cephradine': 'cephradine', #['cephradine', 'cephradine sodium']
'cetirizine': 'cetirizine', #['cetirizine dihydrochloride', 'cetirizine hydrochloride']
#?'cetyl': 'cetyl', #['cetyl alcohol, 14c-labeled', 'cetyl stearyl alcohol']
#'cgp': 'cgp', #['cgp 20712a methanesulfonate', 'cgp 57380']
#'cgs': 'cgs', #['cgs 12066b', 'cgs 15943']
'chaulmoogric': 'chaulmoogric acid', #['chaulmoogric acid', 'chaulmoogric acid, ethyl ester']
'chelerythrine': 'chelerythrine', #['chelerythrine', 'chelerythrine chloride']
#'chk2': 'chk2', #['chk2 inhibitor', 'chk2 inhibitor ii']
'chlorhexidine': 'chlorhexidine', #['chlorhexidine', 'chlorhexidine dihydrochloride']
#'complex': 'complex', #['chlorophyllide cu complex na salt', 'gossypol-acetic acid complex']
'chloroquine': 'chloroquine', #['chloroquine', 'chloroquine diphosphate']
'chlorpheniramine': 'chlorpheniramine', #['chlorpheniramine', 'chlorpheniramine maleate']
'chlorpromazine': 'chlorpromazine', #['chlorpromazine', 'chlorpromazine hydrochloride']
'cholic': 'cholic acid', #['cholic acid', 'cholic acid, methyl ester']
'choline': 'choline', #['choline bromide', 'choline chloride']
'chromomycin': 'chromomycin', #['chromomycin', 'chromomycin a3']
'chrysin': 'chrysin', #['chrysin', 'chrysin dimethyl ether']
'chukrasin': 'chukrasin', #['chukrasin', 'chukrasin methyl ether']
'ciclopirox': 'ciclopirox', #['ciclopirox ethanolamine', 'ciclopirox olamine']
'citalopram': 'citalopram', #['citalopram', 'citalopram hydrobromide']
'clemastine': 'clemastine', #['clemastine', 'clemastine fumarate']
'clomiphene': 'clomiphene', #['clomiphene citrate', 'clomiphene citrate (z,e)']
'clopidogrel': 'clopidogrel', #['clopidogrel', 'clopidogrel sulfate']
'cloxacillin': 'cloxacillin', #['cloxacillin sodium', 'cloxacillin sodium salt']
'colchicine': 'colchicine', #['colchicine', 'colchicine, colchicum autumnale']
'colforsin': 'colforsin', #['colforsin', 'colforsin daropate hydrochloride']
'cotinine': 'cotinine', #['cotinine', 'cotinine fumarate']
#'cox-1': 'cox-1', #['cox-1 inhibitor iv, tfap', 'cox-1 inhibitor, fr122047']
'cr8': 'cr8', #['cr8, (r)-isomer', 'cr8, (s)-isomer']
#'#s)-isomer': 's)-isomer', #['cr8, (s)-isomer', 'roscovitine, (s)-isomer']
'cromolyn': 'cromolyn', #['cromolyn disodium salt', 'cromolyn sodium']
'curcumin': 'curcumin', #['curcumin', 'curcumin, curcuma longa l.']
'cyclizine': 'cyclizine', #['cyclizine', 'cyclizine hydrochloride']
'cyclopiazonic': 'cyclopiazonic acid', #['cyclopiazonic acid', 'cyclopiazonic acid, penicillium cyclopium']
'cycloserine': 'cycloserine', #['cycloserine', 'cycloserine (d)']
'cyclosporin': 'cyclosporin a', #['cyclosporin a', 'cyclosporin a, tolypocladium inflatum']
'cyproterone': 'cyproterone', #['cyproterone', 'cyproterone acetate']
'cytarabine': 'cytarabine', #['cytarabine', 'cytarabine hydrochloride']
#?'c-18': 'c-18', #['d-erythro-sphingosine c-18', 'l-threo-sphingosine c-18']
'dalbergione': 'dalbergione', #['dalbergione', "dalbergione, 4-methoxy-4'-hydroxy"]
'dantrolene': 'dantrolene', #['dantrolene sodium', 'dantrolene sodium salt']
'dasatinib': 'dasatinib', #['dasatinib', 'dasatinib (bms-354825)']
'daunorubicin': 'daunorubicin', #['daunorubicin', 'daunorubicin hydrochloride']
'decamethonium': 'decamethonium', #['decamethonium bromide', 'decamethonium dibromide']
#'dibromide': 'dibromide', #['decamethonium dibromide', 'hexamethonium dibromide dihydrate']
'dehydrocholate': 'dehydrocholate sodium', #['dehydrocholate sodium', 'sodium dehydrocholate']
'demeclocycline': 'demeclocycline', #['demeclocycline', 'demeclocycline hydrochloride']
'deoxyandirobin': 'deoxyandirobin', #['deoxyandirobin', 'deoxyandirobin lactone']
#'7,3'-dimethyl': '7,3'-dimethyl', #["deoxysappanone b 7,3'-dimethyl ether", "deoxysappanone b 7,3'-dimethyl ether acetate"]
'dextromethorphan': 'dextromethorphan', #['dextromethorphan hydrobromide', 'dextromethorphan hydrobromide monohydrate']
'diatrizoic': 'diatrizoic acid', #['diatrizoic acid', 'diatrizoic acid dihydrate']
'dibucaine': 'dibucaine', #['dibucaine', 'dibucaine hydrochloride']
'diclofenac': 'diclofenac', #['diclofenac acid', 'diclofenac sodium']
'dicloxacillin': 'dicloxacillin', #['dicloxacillin sodium', 'dicloxacillin sodium salt hydrate']
'digitoxigenin': 'digitoxigenin', #['digitoxigenin', 'digitoxigenin acetate']
'dihydrogedunin': 'dihydrogedunin', #['dihydrogedunin', 'dihydrogedunin ethanedithioketal']
'dilazep': 'dilazep', #['dilazep dihydrochloride', 'dilazep hydrochloride']
#'furoate': 'furoate', #['diloxanide furoate', 'mometasone furoate']
#'magnesium': 'magnesium', #['dipyrithione magnesium sulfate', 'esomeprazole magnesium dihydrate']
'disopyramide': 'disopyramide', #['disopyramide', 'disopyramide phosphate']
#'dna': 'dna', #['dna base excision repair pathway inhibitor', 'dna methyltransferase inhibitor']
#'mesilate': 'mesilate', #['dolasetron mesilate', 'gabexate mesilate']
'donepezil': 'donepezil', #['donepezil', 'donepezil hydrochloride']
#'dopamine': 'dopamine', #['dopamine hydrochloride', 'oleoyl dopamine']
'doxepin': 'doxepin', #['doxepin', 'doxepin hydrochloride']
'doxorubicin': 'doxorubicin', #['doxorubicin', 'doxorubicin hydrochloride']
'doxycycline': 'doxycycline', #['doxycycline', 'doxycycline hydrochloride']
'dropropizine': 'dropropizine', #['dropropizine', 'dropropizine (r,s)']
'duartin': 'duartin', #['duartin (-)', 'duartin, dimethyl ether']
'eburnamonine': 'eburnamonine', #['eburnamonine', 'eburnamonine (-)']
'nh125': 'nh125', #['eef-2 kinase inhibitor, nh125', 'nh125']
'ellagic': 'ellagic acid', #['ellagic acid', 'ellagic acid, dihydrate']
'enalapril': 'enalapril', #['enalapril', 'enalapril maleate']
'enalaprilat': 'enalaprilat', #['enalaprilat', 'enalaprilat dihydrate']
#'3,5-digallate': '3,5-digallate', #['epicatechin 3,5-digallate', 'epigallocatechin 3,5-digallate']
'ergosterol': 'ergosterol', #['ergosterol', 'ergosterol acetate']
#'lauryl': 'lauryl', #['erythromycin propionate lauryl sulfate', 'sodium lauryl sulfoacetate']
'esomeprazole': 'esomeprazole', #['esomeprazole magnesium dihydrate', 'esomeprazole mg']
#'mg': 'mg', #['esomeprazole mg', 'mg 624']
#'beta': 'beta', #['estradiol-17 beta', 'k-strophanthin beta']
#'lactate': 'lactate', #['ethacridine lactate', 'prenylamine lactate']
'ethambutol': 'ethambutol', #['ethambutol dihydrochloride', 'ethambutol hydrochloride']
'ethynylestradiol': 'ethynylestradiol', #['ethynylestradiol', 'ethynylestradiol 3-methyl ether']
'euphol': 'euphol', #['euphol', 'euphol acetate']
#'g': 'g', #['eutanol g', 'penicillin g potassium']
'felbinac': 'felbinac', #['felbinac', 'felbinac ethyl ester']
'fenoprofen': 'fenoprofen', #['fenoprofen', 'fenoprofen calcium salt dihydrate']
'pd173074': 'pd173074', #['fgf/vegf receptor tyrosine kinase inhibitor, pd173074', 'pd173074']
#?'fissinolide': 'fissinolide', #['fissinolide', 'fissinolide c']
'fludarabine': 'fludarabine', #['fludarabine', 'fludarabine phosphate']
'flumethasone': 'flumethasone', #['flumethasone', 'flumethasone pivalate']
'flunarizine': 'flunarizine', #['flunarizine dihydrochloride', 'flunarizine hydrochloride']
#'meglumine': 'meglumine', #['flunixin meglumine', 'meglumine']
#'acetonide': 'acetonide', #['fluocinolone acetonide', 'triamcinolone acetonide']
'fluoxetine': 'fluoxetine', #['fluoxetine', 'fluoxetine hydrochloride']
'fluvoxamine': 'fluvoxamine', #['fluvoxamine', 'fluvoxamine maleate']
'fomepizole': 'fomepizole', #['fomepizole', 'fomepizole hydrochloride']
'forskolin': 'forskolin', #['forskolin', 'forskolin, coleus forskohlii']
'foscarnet': 'foscarnet', #['foscarnet', 'foscarnet sodium']
'fosinopril': 'fosinopril', #['fosinopril', 'fosinopril sodium']
'furaltadone': 'furaltadone', #['furaltadone', 'furaltadone hydrochloride']
'gabexate': 'gabexate', #['gabexate mesilate', 'gabexate mesylate']
'galanthamine': 'galanthamine', #['galanthamine', 'galanthamine hydrobromide']
'gallamine': 'gallamine', #['gallamine', 'gallamine triethiodide']
'gangleoidin': 'gangleoidin', #['gangleoidin', 'gangleoidin acetate']
#'gbr': 'gbr', #['gbr 12909', 'gbr 12909 dihydrochloride']
'12909': 'gbr-12909', #['gbr 12909', 'gbr 12909 dihydrochloride']
'gemcitabine': 'gemcitabine', #['gemcitabine', 'gemcitabine hydrochloride']
'genistein': 'genistein', #['genistein', 'genistein, 8-methyl']
'gitoxigenin': 'gitoxigenin', #['gitoxigenin', 'gitoxigenin diacetate']
'glafenine': 'glafenine', #['glafenine', 'glafenine hydrochloride']
'glutamine': 'glutamine', #['glutamine', 'glutamine (l)']
'guaiacol': 'guaiacol', #['guaiacol', 'guaiacol carbonate']
'guanethidine': 'guanethidine', #['guanethidine monosulfate', 'guanethidine sulfate']
'h-89': 'h-89', #['h-89', 'h-89, dihydrochloride']
#'ha': 'ha', #['ha 1004, dihydrochloride', 'ha 1077, dihydrochloride                              fasudil']
'haematoxylin': 'haematoxylin', #['haematoxylin', 'haematoxylin pentaacetate']
'havanensin': 'havanensin', #['havanensin', 'havanensin triacetate']
#'protein': 'protein', #['heat shock protein inhibitor i', 'protein methyltransferase inhibitor, ami-5']
'hecogenin': 'hecogenin', #['hecogenin', 'hecogenin acetate']
'hexamethonium': 'hexamethonium', #['hexamethonium bromide', 'hexamethonium dibromide dihydrate']
#'hh/gli': 'hh/gli', #['hh/gli antagonist, gant58', 'hh/gli antagonist, gant61']
'histamine': 'histamine', #['histamine dihydrochloride', 'histamine, r(-)-alpha-methyl-, dihydrochloride']
#'acetyltransferase': 'acetyltransferase', #['histone acetyltransferase activator, ctb', 'histone acetyltransferase inhibitor ii']
#'lysine': 'lysine', #['histone lysine methyltransferase inhibitor', 'lysine']
'hydrocotarnine': 'hydrocotarnine', #['hydrocotarnine', 'hydrocotarnine hbr']
'hydroxychloroquine': 'hydroxychloroquine', #['hydroxychloroquine', 'hydroxychloroquine sulfate']
'hydroxyprogesterone': 'hydroxyprogesterone', #['hydroxyprogesterone', 'hydroxyprogesterone caproate']
'hydroxytacrine': 'hydroxytacrine', #['hydroxytacrine maleate', 'hydroxytacrine maleate (r,s)']
'hydroxyzine': 'hydroxyzine', #['hydroxyzine dihydrochloride', 'hydroxyzine pamoate']
'hymecromone': 'hymecromone', #['hymecromone', 'hymecromone methyl ether']
'hyoscyamine': 'hyoscyamine', #['hyoscyamine', 'hyoscyamine (l)']
'538': 'tyrphostin ag 538', #['i-ome-tyrphostin ag 538', 'tyrphostin ag 538']
#'ici': 'ici', #['ici 204,448 hydrochloride', 'ici 63,137']
'idarubicin': 'idarubicin', #['idarubicin', 'idarubicin hcl']
'ifenprodil': 'ifenprodil', #['ifenprodil', 'ifenprodil tartrate']
#'ikk': 'ikk', #['ikk inhibitor vii', 'ikk inhibitor x']
'imipenem': 'imipenem', #['imipenem', 'imipenem monohydrate']
'indatraline': 'indatraline', #['indatraline', 'indatraline hydrochloride']
'indinavir': 'indinavir', #['indinavir sulfate', 'indinavir sulphate']
#'indirubin': 'indirubin', #['indirubin', 'indirubin derivative e804']
#'derivative': 'derivative', #['indirubin derivative e804', 'raf265 derivative']
"indirubin-3'-monoxime": "indirubin-3'-monoxime", #["indirubin-3'-monoxime", "indirubin-3'-monoxime, 5-iodo-"]
'ipa-3': 'ipa-3', #['ipa-3', 'p21-activated kinase inhibitor iii, ipa-3']
'iproniazid': 'iproniazid', #['iproniazid phosphate', 'iproniazid sulfate']
#'irak-1/4': 'irak-1/4', #['irak-1/4 inhibitor', 'irak-1/4 inhibitor i']
'irigenin': 'irigenin', #['irigenin trimethyl ether', 'irigenin, 7-benzyl ether']
'isoetharine': 'isoetharine mesylate', #['isoetharine mesylate', 'isoetharine mesylate salt']
#'isopropyl': 'isopropyl', #['isopropyl palmitate', 'isopropyl unoprostone']
'unoprostone': 'unoprostone', #['isopropyl unoprostone', 'unoprostone']
'itopride': 'itopride', #['itopride', 'itopride hcl']
#'jak2': 'jak2', #['jak2 inhibitor ii', 'jak2 inhibitor iii, sd-1029']
'jx401': 'jx401', #['jx401', 'p38 map kinase inhibitor vi, jx401']
'kainic': 'kainic acid', #['kainic acid', 'kainic acid, natural']
'kanamycin': 'kanamycin', #['kanamycin a sulfate', 'kanamycin sulfate']
#'differentiation': 'differentiation', #['keratinocyte differentiation inducer', 'neuronal differentiation inducer iii']
'ketanserin': 'ketanserin', #['ketanserin tartrate', 'ketanserin tartrate hydrate']
'ketorolac': 'ketorolac', #['ketorolac tris salt', 'ketorolac tromethamine']
'khayasin': 'khayasin', #['khayasin', 'khayasin c']
#?'kinetin': 'kinetin', #['kinetin', 'kinetin riboside']
'kn-92': 'kn-92', #['kn-92', 'kn-92 (hydrochloride)']
#?'l-3,4-dihydroxyphenylalanine': 'l-3,4-dihydroxyphenylalanine', #['l-3,4-dihydroxyphenylalanine', 'l-3,4-dihydroxyphenylalanine methyl ester hydrochloride']
'l-canavanine': 'l-canavanine', #['l-canavanine', 'l-canavanine sulfate']
'l-mimosine': 'l-mimosine', #['l-mimosine', 'l-mimosine from koa hoale seeds']
'lanosterol': 'lanosterol', #['lanosterol', 'lanosterol acetate']
#'lauric': 'lauric', #['lauric acid diethanolamide', 'lauric acid leelamide']
'leelamine': 'leelamine', #['leelamine', 'leelamine (hydrochloride)']
#'monoacetate': 'monoacetate', #['leoidin monoacetate', 'resorcinol monoacetate']
'lercanidipine': 'lercanidipine', #['lercanidipine', 'lercanidipine hydrochloride hemihydrate']
'leucovorin': 'leucovorin', #['leucovorin', 'leucovorin calcium']
'levulinic': 'levulinic acid', #['levulinic acid', 'levulinic acid,  3-benzylidenyl-']
'lincomycin': 'lincomycin', #['lincomycin', 'lincomycin hydrochloride']
'liothyronine': 'liothyronine', #['liothyronine', 'liothyronine (l- isomer) sodium']
'lobeline': 'lobeline', #['lobeline hydrochloride', 'lobeline sulfate']
'lonchocarpic': 'lonchocarpic acid', #['lonchocarpic acid', 'lonchocarpic acid acetate']
'loxoprofen': 'loxoprofen', #['loxoprofen', 'loxoprofen sodium']
'luciferase': 'luciferase', #['luciferase inhibitor i', 'luciferase inhibitor ii']
'294002': 'ly-294002', #['ly 294002', "ly 294002, 4'-nh2"]
'marmesin': 'marmesin', #['marmesin', 'marmesin acetate']
'mechlorethamine': 'mechlorethamine', #['mechlorethamine', 'mechlorethamine hydrochloride']
'medroxyprogesterone': 'medroxyprogesterone', #['medroxyprogesterone', 'medroxyprogesterone acetate']
'mefexamide': 'mefexamide', #['mefexamide', 'mefexamide hydrochloride']
'mefloquine': 'mefloquine', #['mefloquine', 'mefloquine hydrochloride']
#'mek': 'mek', #['mek inhibitor i', 'mek inhibitor ii']
#'mek1/2': 'mek1/2', #['mek1/2 inhibitor', 'mek1/2 inhibitor ii']
'meloxicam': 'meloxicam', #['meloxicam', 'meloxicam sodium']
'melphalan': 'melphalan', #['melphalan', 'melphalan hydrochloride']
'mephentermine': 'mephentermine', #['mephentermine hemisulfate', 'mephentermine sulfate']
#'meriolin': 'meriolin', #['meriolin 2', 'meriolin 3']
'merogedunin': 'merogedunin', #['merogedunin', 'merogedunin, 3-deoxo-1alpha,3alpha-dihydroxydihydro-']
'mesoridazine': 'mesoridazine', #['mesoridazine', 'mesoridazine besylate']
'methacycline': 'methacycline', #['methacycline', 'methacycline hydrochloride']
'methiothepin': 'methiothepin', #['methiothepin maleate', 'methiothepin mesylate']
#'tetrahydrochloride': 'tetrahydrochloride', #['methoctramine tetrahydrochloride', 'spermine tetrahydrochloride']
'methotrexate': 'methotrexate', #['methotrexate', 'methotrexate hydrate']
#'nicotinate': 'nicotinate', #['methyl nicotinate', 'xanthinol nicotinate']
'robustone': 'robustone', #['methyl robustone', 'robustone']
'methyldopa': 'methyldopa', #['methyldopa', 'methyldopa (l,-)']
#'sulfonate': 'sulfonate', #['methylmethane sulfonate', 'phenol sulfonate']
'metoclopramide': 'metoclopramide', #['metoclopramide hydrochloride', 'metoclopramide monohydrochloride']
#'monohydrochloride': 'monohydrochloride', #['metoclopramide monohydrochloride', 'sematilide monohydrochloride monohydrate']
'mevastatin': 'mevastatin', #['mevastatin', 'mevastatin, sodium salt']
'mexicanolide': 'mexicanolide', #['mexicanolide', 'mexicanolide enol ether']
#'mhpg': 'mhpg', #['mhpg piperazine', 'mhpg sulfate potassium']
#'piperazine': 'piperazine', #['mhpg piperazine', 'piperazine']
'miconazole': 'miconazole', #['miconazole', 'miconazole nitrate']
'milnacipran': 'milnacipran', #['milnacipran', 'milnacipran hydrochloride']
'minaprine': 'minaprine', #['minaprine dihydrochloride', 'minaprine hydrochloride']
'mitomycin': 'mitomycin', #['mitomycin', 'mitomycin c']
'ml-7': 'ml-7', #['ml-7', 'ml-7, hydrochloride']
'morantel': 'morantel', #['morantel citrate', 'morantel tartrate']
'moxalactam': 'moxalactam', #['moxalactam disodium', 'moxalactam disodium salt']
'moxisylyte': 'moxisylyte', #['moxisylyte hydrochloride', 'moxisylyte hydrochoride']
'myriocin': 'myriocin', #['myriocin', 'myriocin, mycelia sterilia']
'nafamostat': 'nafamostat', #['nafamostat', 'nafamostat mesylate']
'nafcillin': 'nafcillin', #['nafcillin sodium', 'nafcillin sodium salt monohydrate']
'naftopidil': 'naftopidil', #['naftopidil', 'naftopidil dihydrochloride']
'nalbuphine': 'nalbuphine', #['nalbuphine', 'nalbuphine hydrochloride']
'naltrexone': 'naltrexone', #['naltrexone hydrochloride', 'naltrexone hydrochloride dihydrate']
'naltrindole': 'naltrindole', #['naltrindole', 'naltrindole hydrochloride']
'naproxen': 'naproxen', #['naproxen', 'naproxen sodium']
'nefopam': 'nefopam', #['nefopam', 'nefopam hydrochloride']
'nelfinavir': 'nelfinavir', #['nelfinavir', 'nelfinavir mesylate']
'neomycin': 'neomycin', #['neomycin', 'neomycin sulfate']
#?'ng-nitro-l-arginine': 'ng-nitro-l-arginine', #['ng-nitro-l-arginine', 'ng-nitro-l-arginine methyl ester hydrochloride']
'nicardipine': 'nicardipine', #['nicardipine', 'nicardipine hydrochloride']
'nicotine': 'nicotine', #['nicotine bitartrate', 'nicotine ditartrate']
#'ditartrate': 'ditartrate', #['nicotine ditartrate', 'pentolinium ditartrate']
'nicotinyl': 'nicotinyl', #['nicotinyl alcohol tartrate', 'nicotinyl tartrate']
'nifekalant': 'nifekalant', #['nifekalant', 'nifekalant hcl']
'nigericin': 'nigericin', #['nigericin sodium', 'nigericin, sodium salt, streptomyces hygroscopicus']
'nilotinib': 'nilotinib', #['nilotinib', 'nilotinib (amn-107)']
'nimustine': 'nimustine', #['nimustine', 'nimustine hydrochloride']
'nordihydroguaiaretic': 'nordihydroguaiaretic acid', #['nordihydroguaiaretic acid', 'nordihydroguaiaretic acid from larrea divaricata (creosote bush)']
'norethindrone': 'norethindrone', #['norethindrone', 'norethindrone acetate']
'norstictic': 'norstictic acid', #['norstictic acid', 'norstictic acid pentaacetate']
'nortriptyline': 'nortriptyline', #['nortriptyline', 'nortriptyline hydrochloride']
'noscapine': 'noscapine', #['noscapine hydrchloride', 'noscapine hydrochloride']
'novobiocin': 'novobiocin', #['novobiocin sodium', 'novobiocin sodium salt']
'nylidrin': 'nylidrin', #['nylidrin', 'nylidrin hydrochloride']
'oleanolic': 'oleanolic acid', #['oleanolic acid', 'oleanolic acid acetate']
'olmesartan': 'olmesartan', #['olmesartan', 'olmesartan medoxomil']
'olomoucine': 'olomoucine', #['olomoucine', 'olomoucine ii']
'orphenadrine': 'orphenadrine', #['orphenadrine citrate', 'orphenadrine hydrochloride']
'orsellinic': 'orsellinic acid', #['orsellinic acid', 'orsellinic acid, ethyl ester']
'ouabain': 'ouabain', #['ouabain', 'ouabain, octahydrate']
#'sesquifumarate': 'sesquifumarate', #['oxotremorine sesquifumarate', 'oxotremorine sesquifumarate salt']
'oxyquinoline': 'oxyquinoline', #['oxyquinoline', 'oxyquinoline sulfate']
'oxytetracycline': 'oxytetracycline', #['oxytetracycline', 'oxytetracycline dihydrate']
'sd-169': 'sd-169', #['p38 map kinase inhibitor vii, sd-169', 'sd-169']
'paclitaxel': 'paclitaxel', #['paclitaxel', 'paclitaxel, taxus sp.']
'palonosetron': 'palonosetron', #['palonosetron hcl', 'palonosetron hydrochloride']
'pancuronium': 'pancuronium', #['pancuronium', 'pancuronium bromide']
'pantoprazole': 'pantoprazole', #['pantoprazole', 'pantoprazole sodium']
'parthenolide': 'parthenolide', #['parthenolide', 'parthenolide, tanacetum parthenium']
'pazopanib': 'pazopanib', #['pazopanib', 'pazopanib hcl']
'pefloxacine': 'pefloxacine', #['pefloxacine', 'pefloxacine mesylate']
#?'penicillin': 'penicillin', #['penicillin g potassium', 'penicillin v potassium']
'pergolide': 'pergolide', #['pergolide mesylate', 'pergolide methanesulfonate']
'perillic': 'perillic acid', #['perillic acid', 'perillic acid (-)']
'perindopril': 'perindopril', #['perindopril', 'perindopril erbumine']
'perospirone': 'perospirone', #['perospirone', 'perospirone hcl']
'phenelzine': 'phenelzine', #['phenelzine sulfate', 'phenelzine sulfate salt']
'phenethicillin': 'phenethicillin', #['phenethicillin potassium', 'phenethicillin potassium salt']
'phentolamine': 'phentolamine', #['phentolamine hydrochloride', 'phentolamine mesylate']
'phenylpropanolamine': 'phenylpropanolamine', #['phenylpropanolamine hydrochloride', 'phenylpropanolamine maleate']
#'3-ka': '3-ka', #['pi 3-ka inhibitor iv', 'pi 3-ka inhibitor viii']
'tgx-221': 'tgx-221', #['pi 3-kß inhibitor vi, tgx-221', 'tgx-221']
'picotamide': 'picotamide', #['picotamide', 'picotamide monohydrate']
#?'picrotin': 'picrotin', #['picrotin', 'picrotin - picrotoxinin']
#?'picrotoxinin': 'picrotoxinin', #['picrotin - picrotoxinin', 'picrotoxinin']
#'pik-75': 'pik-75', #['pik-75', 'pik-75 (hydrochloride)']
#'pim1': 'pim1', #['pim1 kinase inhibitor ii', 'pim1 kinase inhibitor iv']
#'pim1/2': 'pim1/2', #['pim1/2 kinase inhibitor v', 'pim1/2 kinase inhibitor vi']
'pimozide': 'pimozide', #['pimozide', 'stat5 inhibitor iii, pimozide']
'pinacidil': 'pinacidil', #['pinacidil', 'pinacidil monohydrate']
'pinosylvin': 'pinosylvin', #['pinosylvin', 'pinosylvin methyl ether']
'piperacillin': 'piperacillin', #['piperacillin sodium', 'piperacillin sodium salt']
'pirenzepine': 'pirenzepine', #['pirenzepine dihydrochloride', 'pirenzepine hydrochloride']
'pizotyline': 'pizotyline', #['pizotyline', 'pizotyline malate']
#'pkr': 'pkr', #['pkr inhibitor', 'pkr inhibitor, negative control']
#'polo-like': 'polo-like', #['polo-like kinase inhibitor i', 'polo-like kinase inhibitor ii, bto-1']
'pralidoxime': 'pralidoxime', #['pralidoxime chloride', 'pralidoxime mesylate']
'prasterone': 'prasterone', #['prasterone', 'prasterone acetate']
'pravastatin': 'pravastatin', #['pravastatin', 'pravastatin sodium']
'prazosin': 'prazosin', #['prazosin', 'prazosin hydrochloride']
'pridinol': 'pridinol', #['pridinol methanesulfonate', 'pridinol methanesulfonate salt']
'primaquine': 'primaquine', #['primaquine diphosphate', 'primaquine phosphate']
'procainamide': 'procainamide', #['procainamide hcl', 'procainamide hydrochloride']
'prochlorperazine': 'prochlorperazine', #['prochlorperazine dimaleate', 'prochlorperazine edisylate']
'propranolol': 'propranolol', #['propranolol hydrochloride', 'propranolol hydrochloride (+/-)']
'proscillaridin': 'proscillaridin', #['proscillaridin', 'proscillaridin a']
'protoporphyrin': 'protoporphyrin ix', #['protoporphyrin ix', 'protoporphyrin ix disodium']
'protoveratrine': 'protoveratrine', #['protoveratrine a', 'protoveratrine b']
'psoromic': 'psoromic acid', #['psoromic acid', 'psoromic acid hydrate']
'pterostilbene': 'pterostilbene', #['pterostilbene', 'pterostilbene, pterocarpus marsupium']
'puromycin': 'puromycin', #['puromycin', 'puromycin hydrochloride']
'purpurogallin': 'purpurogallin', #['purpurogallin', 'purpurogallin tetramethyl ether']
'pyrantel': 'pyrantel', #['pyrantel pamoate', 'pyrantel tartrate']
'pyrazinecarboxamide': 'pyrazinecarboxamide', #['pyrazinecarboxamide', 'pyrazinecarboxamide, 3,5-diamino-n-(aminoiminomethyl)-6-chloro- [cas]']
'pyridoxine': 'pyridoxine', #['pyridoxine', 'pyridoxine hydrochloride']
'quetiapine': 'quetiapine', #['quetiapine', 'quetiapine hemifumarate']
'quinapril': 'quinapril', #['quinapril hcl', 'quinapril hydrochloride']
'quinine': 'quinine', #['quinine ethyl carbonate', 'quinine sulfate']
'r788': 'r788', #['r788 (fostamatinib)', 'r935788 (fostamatinib disodium, r788 disodium)']
'fostamatinib': 'fostamatinib', #['r788 (fostamatinib)', 'r935788 (fostamatinib disodium, r788 disodium)']
#'raf': 'raf', #['raf kinase inhibitor iv', 'raf kinase inhibitor v']
'raloxifene': 'raloxifene', #['raloxifene', 'raloxifene hydrochloride']
'ranitidine': 'ranitidine', #['ranitidine', 'ranitidine hydrochloride']
'ranolazine': 'ranolazine', #['ranolazine', 'ranolazine dihydrochloride']
'rapamycin': 'rapamycin', #['rapamycin', 'rapamycin (sirolimus)']
'sirolimus': 'sirolimus', #['rapamycin (sirolimus)', 'sirolimus']
#'sch': 'sch', #['ras/rac transformation blocker, sch 51344', 'sch 58261']
'rauwolscine': 'rauwolscine', #['rauwolscine', 'rauwolscine hydrochloride']
'resorcinol': 'resorcinol', #['resorcinol', 'resorcinol monoacetate']
'resveratrol': 'resveratrol', #['resveratrol', "resveratrol 4'-methyl ether"]
'retinyl': 'retinyl', #['retinyl acetate', 'retinyl palmitate']
'rockout': 'rockout', #['rho kinase inhibitor iii, rockout', 'rockout']
#?'rhodomyrtoxin': 'rhodomyrtoxin', #['rhodomyrtoxin', 'rhodomyrtoxin b']
'ribostamycin': 'ribostamycin', #['ribostamycin sulfate', 'ribostamycin sulfate salt']
'rilmenidine': 'rilmenidine', #['rilmenidine', 'rilmenidine hemifumarate']
'riluzole': 'riluzole', #['riluzole', 'riluzole hydrochloride']
'robustic': 'robustic acid', #['robustic acid', 'robustic acid methyl ether']
#'7': '7', #['rohituka 7', 'toll-like receptor 7 ligand ii']
'ropinirole': 'ropinirole', #['ropinirole hcl', 'ropinirole hydrochloride']
'rotenonic': 'rotenonic acid', #['rotenonic acid', 'rotenonic acid, methyl ether']
'roxatidine': 'roxatidine', #['roxatidine acetate hcl', 'roxatidine acetate hydrochloride']
'rufloxacin': 'rufloxacin', #['rufloxacin', 'rufloxacin hcl']
'ruxolitinib': 'ruxolitinib', #['ruxolitinib', 'ruxolitinib (incb018424)']
'incb018424': 'ruxolitinib', #['ruxolitinib (incb018424)', 's-ruxolitinib (incb018424)']
's-nitroso-n-acetylpenicillamine': 's-nitroso-n-acetylpenicillamine', #['s-nitroso-n-acetylpenicillamine', 'snap (s-nitroso-n-acetylpenicillamine)']
'salbutamol': 'salbutamol', #['salbutamol', 'salbutamol sulfate']
'salermide': 'salermide', #['salermide', 'sirt1/2 inhibitor viii, salermide']
'salinomycin': 'salinomycin', #['salinomycin', 'salinomycin, sodium']
'salmeterol': 'salmeterol', #['salmeterol', 'salmeterol xinafoate']
'sarafloxacin': 'sarafloxacin', #['sarafloxacin', 'sarafloxacin hydrochloride']
#'sulfone': 'sulfone', #['sb 203580, sulfone', 'sulindac sulfone']
'scandenin': 'scandenin', #['scandenin', 'scandenin diacetate']
'sch-202676': 'sch-202676', #['sch-202676', 'sch-202676 hydrobromide']
#'semicarbazide': 'semicarbazide', #['semicarbazide hydrochloride', 'strophanthidin semicarbazide']
'sennoside': 'sennoside', #['sennoside a', 'sennoside b']
'sericetin': 'sericetin', #['sericetin', 'sericetin diacetate']
'sertraline': 'sertraline', #['sertraline', 'sertraline hydrochloride']
#'sib': 'sib', #['sib 1757', 'sib 1893']
'sildenafil': 'sildenafil', #['sildenafil', 'sildenafil citrate']
#?'sulfadiazine': 'sulfadiazine', #['silver sulfadiazine', 'sulfadiazine']
'simvastatin': 'simvastatin', #['simvastatin', 'simvastatin, sodium salt']
#'sirt1': 'sirt1', #['sirt1 inhibitor iii', 'sirt1 inhibitor iv, (s)-35']
#'sirt1/2': 'sirt1/2', #['sirt1/2 inhibitor iv, cambinol', 'sirt1/2 inhibitor viii, salermide']
'smilagenin': 'smilagenin', #['smilagenin', 'smilagenin acetate']
'sns-314': 'sns-314', #['sns-314', 'sns-314 mesylate']
#'sp': 'sp', #['sp 600125', 'z)-7-oxozeaenol, curvularia sp']
'spermidine': 'spermidine', #['spermidine', 'spermidine trihydrochloride']
'spiperone': 'spiperone', #['spiperone', 'spiperone hydrochloride']
'wp1066': 'wp1066', #['stat3 inhibitor iii, wp1066', 'wp1066']
'stattic': 'stattic', #['stat3 inhibitor v, stattic', 'stattic']
#'stat5': 'stat5', #['stat5 inhibitor', 'stat5 inhibitor iii, pimozide']
'sulfacetamide': 'sulfacetamide', #['sulfacetamide', 'sulfacetamide sodic hydrate']
'sulfamethazine': 'sulfamethazine', #['sulfamethazine', 'sulfamethazine sodium salt']
'sulfisoxazole': 'sulfisoxazole', #['sulfisoxazole', 'sulfisoxazole acetyl']
'sumatrol': 'sumatrol', #['sumatrol', 'sumatrol acetate']
'suprofen': 'suprofen', #['suprofen', 'suprofen methyl ester']
'swietenolide': 'swietenolide', #['swietenolide', 'swietenolide diacetate']
'tacrolimus': 'tacrolimus', #['tacrolimus', 'tacrolimus, fk-506, fujimycin']
'tamoxifen': 'tamoxifen', #['tamoxifen', 'tamoxifen citrate']
'telenzepine': 'telenzepine', #['telenzepine dihydrochloride', 'telenzepine hydrochloride']
'terazosin': 'terazosin', #['terazosin', 'terazosin hydrochloride']
'testosterone': 'testosterone', #['testosterone', 'testosterone propionate']
'tetracycline': 'tetracycline', #['tetracycline', 'tetracycline hydrochloride']
#'tgf-b': 'tgf-b', #['tgf-b ri inhibitor iii', 'tgf-b ri kinase inhibitor']
#'ri': 'ri', #['tgf-b ri inhibitor iii', 'tgf-b ri kinase inhibitor']
'theophylline': 'theophylline', #['theophylline', 'theophylline monohydrate']
'thiamine': 'thiamine', #['thiamine', 'thiamine hydrochloride']
'thyroxine': 'thyroxine', #['thyroxine', 'thyroxine (l)']
'ticarcillin': 'ticarcillin', #['ticarcillin disodium', 'ticarcillin sodium']
'ticlopidine': 'ticlopidine', #['ticlopidine hcl', 'ticlopidine hydrochloride']
'timolol': 'timolol', #['timolol maleate', 'timolol maleate salt']
'tiratricol': 'tiratricol', #['tiratricol', "tiratricol, 3,3',5-triiodothyroacetic acid"]
'tizanidine': 'tizanidine', #['tizanidine hcl', 'tizanidine hydrochloride']
#'tnf-a': 'tnf-a', #['tnf-a antagonist iii, r-7050', 'tnf-a inhibitor']
'tobramycin': 'tobramycin', #['tobramycin', 'tobramycin, free base']
#'free': 'free', #['tobramycin, free base', 'xl-184 free base (cabozantinib)']
'tocainide': 'tocainide', #['tocainide', 'tocainide hydrochloride']
'tolmetin': 'tolmetin', #['tolmetin sodium', 'tolmetin sodium salt dihydrate']
'tomatidine': 'tomatidine', #['tomatidine', 'tomatidine hydrochloride']
'tomoxetine': 'tomoxetine', #['tomoxetine', 'tomoxetine hydrochloride']
'toremifene': 'toremifene', #['toremifene', 'toremifene citrate']
#'torin': 'torin', #['torin 1', 'torin 2']
'tosufloxacin': 'tosufloxacin', #['tosufloxacin hydrochloride', 'tosufloxacin tosylate']
#'tpl2': 'tpl2', #['tpl2 kinase inhibitor', 'tpl2 kinase inhibitor ii']
'tracazolate': 'tracazolate', #['tracazolate', 'tracazolate hydrochloride']
'tramadol': 'tramadol', #['tramadol', 'tramadol hydrochloride']
'trichlormethine': 'trichlormethine', #['trichlormethine', 'trichlormethine hydrochloride']
'trifluoperazine': 'trifluoperazine', #['trifluoperazine dihydrochloride', 'trifluoperazine hydrochloride']
'trimebutine': 'trimebutine', #['trimebutine', 'trimebutine maleate']
'trimipramine': 'trimipramine', #['trimipramine maleate', 'trimipramine maleate salt']
'tripelennamine': 'tripelennamine', #['tripelennamine citrate', 'tripelennamine hydrochloride']
'triptolide': 'triptolide', #['triptolide', 'triptolide, tripterygium wilfordii']
'tropisetron': 'tropisetron', #['tropisetron hcl', 'tropisetron hydrochloride']
'tryptamine': 'tryptamine', #['tryptamine', 'tryptamine hydrochloride']
'tulobuterol': 'tulobuterol', #['tulobuterol', 'tulobuterol hydrochloride']
'tylosin': 'tylosin', #['tylosin', 'tylosin tartrate']
'u-74389g': 'u-74389g', #['u-74389g', 'u-74389g maleate']
#'uch-l1': 'uch-l1', #['uch-l1 inhibitor', 'uch-l1 inhibitor ii']
'undecylenic': 'undecylenic acid', #['undecylenic acid', 'undecylenic acid monoethanolamine sulfosuccinate sodium']
#'uracil': 'uracil', #['uracil', 'uracil mustard']
#?'uridine': 'uridine', #["uridine 5'-diphosphate sodium", 'uridine triphosphate trisodium']
'ursolic': 'ursolic acid', #['ursolic acid', 'ursolic acid acetate']
'vancomycin': 'vancomycin', #['vancomycin hydrochloride', 'vancomycin hydrochloride from streptomyces orientalis']
'vandetanib': 'vandetanib', #['vandetanib', 'vandetanib (zactima)']
'vatalanib': 'vatalanib', #['vatalanib', 'vatalanib dihydrochloride (ptk787)']
'vigabatrin': 'vigabatrin', #['vigabatrin', 'vigabatrin hydrochloride']
'vinblastine': 'vinblastine', #['vinblastine sulfate', 'vinblastine sulfate salt']
'vinorelbine': 'vinorelbine', #['vinorelbine', 'vinorelbine bitatrate']
#'wee1': 'wee1', #['wee1 inhibitor', 'wee1 inhibitor ii']
#'win': 'win', #['win 55,212-2', 'win 62,577']
#'wnt': 'wnt', #['wnt agonist', 'wnt antagonist i, iwr-1-endo']
'xylazine': 'xylazine', #['xylazine', 'xylazine hydrochloride']
'zidovudine': 'zidovudine', #['zidovudine [azt]', 'zidovudine, azt']
#'azt': 'azt', #['zidovudine [azt]', 'zidovudine, azt']
'zimelidine': 'zimelidine', #['zimelidine dihydrochloride', 'zimelidine dihydrochloride monohydrate']
'ziprasidone': 'ziprasidone', #['ziprasidone  hydrochloride', 'ziprasidone hydrochloride monohydrate']
'39923': 'zm 39923', #['zm 39923 hcl', 'zm 39923 hydrochloride']
'zomepirac': 'zomepirac', #['zomepirac sodium', 'zomepirac sodium salt']
'zonisamide': 'zonisamide', #['zonisamide', 'zonisamide sodium']
'zuclopenthixol': 'zuclopenthixol', #['zuclopenthixol dihydrochloride', 'zuclopenthixol hydrochloride']
}

Now, for names containing the above words, I replace them with the dict value pointed to by the word.

In [29]:
sbi2drugname_merge2 = defaultdict(set)
for sbis, names in sbi2drugname_merge1.items():
    for name in names:
        sbi2drugname_merge2[sbis].add(name)
        words = [re.sub('[\)\]]$', '', re.sub('^[\(\[]', '', w)) for w in re.split(',? ', name)]
        for word in words:
            if word in word2stdname:
                stdname = word2stdname[word]
                sbi2drugname_merge2[sbis].discard(name)
                sbi2drugname_merge2[sbis].add(stdname)

I'll make another mock df to merge the new synonyms that this creates.

In [30]:
mock_df2_cols = []
for sbis, names in sbi2drugname_merge2.items():
    if set(sbis).intersection('SBI-0206862.P001 SBI-0634390.P001'.split()):
        names.add('arsenic trioxide')
    mock_df2_cols.append(SYN_DELIM.join(list(sbis) + list(names)))
mock_df2 = pd.DataFrame(columns=mock_df2_cols)
mock_df2 = expand_cols_to_synonymns([mock_df2], syn_delim=SYN_DELIM)[0]
len(mock_df2_cols) - mock_df2.shape[1] # number of similar drugs combined based on name

234

Now I need a dictionary whose keys are single SBI IDs rather than tuples of IDs. And again, rather than choose a name, I will keep them all as synonyms.

In [31]:
sbi2drugname_merge3 = {}
for syn_col in mock_df2.columns:
    sbis = [s for s in syn_col.split(SYN_DELIM) if s.startswith('SBI')]
    names = [s for s in syn_col.split(SYN_DELIM) if not s.startswith('SBI')]
    namestring = SYN_DELIM.join(sorted(names))
    for sbi in sbis:
        sbi2drugname_merge3[sbi] = namestring

In [32]:
json.dump(sbi2drugname_merge3, open(SBI_TO_DRUGNAME_MERGED_SIMILAR_FILE, 'w'))

Now I can make another SBI-to-CID dictionary that merges based on the similar names I just merged.

In [33]:
sbi2drugname_merge3 = json.load(open(SBI_TO_DRUGNAME_MERGED_SIMILAR_FILE, 'r'))
from modules.utils import reverse_item2set_dict

In [34]:
drugname2sbis = defaultdict(set)
for sbi, syn_string in sbi2drugname_merge3.items():
    drugname2sbis[syn_string].add(sbi)

In [35]:
sbi2cid = json.load(open(SBI_TO_CID_UNION_FILE, 'r'))
sbi2cid_sim = defaultdict(set)
for sbi, cids in sbi2cid.items():
    sbi2cid_sim[sbi].update(cids)
    if sbi in sbi2drugname_merge3:
        syn_string = sbi2drugname_merge3[sbi]
        sbis = drugname2sbis[syn_string]
        for sbi2 in sbis:
            if sbi2 in sbi2cid:
                sbi2cid_sim[sbi].update(sbi2cid[sbi2])

In [36]:
json.dump({key: sorted(val) for key, val in sbi2cid_sim.items()}, open(SBI_TO_CID_MERGED_SIMILAR_FILE, 'w'))